# Exploring Baroque Ceiling Painting Data in the NFDI4Culture Knowledge Graph

This notebook is a starting point for a data story about baroque art and ceiling paintings using the NFDI4Culture Knowledge Graph.

Focus:
- Work with **data portals** (especially CbDD and the Color Slide Archive of Wall and Ceiling Painting)
- Use **SPARQL** to query the KG
- Prepare results for visualisation (maps, timelines, comparisons)

You can adapt the queries step by step as you learn more about the concrete RDF schema of the datasets.

In [13]:
# Install dependencies (run once per environment)
!pip install SPARQLWrapper pandas matplotlib --quiet

In [50]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

# NFDI4Culture SPARQL endpoint
ENDPOINT_URL = "https://nfdi4culture.de/sparql"

# Prefixes used in queries
# NOTE: The KG uses http://schema.org/ (not https://)
PREFIXES = """\
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nfdicore: <https://nfdi.fiz-karlsruhe.de/ontology/>
PREFIX schema:  <http://schema.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcat:    <http://www.w3.org/ns/dcat#>
PREFIX n4c:     <https://nfdi4culture.de/id/>
PREFIX cto:     <https://nfdi4culture.de/ontology/>
"""

def run_sparql(query: str) -> pd.DataFrame:
    """Run a SPARQL query against the NFDI4Culture endpoint and return a pandas DataFrame.

    The query body should *not* include prefixes, they are automatically prepended.
    This version accesses the JSON result safely to avoid indexing errors in static type checkers.
    """
    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(PREFIXES + "\n" + query)
    results = sparql.query().convert()

    # Be defensive: ensure results is a dict and extract bindings safely
    if not isinstance(results, dict):
        return pd.DataFrame()

    bindings = results.get("results", {}).get("bindings", [])
    rows = []
    for binding in bindings:
        # each binding is a dict of variable -> { "type": ..., "value": ... }
        row = {var: val.get("value") for var, val in binding.items()}
        rows.append(row)
    return pd.DataFrame(rows)

## 1. Inspect the CbDD portal (Corpus of Baroque Ceiling Painting in Germany)

- Portal ID from the registry: `n4c:E4264`
- Goal: See which properties connect the portal to data feeds, homepages, subjects, etc.

Run this once and scan the property list. It tells you which predicates to use in later queries.

In [15]:
query_inspect_cbdd = """\
SELECT ?p ?o
WHERE {
  n4c:E4264 ?p ?o .
}
ORDER BY ?p
LIMIT 200
"""

df_cbdd_props = run_sparql(query_inspect_cbdd)
df_cbdd_props

,p,o
0,http://schema.org/contributor,nodeID://b696016
1,http://schema.org/contributor,nodeID://b696863
2,http://schema.org/contributor,nodeID://b698053
3,http://schema.org/contributor,nodeID://b698416
4,http://schema.org/description,\n The Corpus of Baroque Ceiling Painting i...
5,http://schema.org/hasPart,https://nfdi4culture.de/id/E6077
6,http://schema.org/image,https://nfdi4culture.de//fileadmin/user_upload...
7,http://schema.org/keywords,https://nfdi4culture.de/id/E3953
8,http://schema.org/keywords,https://nfdi4culture.de/id/E3959
9,http://schema.org/keywords,https://nfdi4culture.de/id/E3968


## 2. Discover the CbDD Data Feed

The CbDD portal (`n4c:E4264`) contains a data feed that holds all painting records. 
Let's find the feed and understand how paintings are connected to it.

In [16]:
# Find what points TO the CbDD portal - this reveals the data feed
query_find_feed = """
SELECT ?feed ?feedLabel ?feedType ?predicate
WHERE {
  ?feed ?predicate n4c:E4264 .
  OPTIONAL { ?feed rdfs:label ?feedLabel . }
  OPTIONAL { ?feed rdf:type ?feedType . }
}
LIMIT 20
"""

df_feeds = run_sparql(query_find_feed)
print("Entities pointing to the CbDD portal:")
print(df_feeds)

# The main feed is E6077 - let's verify its structure
print("\n" + "="*60)
print("Verifying E6077 feed structure:")

query_feed_structure = """
SELECT ?p (COUNT(?o) AS ?count) 
WHERE {
  n4c:E6077 ?p ?o .
}
GROUP BY ?p
ORDER BY DESC(?count)
LIMIT 10
"""
df_feed_struct = run_sparql(query_feed_structure)
print(df_feed_struct)

Entities pointing to the CbDD portal:
                                feed                                          feedLabel  \
0   https://nfdi4culture.de/id/E2971                                               JPEG   
1   https://nfdi4culture.de/id/E2971                                               JPEG   
2   https://nfdi4culture.de/id/E3978                                            CC0 1.0   
3   https://nfdi4culture.de/id/E3978                                            CC0 1.0   
4   https://nfdi4culture.de/id/E2312                                       Architecture   
5   https://nfdi4culture.de/id/E2312                                       Architecture   
6   https://nfdi4culture.de/id/E2313                                        Art History   
7   https://nfdi4culture.de/id/E2313                                        Art History   
8   https://nfdi4culture.de/id/E2957                                  Image File Format   
9   https://nfdi4culture.de/id/E3596                

In [17]:
# Define the CbDD feed URI - this is the main entry point for querying paintings
CBDD_FEED_URI = "n4c:E6077"

# Verify the data path: Feed -> DataFeedItem -> Painting
query_verify_path = f"""
SELECT (COUNT(DISTINCT ?painting) AS ?totalPaintings)
WHERE {{
  {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
  ?feedItem schema:item ?painting .
}}
"""
df_verify = run_sparql(query_verify_path)
print(f"✓ CbDD Feed URI: {CBDD_FEED_URI}")
print(f"✓ Total paintings accessible: {df_verify['totalPaintings'].iloc[0]}")
print(f"\nData path: Feed → schema:dataFeedElement → DataFeedItem → schema:item → Painting")  

✓ CbDD Feed URI: n4c:E6077
✓ Total paintings accessible: 6228

Data path: Feed → schema:dataFeedElement → DataFeedItem → schema:item → Painting


## 3. Explore Painting Properties

Now let's discover what properties are available on the painting records.

In [18]:
# Discover all predicates used by paintings in the dataset
query_painting_predicates = f"""
SELECT ?predicate (COUNT(?o) AS ?count) (SAMPLE(?o) AS ?sampleValue)
WHERE {{
  {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
  ?feedItem schema:item ?painting .
  ?painting ?predicate ?o .
}}
GROUP BY ?predicate
ORDER BY DESC(?count)
LIMIT 30
"""

df_painting_preds = run_sparql(query_painting_predicates)

# Add resolved labels using the ontology resolver (defined in cell 13)
# This will be populated after running the ontology resolution cell
def add_resolved_labels(df):
    """Add a 'resolved_label' column with human-readable property names."""
    if 'resolve_property_name' in dir():
        df['resolved_label'] = df['predicate'].apply(resolve_property_name)
    else:
        # Fallback: extract last part of URI
        df['resolved_label'] = df['predicate'].apply(
            lambda x: x.split('/')[-1] if '/' in x else x
        )
    return df

df_painting_preds = add_resolved_labels(df_painting_preds)

print("All predicates used by paintings (with resolved ontology labels):")
print("="*80)
print("\nRun the 'Automatic Ontology Resolution' cell first to get full CTO/NFDI labels.\n")

# Display with resolved labels
df_painting_preds[['resolved_label', 'count', 'predicate', 'sampleValue']]

All predicates used by paintings (with resolved ontology labels):

Run the 'Automatic Ontology Resolution' cell first to get full CTO/NFDI labels.



,resolved_label,count,predicate,sampleValue
0,CTO_0001026,23359,https://nfdi4culture.de/ontology/CTO_0001026,http://vocab.getty.edu/aat/300004792
1,CTO_0001009,6672,https://nfdi4culture.de/ontology/CTO_0001009,nodeID://b2646938
2,CTO_0001025,6230,https://nfdi4culture.de/ontology/CTO_0001025,nodeID://b2640229
3,rdf-schema#label,6228,http://www.w3.org/2000/01/rdf-schema#label,"Hofhegnenberg, Schloss"
4,CTO_0001049,6228,https://nfdi4culture.de/ontology/CTO_0001049,https://nfdi4culture.de/ontology/CTO_0001047
5,NFDI_0001008,6228,https://nfdi.fiz-karlsruhe.de/ontology/NFDI_00...,https://www.deckenmalerei.eu/50c603ef-f42c-43f...
6,CTO_0001006,6228,https://nfdi4culture.de/ontology/CTO_0001006,https://nfdi4culture.de/id/E6077
7,22-rdf-syntax-ns#type,6228,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://nfdi4culture.de/ontology/CTO_0001005
8,NFDI_0000142,6228,https://nfdi.fiz-karlsruhe.de/ontology/NFDI_00...,https://nfdi4culture.de/id/E6404
9,NFDI_0000191,6228,https://nfdi.fiz-karlsruhe.de/ontology/NFDI_00...,https://nfdi4culture.de/id/E2430


In [19]:
# Get a sample of paintings with key properties to understand the data
# Key properties: CTO_0001073 = creation period/year
query_sample_paintings = f"""
SELECT ?painting ?label ?year ?lat ?lon 
WHERE {{
  {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
  ?feedItem schema:item ?painting .
  ?painting rdfs:label ?label .
  OPTIONAL {{ ?painting <https://nfdi4culture.de/ontology/CTO_0001073> ?year . }}
  OPTIONAL {{
    ?painting schema:latitude ?lat .
    ?painting schema:longitude ?lon .
  }}
}}
LIMIT 10
"""

df_sample_paintings = run_sparql(query_sample_paintings)
print(f"Sample paintings ({len(df_sample_paintings)} records):")
print(df_sample_paintings)

# =============================================================================
# Function to get ALL metadata for a specific painting
# Uses the automatic ontology resolver for human-readable property names
# =============================================================================
def get_painting_metadata(painting_uri: str, use_ontology_labels: bool = True) -> pd.DataFrame:
    """
    Retrieve ALL properties (predicates and values) for a specific painting URI.
    This shows the complete metadata stored in the knowledge graph.
    
    Integrates with the CTO/NFDI ontology resolver for human-readable names.
    
    Args:
        painting_uri: The full URI of the painting (e.g., 'https://nfdi4culture.de/id/...')
        use_ontology_labels: If True, use resolved ontology labels (requires cell 13 to be run)
        
    Returns:
        DataFrame with columns: property_name, value, value_type, property
    """
    query = f"""
    SELECT ?property ?value
    WHERE {{
      <{painting_uri}> ?property ?value .
    }}
    ORDER BY ?property
    """
    
    df = run_sparql(query)
    
    if not df.empty:
        # Add a readable property name column using ontology resolver if available
        if use_ontology_labels and 'resolve_property_name' in dir():
            df['property_name'] = df['property'].apply(resolve_property_name)
        else:
            # Fallback: extract last part of URI
            df['property_name'] = df['property'].apply(
                lambda x: x.split('/')[-1] if '/' in x else x
            )
        
        # Detect value type (URI vs literal)
        df['value_type'] = df['value'].apply(
            lambda x: 'URI' if x.startswith('http') else 'Literal'
        )
        # Reorder columns for better readability
        df = df[['property_name', 'value', 'value_type', 'property']]
    
    return df

# Show all metadata for the first painting in our sample
print("\n" + "="*80)
print("📋 COMPLETE METADATA for first painting:")
print("   (Property names resolved via CTO/NFDI ontology when available)")
print("="*80)

if not df_sample_paintings.empty:
    first_painting_uri = df_sample_paintings.iloc[0]['painting']
    first_painting_label = df_sample_paintings.iloc[0]['label']
    print(f"\n🖼️  {first_painting_label}")
    print(f"URI: {first_painting_uri}\n")
    
    df_metadata = get_painting_metadata(first_painting_uri)
    print(f"Found {len(df_metadata)} property values:\n")
    
    # Group by property for cleaner display
    for prop_name in df_metadata['property_name'].unique():
        prop_rows = df_metadata[df_metadata['property_name'] == prop_name]
        values = prop_rows['value'].tolist()
        value_type = prop_rows['value_type'].iloc[0]
        
        if len(values) == 1:
            val_display = values[0][:80] + '...' if len(values[0]) > 80 else values[0]
            print(f"  • {prop_name}: {val_display}")
        else:
            print(f"  • {prop_name}: ({len(values)} values)")
            for v in values[:3]:  # Show first 3 values
                val_display = v[:70] + '...' if len(v) > 70 else v
                print(f"      - {val_display}")
            if len(values) > 3:
                print(f"      ... and {len(values)-3} more")

print("\n✅ Function defined: get_painting_metadata(painting_uri)")
print("   Use it to explore any painting: get_painting_metadata(df_sample_paintings.iloc[N]['painting'])")
print("   Set use_ontology_labels=False to disable ontology resolution")

Sample paintings (10 records):
                                            painting                                            label  \
0  https://www.deckenmalerei.eu/00e1625e-0ac7-423...                        Burggen, Kapelle St. Anna   
1  https://www.deckenmalerei.eu/021afb11-438b-4f7...                       Iffeldorf, Heuwinklkapelle   
2  https://www.deckenmalerei.eu/02f7125d-cfb1-4fa...  Hessental, Hällische Erbschänke, Gasthaus Krone   
3  https://www.deckenmalerei.eu/03414469-1219-4fc...                             Lauchheim, Pfarrhaus   
4  https://www.deckenmalerei.eu/037d1d8a-4487-439...                             Berlin, Stadtschloss   
5  https://www.deckenmalerei.eu/043e1e20-2c95-42b...        Eisenberg, Residenzschloss Christiansburg   
6  https://www.deckenmalerei.eu/0656df8b-2e41-4cc...    Schmidmühlen, Unteres Schloss (Hammerschloss)   
7  https://www.deckenmalerei.eu/0678f9cc-e52d-46e...                           Weimar, Römisches Haus   
8  https://www.deckenmal

### Automatic Ontology Resolution for CTO/NFDI Codes

The painting metadata uses property codes from two namespaces:

1. **CTO (Culture Ontology)**: `https://nfdi4culture.de/ontology/CTO_XXXXXXX`
   - Domain-specific extension for NFDI4Culture cultural heritage data
   - Example: `CTO_0001009` = "has related person", `CTO_0001011` = "has related location"

2. **NFDIcore**: `https://nfdi.fiz-karlsruhe.de/ontology/NFDI_XXXXXXX`
   - Mid-level ontology for all NFDI consortia
   - Example: `NFDI_0001006` = "has external identifier" (links to GND, etc.)

**Automatic Resolution:**

Instead of hardcoding property labels, we dynamically fetch and parse the official ontology files from the GitHub repositories:

- **CTO**: [cto.ttl](https://github.com/ISE-FIZKarlsruhe/nfdi4culture/blob/main/cto.ttl)
- **NFDIcore**: [nfdicore.ttl](https://github.com/ISE-FIZKarlsruhe/nfdicore/blob/main/nfdicore.ttl)

The `rdfs:label` annotations are extracted for each CTO/NFDI entity, providing human-readable names automatically.

In [20]:
# =============================================================================
# Automatic CTO/NFDI Ontology Resolution
# =============================================================================
# Dynamically resolve ontology codes to human-readable labels by parsing
# the official OWL/TTL files from the GitHub repositories.
#
# Sources:
#   - CTO (NFDI4Culture Ontology): https://github.com/ISE-FIZKarlsruhe/nfdi4culture
#   - NFDIcore (Mid-level Ontology): https://github.com/ISE-FIZKarlsruhe/nfdicore
#
# This approach fetches the ontology files once and extracts rdfs:label
# for all CTO_* and NFDI_* entities, avoiding hardcoded mappings.

import requests
from functools import lru_cache
import re

# =============================================================================
# Ontology Sources (Raw TTL files from GitHub)
# =============================================================================
ONTOLOGY_SOURCES = {
    'CTO': {
        'url': 'https://raw.githubusercontent.com/ISE-FIZKarlsruhe/nfdi4culture/main/cto.ttl',
        'namespace': 'https://nfdi4culture.de/ontology/',
        'prefix_pattern': r'CTO_\d+',
    },
    'NFDIcore': {
        'url': 'https://raw.githubusercontent.com/ISE-FIZKarlsruhe/nfdicore/main/nfdicore.ttl',
        'namespace': 'https://nfdi.fiz-karlsruhe.de/ontology/',
        'prefix_pattern': r'NFDI_\d+',
    }
}

# Global cache for resolved ontology labels
_ontology_cache = {}
_ontology_loaded = False

def _parse_ttl_labels(ttl_content: str, namespace: str, prefix_pattern: str) -> dict:
    """
    Parse a TTL file and extract rdfs:label for entities matching the prefix pattern.
    Handles both full URI format and prefix notation (used in nfdicore.ttl).
    
    Args:
        ttl_content: The TTL file content as a string
        namespace: The namespace URI (e.g., 'https://nfdi4culture.de/ontology/')
        prefix_pattern: Regex pattern for codes (e.g., 'CTO_\\d+')
    
    Returns:
        dict mapping code -> label (e.g., 'CTO_0001009' -> 'has related person')
    """
    labels = {}
    
    # Pattern 1: Full URI format - <namespace/CODE> ... rdfs:label "Label"@en .
    entity_pattern = re.compile(
        rf'<{re.escape(namespace)}({prefix_pattern})>\s+[^;]*?'
        rf'rdfs:label\s+"([^"]+)"(?:@en)?\s*[;.]',
        re.MULTILINE | re.DOTALL
    )
    
    for match in entity_pattern.finditer(ttl_content):
        code = match.group(1)
        label = match.group(2)
        labels[code] = label
    
    # Pattern 2: Prefix notation - ontology:NFDI_XXXXXX ... rdfs:label "Label"@en
    # First find the prefix definition
    prefix_match = re.search(r'@prefix\s+(\w+):\s+<' + re.escape(namespace) + r'>\s*\.', ttl_content)
    if prefix_match:
        prefix_name = prefix_match.group(1)
        # Now find entities using that prefix
        prefix_entity_pattern = re.compile(
            rf'{prefix_name}:({prefix_pattern})\s+[^;]*?'
            rf'rdfs:label\s+"([^"]+)"(?:@en)?\s*[;.]',
            re.MULTILINE | re.DOTALL
        )
        for match in prefix_entity_pattern.finditer(ttl_content):
            code = match.group(1)
            label = match.group(2)
            if code not in labels:
                labels[code] = label
    
    # Pattern 3: Multi-line format with entity definition on one line, label on another
    lines = ttl_content.split('\n')
    current_entity = None
    
    for line in lines:
        # Check for full URI entity definition
        entity_match = re.match(rf'^<{re.escape(namespace)}({prefix_pattern})>', line)
        if entity_match:
            current_entity = entity_match.group(1)
        
        # Check for prefix notation entity definition (e.g., "ontology:NFDI_0000004")
        if prefix_match:
            prefix_name = prefix_match.group(1)
            prefix_entity_match = re.match(rf'^{prefix_name}:({prefix_pattern})\s', line)
            if prefix_entity_match:
                current_entity = prefix_entity_match.group(1)
        
        # Check for rdfs:label in the current context
        if current_entity:
            label_match = re.search(r'rdfs:label\s+"([^"]+)"(?:@en)?', line)
            if label_match and current_entity not in labels:
                labels[current_entity] = label_match.group(1)
            
            # Reset current entity on blank line or new entity definition
            if line.strip() == '':
                current_entity = None
    
    return labels

def load_ontology_labels(force_reload: bool = False) -> dict:
    """
    Load and cache all ontology labels from CTO and NFDIcore.
    
    Args:
        force_reload: If True, reload even if already cached
    
    Returns:
        dict mapping code -> {'label': str, 'namespace': str, 'uri': str}
    """
    global _ontology_cache, _ontology_loaded
    
    if _ontology_loaded and not force_reload:
        return _ontology_cache
    
    print("Loading ontology labels from GitHub...")
    
    for source_name, source_info in ONTOLOGY_SOURCES.items():
        try:
            print(f"   Fetching {source_name} from {source_info['url'][:50]}...")
            response = requests.get(source_info['url'], timeout=30)
            response.raise_for_status()
            
            labels = _parse_ttl_labels(
                response.text,
                source_info['namespace'],
                source_info['prefix_pattern']
            )
            
            for code, label in labels.items():
                _ontology_cache[code] = {
                    'label': label,
                    'namespace': source_info['namespace'],
                    'uri': f"{source_info['namespace']}{code}",
                    'source': source_name
                }
            
            print(f"   Loaded {len(labels)} labels from {source_name}")
            
        except Exception as e:
            print(f"   Failed to load {source_name}: {e}")
    
    _ontology_loaded = True
    print(f"\nTotal: {len(_ontology_cache)} ontology codes resolved")
    return _ontology_cache

@lru_cache(maxsize=500)
def resolve_ontology_code(code: str) -> dict:
    """
    Resolve a CTO/NFDI ontology code to its label.
    
    Args:
        code: Ontology code like 'CTO_0001009' or 'NFDI_0001006'
    
    Returns:
        dict with 'code', 'label', 'uri', 'source', 'resolved' keys
    """
    result = {'code': code, 'label': code, 'uri': None, 'source': None, 'resolved': False}
    
    # Ensure ontology is loaded
    if not _ontology_loaded:
        load_ontology_labels()
    
    if code in _ontology_cache:
        cached = _ontology_cache[code]
        result['label'] = cached['label']
        result['uri'] = cached['uri']
        result['source'] = cached['source']
        result['resolved'] = True
    else:
        # Construct URI even if label not found
        if code.startswith('CTO_'):
            result['uri'] = f"https://nfdi4culture.de/ontology/{code}"
            result['source'] = 'CTO'
        elif code.startswith('NFDI_'):
            result['uri'] = f"https://nfdi.fiz-karlsruhe.de/ontology/{code}"
            result['source'] = 'NFDIcore'
    
    return result

def resolve_property_name(property_uri: str) -> str:
    """
    Convert a full property URI to a human-readable label.
    
    Args:
        property_uri: Full URI like 'https://nfdi4culture.de/ontology/CTO_0001009'
    
    Returns:
        Human-readable label like 'has related person (CTO_0001009)'
    """
    # Extract the code from the URI
    code = property_uri.split('/')[-1] if '/' in property_uri else property_uri
    
    # Handle standard vocabularies
    if 'schema.org' in property_uri:
        return code
    if 'w3.org' in property_uri:
        return code.split('#')[-1] if '#' in code else code
    
    # Resolve CTO/NFDI codes
    if code.startswith('CTO_') or code.startswith('NFDI_'):
        resolved = resolve_ontology_code(code)
        if resolved['resolved'] and resolved['label'] != code:
            return f"{resolved['label']} ({code})"
    
    return code

def get_ontology_reference_table() -> pd.DataFrame:
    """
    Get a DataFrame with all resolved ontology codes for reference.
    
    Returns:
        DataFrame with columns: code, label, source, uri
    """
    if not _ontology_loaded:
        load_ontology_labels()
    
    rows = []
    for code, info in sorted(_ontology_cache.items()):
        rows.append({
            'code': code,
            'label': info['label'],
            'source': info['source'],
            'uri': info['uri']
        })
    
    return pd.DataFrame(rows)

# =============================================================================
# Load ontology on first run
# =============================================================================
ontology_labels = load_ontology_labels()

# Display summary
print("\n" + "="*70)
print("CTO/NFDI Ontology Code Reference (Auto-loaded from GitHub)")
print("="*70)

# Show some key properties used in CbDD dataset
key_codes = ['CTO_0001005', 'CTO_0001009', 'CTO_0001010', 'CTO_0001011',
             'CTO_0001019', 'CTO_0001026', 'CTO_0001073', 'CTO_0001021',
             'NFDI_0000004', 'NFDI_0000005', 'NFDI_0000008', 'NFDI_0000015']

print("\nKey properties used in the CbDD ceiling painting dataset:\n")
for code in key_codes:
    resolved = resolve_ontology_code(code)
    status = '[OK]' if resolved['resolved'] else '[??]'
    print(f"  {status} {code:15} -> {resolved['label']}")

print("\n" + "="*70)
print("\nOntology Sources:")
for name, info in ONTOLOGY_SOURCES.items():
    print(f"  - {name}: {info['url']}")

print("\nFunctions defined:")
print("   - resolve_ontology_code(code) -> resolve CTO/NFDI codes to labels")
print("   - resolve_property_name(uri) -> human-readable property names")
print("   - get_ontology_reference_table() -> DataFrame with all codes")
print("   - load_ontology_labels(force_reload=True) -> refresh from GitHub")

Loading ontology labels from GitHub...
   Fetching CTO from https://raw.githubusercontent.com/ISE-FIZKarlsruhe...
   Loaded 70 labels from CTO
   Fetching NFDIcore from https://raw.githubusercontent.com/ISE-FIZKarlsruhe...


Loading ontology labels from GitHub...
   Fetching CTO from https://raw.githubusercontent.com/ISE-FIZKarlsruhe...
   Loaded 70 labels from CTO
   Fetching NFDIcore from https://raw.githubusercontent.com/ISE-FIZKarlsruhe...


   Loaded 197 labels from NFDIcore

Total: 267 ontology codes resolved

CTO/NFDI Ontology Code Reference (Auto-loaded from GitHub)

Key properties used in the CbDD ceiling painting dataset:

  [OK] CTO_0001005     -> source item
  [OK] CTO_0001009     -> has related person
  [OK] CTO_0001010     -> has related organization
  [OK] CTO_0001011     -> has related location
  [OK] CTO_0001019     -> has related item
  [OK] CTO_0001026     -> has external classifier
  [OK] CTO_0001073     -> has creation period
  [OK] CTO_0001021     -> has content url
  [OK] NFDI_0000004    -> person
  [OK] NFDI_0000005    -> place
  [OK] NFDI_0000008    -> creative work
  [OK] NFDI_0000015    -> identifier


Ontology Sources:
  - CTO: https://raw.githubusercontent.com/ISE-FIZKarlsruhe/nfdi4culture/main/cto.ttl
  - NFDIcore: https://raw.githubusercontent.com/ISE-FIZKarlsruhe/nfdicore/main/nfdicore.ttl

Functions defined:
   - resolve_ontology_code(code) -> resolve CTO/NFDI codes to labels
   - resolve_prope

In [21]:
# =============================================================================
# Subject Resolution via External SPARQL Endpoints
# =============================================================================
# Resolves subject URIs from CTO_0001026 ("has external classifier") to labels
# using the official ICONCLASS and Getty AAT SPARQL endpoints.
#
# Integrates with the CTO/NFDI ontology resolver for consistent
# property name resolution throughout the notebook.

import requests
import time
from functools import lru_cache
import urllib.parse

@lru_cache(maxsize=500)
def query_iconclass_sparql(notation):
    """Query ICONCLASS SPARQL endpoint for a label."""
    try:
        # URL-decode the notation (e.g., "48C14%28SCHEINARCHITEKTUR%29" -> "48C14(SCHEINARCHITEKTUR)")
        notation_decoded = urllib.parse.unquote(notation)
        
        endpoint = "https://iconclass.org/sparql"
        query = f"""
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        
        SELECT ?label
        WHERE {{
          <https://iconclass.org/{notation_decoded}> skos:prefLabel ?label .
          FILTER(LANG(?label) = "en")
        }}
        LIMIT 1
        """.strip()  # IMPORTANT: strip whitespace!
        
        resp = requests.get(
            endpoint,
            params={'query': query, 'format': 'json'},
            headers={'Accept': 'application/sparql-results+json'},
            timeout=10
        )
        if resp.ok:
            data = resp.json()
            bindings = data.get("results", {}).get("bindings", [])
            if bindings:
                return bindings[0].get("label", {}).get("value")
    except Exception as e:
        pass
    return None

@lru_cache(maxsize=500)
def query_getty_sparql(aat_id):
    """Query Getty AAT SPARQL endpoint for a label using gvp:prefLabelGVP."""
    try:
        endpoint = "http://vocab.getty.edu/sparql"
        # Getty uses gvp:prefLabelGVP/xl:literalForm for preferred labels
        # IMPORTANT: Must strip whitespace - Getty returns empty response if query has leading whitespace!
        query = f"""
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX xl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX aat: <http://vocab.getty.edu/aat/>

SELECT ?label
WHERE {{
  aat:{aat_id} gvp:prefLabelGVP/xl:literalForm ?label .
}}
LIMIT 1
""".strip()
        
        resp = requests.get(
            endpoint,
            params={'query': query, 'format': 'json'},
            headers={'Accept': 'application/sparql-results+json'},
            timeout=10
        )
        if resp.ok and resp.text:  # Also check response is not empty
            data = resp.json()
            bindings = data.get("results", {}).get("bindings", [])
            if bindings:
                return bindings[0].get("label", {}).get("value")
    except Exception as e:
        pass
    return None

def resolve_subject_from_sparql(uri):
    """
    Resolve a subject URI to its label using external SPARQL endpoints.
    
    Handles subjects from CTO_0001026 ("has external classifier"):
    - ICONCLASS: iconographic classification for art
    - Getty AAT: Art & Architecture Thesaurus
    
    Args:
        uri: Subject URI (e.g., 'https://iconclass.org/92D1521' or 'http://vocab.getty.edu/aat/300004792')
    
    Returns:
        dict with 'uri', 'code', 'label', 'source', 'resolved' keys
    """
    code = uri.split('/')[-1]
    
    if 'iconclass.org' in uri:
        label = query_iconclass_sparql(code)
        source = 'ICONCLASS'
    elif 'vocab.getty.edu' in uri:
        label = query_getty_sparql(code)
        source = 'Getty AAT'
    else:
        label = None
        source = 'Unknown'
    
    return {
        'uri': uri,
        'code': code,
        'label': label or f'[{code}]',
        'source': source,
        'resolved': label is not None
    }

# Test with sample codes
print("Testing external SPARQL endpoints for subject resolution...")
print("="*70)
print(f"\nSubjects come from CTO_0001026", end="")
if 'resolve_ontology_code' in dir():
    resolved = resolve_ontology_code('CTO_0001026')
    print(f" ({resolved['label']})")
else:
    print(" (has external classifier)")

print("\n1. ICONCLASS tests:")
for code in ["92D1521", "25HH", "5"]:
    label = query_iconclass_sparql(code)
    print(f"   {code}: {label}")

print("\n2. Getty AAT tests (using gvp:prefLabelGVP/xl:literalForm):")
for code in ["300004792", "300411453"]:
    label = query_getty_sparql(code)
    print(f"   {code}: {label}")

print("\n" + "="*70)
print("✅ Functions defined:")
print("   - resolve_subject_from_sparql(uri) -> resolve ICONCLASS/AAT URIs to labels")
print("   - query_iconclass_sparql(notation) -> query ICONCLASS endpoint")
print("   - query_getty_sparql(aat_id) -> query Getty AAT endpoint")
print("\nThese integrate with CTO_0001026 ('has external classifier') property.")

Testing external SPARQL endpoints for subject resolution...

Subjects come from CTO_0001026 (has external classifier)

1. ICONCLASS tests:
   92D1521: Cupid shooting a dart
   25HH: landscapes - HH - ideal landscapes
   5: Abstract Ideas and Concepts

2. Getty AAT tests (using gvp:prefLabelGVP/xl:literalForm):
   300004792: buildings (structures)
   300411453: ceiling paintings

✅ Functions defined:
   - resolve_subject_from_sparql(uri) -> resolve ICONCLASS/AAT URIs to labels
   - query_iconclass_sparql(notation) -> query ICONCLASS endpoint
   - query_getty_sparql(aat_id) -> query Getty AAT endpoint

These integrate with CTO_0001026 ('has external classifier') property.


### CbDD Graph Data Integration

The CbDD (Corpus of Baroque Ceiling Painting in Germany) provides a pre-exported graph dataset (`graphData.json`) with rich relational data that complements the NFDI4Culture Knowledge Graph.

**Data available from the CbDD graph:**
- **Painters** (link type: `PAINTERS`) - directly named, no GND resolution needed
- **Commissioners** (link type: `COMMISSIONERS`) - patrons who commissioned the work
- **Rooms** (link type: `PART` to `OBJECT_ROOM`) - where the painting is located
- **Buildings** (link type: `PART` to `OBJECT_BUILDING`) - the church/palace containing the room
- **Dates** (link type: `DATE`) - creation dates
- **Architects, Plasterers, etc.** - other related persons

**Matching strategy:** Paintings are matched by their `rdfs:label` from NFDI4Culture to the `name` field in the CbDD graph.

This approach is more reliable than GND resolution because:
1. Names are pre-resolved and curated in the CbDD database
2. Role classification (painter vs commissioner) is explicit in the graph structure
3. No external API calls needed, making it faster and more robust

In [22]:
# =============================================================================
# CbDD Graph Data Loader (Enhanced)
# =============================================================================
# Loads the pre-exported CbDD graph data (graphData.json) and provides
# comprehensive functions to enrich painting data with:
#   - People: painters, commissioners, architects, plasterers, sculptors, etc.
#   - Locations: room → building → state hierarchy
#   - Building metadata: function, architects, commissioners
#   - Relationships: painter networks, template providers
#
# Link types extracted: PAINTERS, COMMISSIONERS, ARCHITECTS, PLASTERERS,
#   SCULPTORS, DESIGNERS, TEMPLATE_PROVIDERS, BUILDERS, FUNCTION, LOCATION,
#   DATE, METHOD, MATERIAL, PART, and more.

import json
import os
from typing import Optional, Dict, List, Any
from collections import defaultdict

# =============================================================================
# Load and Parse CbDD Graph
# =============================================================================
CBDD_GRAPH_PATH = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'graphData.json')

# Global cache for the graph data and indices
_cbdd_graph = None
_cbdd_nodes_by_id = None
_cbdd_nodes_by_name = None
_cbdd_paintings_by_name = None
_cbdd_links_by_source = None
_cbdd_links_by_target = None
_cbdd_buildings_by_name = None
_cbdd_painter_to_paintings = None
_cbdd_graph_loaded = False

def load_cbdd_graph(force_reload: bool = False) -> dict:
    """
    Load the CbDD graph data from graphData.json and build lookup indices.
    
    Returns:
        dict with 'nodes', 'links', 'exportDate' and lookup indices
    """
    global _cbdd_graph, _cbdd_nodes_by_id, _cbdd_nodes_by_name
    global _cbdd_paintings_by_name, _cbdd_links_by_source, _cbdd_links_by_target
    global _cbdd_buildings_by_name, _cbdd_painter_to_paintings, _cbdd_graph_loaded
    
    if _cbdd_graph_loaded and not force_reload:
        return _cbdd_graph
    
    print("📥 Loading CbDD graph data from graphData.json...")
    
    try:
        with open('graphData.json', encoding='utf-8') as f:
            _cbdd_graph = json.load(f)
        
        # Build lookup indices for fast access
        _cbdd_nodes_by_id = {n['id']: n for n in _cbdd_graph['nodes']}
        
        # Build name lookup (case-insensitive, normalized)
        _cbdd_nodes_by_name = {}
        for n in _cbdd_graph['nodes']:
            name = n.get('name', '').strip()
            if name:
                key = name.lower()
                if key not in _cbdd_nodes_by_name:
                    _cbdd_nodes_by_name[key] = []
                _cbdd_nodes_by_name[key].append(n)
        
        # Build painting-specific lookup by exact name
        _cbdd_paintings_by_name = {}
        paintings = [n for n in _cbdd_graph['nodes'] if n.get('type') == 'OBJECT_PAINTING']
        for p in paintings:
            name = p.get('name', '').strip()
            if name:
                if name not in _cbdd_paintings_by_name:
                    _cbdd_paintings_by_name[name] = []
                _cbdd_paintings_by_name[name].append(p)
        
        # Build building lookup by name
        _cbdd_buildings_by_name = {}
        buildings = [n for n in _cbdd_graph['nodes'] if n.get('type') == 'OBJECT_BUILDING']
        for b in buildings:
            name = b.get('name', '').strip()
            if name:
                _cbdd_buildings_by_name[name] = b
        
        # Build links index by source AND target for fast lookup
        _cbdd_links_by_source = {}
        _cbdd_links_by_target = {}
        for link in _cbdd_graph['links']:
            src, tgt = link['source'], link['target']
            if src not in _cbdd_links_by_source:
                _cbdd_links_by_source[src] = []
            _cbdd_links_by_source[src].append(link)
            if tgt not in _cbdd_links_by_target:
                _cbdd_links_by_target[tgt] = []
            _cbdd_links_by_target[tgt].append(link)
        
        # Build painter -> paintings index for network analysis
        _cbdd_painter_to_paintings = defaultdict(list)
        for link in _cbdd_graph['links']:
            if link['type'] == 'PAINTERS':
                painter_id = link['target']
                painting_id = link['source']
                painter = _cbdd_nodes_by_id.get(painter_id)
                painting = _cbdd_nodes_by_id.get(painting_id)
                if painter and painting:
                    _cbdd_painter_to_paintings[painter.get('name', '')].append({
                        'id': painting_id,
                        'name': painting.get('name', '')
                    })
        
        _cbdd_graph_loaded = True
        
        # Statistics
        node_types = {}
        link_types = {}
        for n in _cbdd_graph['nodes']:
            t = n.get('type', 'UNKNOWN')
            node_types[t] = node_types.get(t, 0) + 1
        for l in _cbdd_graph['links']:
            t = l['type']
            link_types[t] = link_types.get(t, 0) + 1
        
        print(f"   ✓ Loaded {len(_cbdd_graph['nodes']):,} nodes, {len(_cbdd_graph['links']):,} links")
        print(f"   ✓ Export date: {_cbdd_graph.get('exportDate', 'unknown')}")
        print(f"\n   Node types:")
        for t, count in sorted(node_types.items(), key=lambda x: -x[1])[:8]:
            print(f"      {t}: {count:,}")
        print(f"\n   Key link types:")
        for t in ['PAINTERS', 'COMMISSIONERS', 'ARCHITECTS', 'FUNCTION', 'LOCATION', 'PART', 'TEMPLATE_PROVIDERS']:
            print(f"      {t}: {link_types.get(t, 0):,}")
        print(f"\n   ✓ Indices built: {len(_cbdd_paintings_by_name):,} paintings, {len(_cbdd_buildings_by_name):,} buildings")
        print(f"   ✓ Painter network: {len(_cbdd_painter_to_paintings):,} painters tracked")
        
        return _cbdd_graph
        
    except FileNotFoundError:
        print("   ⚠ graphData.json not found! Download it from the CbDD portal.")
        _cbdd_graph_loaded = False
        return None
    except Exception as e:
        print(f"   ⚠ Error loading graph: {e}")
        _cbdd_graph_loaded = False
        return None


def get_painting_from_graph(painting_name: str) -> Optional[Dict]:
    """Find a painting in the CbDD graph by its name."""
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    if not _cbdd_paintings_by_name:
        return None
    
    name = painting_name.strip()
    if name in _cbdd_paintings_by_name:
        return _cbdd_paintings_by_name[name][0]
    
    # Try case-insensitive match
    name_lower = name.lower()
    for key, paintings in _cbdd_paintings_by_name.items():
        if key.lower() == name_lower:
            return paintings[0]
    
    return None


def get_building_info(building_id: str) -> Dict[str, Any]:
    """
    Extract comprehensive information about a building from the CbDD graph.
    
    Args:
        building_id: The UUID of the building in the CbDD graph
    
    Returns:
        dict with building details: name, function, location, architects, commissioners, etc.
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    result = {
        'building_name': None,
        'building_id': building_id,
        'function': None,
        'location_state': None,
        'architects': [],
        'building_commissioners': [],
        'builders': [],
        'sculptors': [],
        'owners': [],
        'construction_date': None,
        'ensemble': None,
        'ensemble_id': None,
    }
    
    building = _cbdd_nodes_by_id.get(building_id)
    if not building:
        return result
    
    result['building_name'] = building.get('name')
    
    # Get all outgoing links from this building
    links = _cbdd_links_by_source.get(building_id, [])
    
    for link in links:
        target = _cbdd_nodes_by_id.get(link['target'])
        if not target:
            continue
        
        link_type = link['type']
        target_name = target.get('name', '')
        
        if link_type == 'FUNCTION':
            # Clean up function name (e.g., "Funktion: Kirche -> Abteikirche" -> "Abteikirche")
            func = target_name
            if func.startswith('Funktion: '):
                func = func[10:]
            if ' -> ' in func:
                func = func.split(' -> ')[-1]  # Take most specific function
            result['function'] = func
        elif link_type == 'LOCATION':
            result['location_state'] = target_name
        elif link_type == 'ARCHITECTS':
            result['architects'].append(target_name)
        elif link_type == 'COMMISSIONERS':
            result['building_commissioners'].append(target_name)
        elif link_type == 'SCULPTORS':
            result['sculptors'].append(target_name)
        elif link_type == 'OWNERS':
            result['owners'].append(target_name)
        elif link_type == 'BUILDERS':
            result['builders'].append(target_name)
        elif link_type == 'DATE':
            # Get construction date
            if not result['construction_date']:
                result['construction_date'] = target_name
    
    # Check if building is part of an ensemble
    parent_links = _cbdd_links_by_target.get(building_id, [])
    for link in parent_links:
        if link['type'] == 'PART':
            parent = _cbdd_nodes_by_id.get(link['source'])
            if parent and parent.get('type') == 'OBJECT_ENSEMBLE':
                result['ensemble'] = parent.get('name')
                result['ensemble_id'] = parent['id']
                break
    
    return result


def get_room_info(room_id: str) -> Dict[str, Any]:
    """
    Extract information about a room from the CbDD graph.
    
    Args:
        room_id: The UUID of the room in the CbDD graph
    
    Returns:
        dict with room details: name, function, architects, commissioners, etc.
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    result = {
        'room_name': None,
        'room_id': room_id,
        'function': None,
        'architects': [],
        'commissioners': [],
        'plasterers': [],
        'painters': [],
        'sculptors': [],
        'designers': [],
    }
    
    room = _cbdd_nodes_by_id.get(room_id)
    if not room:
        return result
    
    result['room_name'] = room.get('name')
    
    # Get all outgoing links from this room
    links = _cbdd_links_by_source.get(room_id, [])
    
    for link in links:
        target = _cbdd_nodes_by_id.get(link['target'])
        if not target:
            continue
        
        link_type = link['type']
        target_name = target.get('name', '')
        
        if link_type == 'FUNCTION':
            func = target_name
            if func.startswith('Funktion: '):
                func = func[10:]
            if ' -> ' in func:
                func = func.split(' -> ')[-1]
            result['function'] = func
        elif link_type == 'ARCHITECTS':
            result['architects'].append(target_name)
        elif link_type == 'COMMISSIONERS':
            result['commissioners'].append(target_name)
        elif link_type == 'PLASTERERS':
            result['plasterers'].append(target_name)
        elif link_type == 'PAINTERS':
            result['painters'].append(target_name)
        elif link_type == 'SCULPTORS':
            result['sculptors'].append(target_name)
        elif link_type == 'DESIGNERS':
            result['designers'].append(target_name)
    
    return result


def get_painting_relations(painting_id: str) -> Dict[str, Any]:
    """
    Extract ALL relationships for a painting from the CbDD graph.
    
    Args:
        painting_id: The UUID of the painting in the CbDD graph
    
    Returns:
        dict with comprehensive relationship data
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    result = {
        # People (from painting directly)
        'painters': [],
        'commissioners': [],
        'architects': [],
        'plasterers': [],
        'sculptors': [],
        'designers': [],
        'template_providers': [],
        'other_artists': [],
        'reference_persons': [],
        'donors': [],
        # Room data
        'room': None,
        'room_id': None,
        'room_function': None,
        'room_architects': [],
        'room_commissioners': [],
        'room_plasterers': [],
        'room_painters': [],
        # Building data
        'building': None,
        'building_id': None,
        'building_function': None,
        'location_state': None,
        'building_architects': [],
        'building_commissioners': [],
        'building_builders': [],
        'building_sculptors': [],
        'building_owners': [],
        'building_date': None,
        # Ensemble data
        'ensemble': None,
        'ensemble_id': None,
        # Artwork metadata
        'date': None,
        'method': None,
        'material': None,
    }
    
    if not _cbdd_links_by_source or not _cbdd_nodes_by_id:
        return result
    
    # Get all links FROM this painting (outgoing)
    links = _cbdd_links_by_source.get(painting_id, [])
    
    for link in links:
        target = _cbdd_nodes_by_id.get(link['target'])
        if not target:
            continue
        
        link_type = link['type']
        target_name = target.get('name', '')
        
        # People relationships
        if link_type == 'PAINTERS':
            result['painters'].append(target_name)
        elif link_type == 'COMMISSIONERS':
            result['commissioners'].append(target_name)
        elif link_type == 'ARCHITECTS':
            result['architects'].append(target_name)
        elif link_type == 'PLASTERERS':
            result['plasterers'].append(target_name)
        elif link_type == 'SCULPTORS':
            result['sculptors'].append(target_name)
        elif link_type == 'DESIGNERS':
            result['designers'].append(target_name)
        elif link_type == 'TEMPLATE_PROVIDERS':
            result['template_providers'].append(target_name)
        elif link_type in ('ARTISTS', 'IMAGE_CARVERS', 'CABINETMAKERS', 'CARPENTERS'):
            result['other_artists'].append(target_name)
        elif link_type == 'REFERENCE_PERSONS':
            result['reference_persons'].append(target_name)
        elif link_type == 'DONORS':
            result['donors'].append(target_name)
        # Metadata
        elif link_type == 'DATE':
            result['date'] = target_name
        elif link_type == 'METHOD':
            method = target_name
            if method.startswith('Technik: '):
                method = method[9:]
            result['method'] = method
        elif link_type == 'MATERIAL':
            result['material'] = target_name
    
    # Find room/building via PART links by traversing ALL the way up the hierarchy
    # PART links go from PARENT → CHILD (source → target)
    # Hierarchy can be: PAINTING -> ROOM -> ROOM -> ... -> BUILDING -> ENSEMBLE
    # We need to traverse until we find OBJECT_BUILDING
    
    def traverse_to_building(node_id: str, depth: int = 0, max_depth: int = 10) -> Optional[Dict]:
        """Recursively traverse up the PART hierarchy to find the building."""
        if depth >= max_depth:
            return None
        
        part_links = _cbdd_links_by_target.get(node_id, [])
        for link in part_links:
            if link['type'] != 'PART':
                continue
            
            parent = _cbdd_nodes_by_id.get(link['source'])
            if not parent:
                continue
            
            parent_type = parent.get('type', '')
            
            if parent_type == 'OBJECT_BUILDING':
                return parent
            elif parent_type in ('OBJECT_ROOM', 'OBJECT_ENSEMBLE'):
                # Continue traversing up
                found = traverse_to_building(parent['id'], depth + 1, max_depth)
                if found:
                    return found
        
        return None
    
    # Get immediate parent (room) first
    part_links = _cbdd_links_by_target.get(painting_id, [])
    
    for link in part_links:
        if link['type'] != 'PART':
            continue
        
        parent = _cbdd_nodes_by_id.get(link['source'])
        if not parent:
            continue
        
        parent_type = parent.get('type', '')
        
        if parent_type == 'OBJECT_ROOM':
            result['room'] = parent.get('name')
            result['room_id'] = parent['id']
            
            # Get room info (function, architects, etc.)
            room_info = get_room_info(parent['id'])
            result['room_function'] = room_info.get('function')
            result['room_architects'] = room_info.get('architects', [])
            result['room_commissioners'] = room_info.get('commissioners', [])
            result['room_plasterers'] = room_info.get('plasterers', [])
            result['room_painters'] = room_info.get('painters', [])
            
            # Traverse ALL the way up to find the building
            building = traverse_to_building(parent['id'])
            if building:
                result['building'] = building.get('name')
                result['building_id'] = building['id']
                
                # Get building info
                building_info = get_building_info(building['id'])
                result['building_function'] = building_info.get('function')
                result['location_state'] = building_info.get('location_state')
                result['building_architects'] = building_info.get('architects', [])
                result['building_commissioners'] = building_info.get('building_commissioners', [])
                result['building_builders'] = building_info.get('builders', [])
                result['building_sculptors'] = building_info.get('sculptors', [])
                result['building_owners'] = building_info.get('owners', [])
                result['building_date'] = building_info.get('construction_date')
                result['ensemble'] = building_info.get('ensemble')
                result['ensemble_id'] = building_info.get('ensemble_id')
            break
        
        elif parent_type == 'OBJECT_BUILDING':
            # Painting directly in building (no room)
            result['building'] = parent.get('name')
            result['building_id'] = parent['id']
            building_info = get_building_info(parent['id'])
            result['building_function'] = building_info.get('function')
            result['location_state'] = building_info.get('location_state')
            result['building_architects'] = building_info.get('architects', [])
            result['building_commissioners'] = building_info.get('building_commissioners', [])
            result['building_builders'] = building_info.get('builders', [])
            result['building_sculptors'] = building_info.get('sculptors', [])
            result['building_owners'] = building_info.get('owners', [])
            result['building_date'] = building_info.get('construction_date')
            result['ensemble'] = building_info.get('ensemble')
            result['ensemble_id'] = building_info.get('ensemble_id')
            break
    
    return result


def enrich_painting_from_graph(painting_name: str) -> Optional[Dict[str, Any]]:
    """
    Get all enrichment data for a painting by its name.
    
    This is the main function to use for enriching NFDI4Culture data with CbDD graph data.
    
    Args:
        painting_name: The painting label (rdfs:label from NFDI4Culture)
    
    Returns:
        dict with all available data, or None if painting not found in graph
    """
    painting = get_painting_from_graph(painting_name)
    if not painting:
        return None
    
    relations = get_painting_relations(painting['id'])
    
    return {
        'cbdd_id': painting['id'],
        'cbdd_name': painting.get('name'),
        **relations
    }


def get_painter_network(painter_name: str) -> Dict[str, Any]:
    """
    Get network information for a painter: their paintings and co-painters.
    
    Args:
        painter_name: Name of the painter
        
    Returns:
        dict with paintings list, co_painters, building_count, etc.
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    result = {
        'painter_name': painter_name,
        'painting_count': 0,
        'paintings': [],
        'co_painters': {},
        'buildings_worked_in': set(),
        'commissioners_worked_for': set(),
    }
    
    paintings = _cbdd_painter_to_paintings.get(painter_name, [])
    result['painting_count'] = len(paintings)
    result['paintings'] = paintings[:20]  # Limit for display
    
    # Find co-painters and other info
    for painting_info in paintings:
        painting_id = painting_info['id']
        
        # Get other painters on same painting
        links = _cbdd_links_by_source.get(painting_id, [])
        for link in links:
            if link['type'] == 'PAINTERS':
                other_painter = _cbdd_nodes_by_id.get(link['target'])
                if other_painter:
                    other_name = other_painter.get('name', '')
                    if other_name != painter_name:
                        result['co_painters'][other_name] = result['co_painters'].get(other_name, 0) + 1
            elif link['type'] == 'COMMISSIONERS':
                commissioner = _cbdd_nodes_by_id.get(link['target'])
                if commissioner:
                    result['commissioners_worked_for'].add(commissioner.get('name', ''))
        
        # Get building
        relations = get_painting_relations(painting_id)
        if relations.get('building'):
            result['buildings_worked_in'].add(relations['building'])
    
    # Convert sets to sorted lists
    result['buildings_worked_in'] = sorted(result['buildings_worked_in'])
    result['commissioners_worked_for'] = sorted(result['commissioners_worked_for'])
    result['co_painters'] = dict(sorted(result['co_painters'].items(), key=lambda x: -x[1]))
    
    return result


def enrich_dataframe_from_graph(df: pd.DataFrame, name_column: str = 'label') -> pd.DataFrame:
    """
    Enrich a DataFrame of paintings with comprehensive data from the CbDD graph.
    
    Extracts ALL available data from graphData.json including:
    - People: painters, commissioners, architects, plasterers, sculptors, designers, 
              template_providers, reference_persons, donors
    - Room: room name, function, architects, commissioners, plasterers, painters
    - Building: name, function, architects, commissioners, builders, sculptors, owners, date
    - Location: state (Bundesland), ensemble
    - Artwork: date, method, material
    
    Args:
        df: DataFrame with painting data (must have a name/label column)
        name_column: Name of the column containing painting names
    
    Returns:
        DataFrame with additional columns from CbDD graph
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    if df.empty:
        return df
    
    # Initialize ALL new columns
    enrichment_cols = [
        # People from painting
        'painters', 'commissioners', 'architects', 'plasterers', 'sculptors',
        'designers', 'template_providers', 'other_artists', 'reference_persons', 'donors',
        # Room data
        'room', 'room_function', 'room_architects', 'room_commissioners', 
        'room_plasterers', 'room_painters',
        # Building data
        'building', 'building_address', 'building_function', 'building_architects',
        'building_commissioners', 'building_builders', 'building_sculptors', 
        'building_owners', 'building_date',
        # Location
        'location_state', 'ensemble',
        # Artwork metadata
        'date_cbdd', 'method', 'material',
        # ID
        'cbdd_id'
    ]
    for col in enrichment_cols:
        if col not in df.columns:
            df[col] = None
    
    matched = 0
    for idx, row in df.iterrows():
        name = row.get(name_column)
        if not name:
            continue
        
        enrichment = enrich_painting_from_graph(name)
        if enrichment:
            matched += 1
            
            # ID
            df.at[idx, 'cbdd_id'] = enrichment.get('cbdd_id')
            
            # People from painting
            # NOTE: Use ' | ' as separator to avoid splitting German names like 'Harms, Johann Oswald'
            df.at[idx, 'painters'] = ' | '.join(enrichment.get('painters', [])) or None
            df.at[idx, 'commissioners'] = ' | '.join(enrichment.get('commissioners', [])) or None
            df.at[idx, 'architects'] = ' | '.join(enrichment.get('architects', [])) or None
            df.at[idx, 'plasterers'] = ' | '.join(enrichment.get('plasterers', [])) or None
            df.at[idx, 'sculptors'] = ' | '.join(enrichment.get('sculptors', [])) or None
            df.at[idx, 'designers'] = ' | '.join(enrichment.get('designers', [])) or None
            df.at[idx, 'template_providers'] = ' | '.join(enrichment.get('template_providers', [])) or None
            df.at[idx, 'other_artists'] = ' | '.join(enrichment.get('other_artists', [])) or None
            df.at[idx, 'reference_persons'] = ' | '.join(enrichment.get('reference_persons', [])) or None
            df.at[idx, 'donors'] = ' | '.join(enrichment.get('donors', [])) or None
            
            # Room data
            df.at[idx, 'room'] = enrichment.get('room')
            df.at[idx, 'room_architects'] = ' | '.join(enrichment.get('room_architects', [])) or None
            df.at[idx, 'room_commissioners'] = ' | '.join(enrichment.get('room_commissioners', [])) or None
            df.at[idx, 'room_plasterers'] = ' | '.join(enrichment.get('room_plasterers', [])) or None
            df.at[idx, 'room_painters'] = ' | '.join(enrichment.get('room_painters', [])) or None
            df.at[idx, 'room_painters'] = ', '.join(enrichment.get('room_painters', [])) or None
            
            # Building data
            df.at[idx, 'building'] = enrichment.get('building')
            df.at[idx, 'building_address'] = enrichment.get('building')  # Building name IS the address
            df.at[idx, 'building_architects'] = ' | '.join(enrichment.get('building_architects', [])) or None
            df.at[idx, 'building_commissioners'] = ' | '.join(enrichment.get('building_commissioners', [])) or None
            df.at[idx, 'building_builders'] = ' | '.join(enrichment.get('building_builders', [])) or None
            df.at[idx, 'building_sculptors'] = ' | '.join(enrichment.get('building_sculptors', [])) or None
            df.at[idx, 'building_owners'] = ' | '.join(enrichment.get('building_owners', [])) or None
            df.at[idx, 'building_owners'] = ', '.join(enrichment.get('building_owners', [])) or None
            df.at[idx, 'building_date'] = enrichment.get('building_date')
            
            # Location
            df.at[idx, 'location_state'] = enrichment.get('location_state')
            df.at[idx, 'ensemble'] = enrichment.get('ensemble')
            
            # Artwork metadata
            df.at[idx, 'date_cbdd'] = enrichment.get('date')
            df.at[idx, 'method'] = enrichment.get('method')
            df.at[idx, 'material'] = enrichment.get('material')
    
    print(f"   ✓ Matched {matched}/{len(df)} paintings ({100*matched/len(df):.1f}%) with CbDD graph")
    return df


def get_top_painters(limit: int = 20) -> List[Dict]:
    """Get list of most prolific painters with their painting counts."""
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    painters = [(name, len(paintings)) for name, paintings in _cbdd_painter_to_paintings.items()]
    painters.sort(key=lambda x: -x[1])
    
    return [{'name': name, 'count': count} for name, count in painters[:limit]]


# =============================================================================
# Load the graph on first run
# =============================================================================
cbdd_graph = load_cbdd_graph()

# Test with a sample painting name
print("\n" + "="*70)
print("Testing enhanced CbDD graph lookup:")
print("="*70)

test_names = ["Spes", "Der Goldene Saal", "Mannheim, Kurfürstliches Residenzschloss"]
for name in test_names:
    result = enrich_painting_from_graph(name)
    if result:
        print(f"\n✓ '{name}':")
        if result.get('painters'):
            print(f"   🎨 Painters: {', '.join(result['painters'][:3])}")
        if result.get('commissioners'):
            print(f"   👤 Commissioners: {', '.join(result['commissioners'][:2])}")
        if result.get('template_providers'):
            print(f"   📐 Template providers: {', '.join(result['template_providers'][:2])}")
        if result.get('room'):
            print(f"   🚪 Room: {result['room']}")
        if result.get('building'):
            print(f"   🏛️ Building: {result['building']}")
        if result.get('building_function'):
            print(f"   ⛪ Function: {result['building_function']}")
        if result.get('location_state'):
            print(f"   📍 State: {result['location_state']}")
        if result.get('building_architects'):
            print(f"   🏗️ Building architects: {', '.join(result['building_architects'][:2])}")
    else:
        print(f"\n✗ '{name}': Not found in graph")

# Show top painters
print("\n" + "="*70)
print("Top 10 most prolific painters in CbDD:")
print("="*70)
for p in get_top_painters(10):
    print(f"   🎨 {p['name']}: {p['count']} paintings")

print("\n" + "="*70)
print("✅ Enhanced CbDD Graph functions defined:")
print("   - load_cbdd_graph() -> load/reload the graph data")
print("   - get_painting_from_graph(name) -> find painting by name")
print("   - get_painting_relations(id) -> get all relations for a painting")
print("   - get_building_info(id) -> get building details (function, architects)")
print("   - enrich_painting_from_graph(name) -> get enrichment data by name")
print("   - enrich_dataframe_from_graph(df) -> enrich a whole DataFrame")
print("   - get_painter_network(name) -> painter's works and collaborators")
print("   - get_top_painters(limit) -> most prolific painters")

📥 Loading CbDD graph data from graphData.json...
   ✓ Loaded 13,835 nodes, 60,150 links
   ✓ Export date: 2025-12-01

   Node types:
      OBJECT_PAINTING: 5,839
      ACTOR_PERSON: 2,772
      OBJECT_ROOM: 2,376
      OBJECT_BUILDING: 1,260
      TEXT: 1,230
      FUNCTION: 200
      ACTOR_SOCIETY: 59
      OBJECT_ENSEMBLE: 32

   Key link types:
      PAINTERS: 7,051
      COMMISSIONERS: 11,160
      ARCHITECTS: 1,743
      FUNCTION: 2,910
      LOCATION: 1,308
      PART: 8,276
      TEMPLATE_PROVIDERS: 1,646

   ✓ Indices built: 5,109 paintings, 1,260 buildings
   ✓ Painter network: 553 painters tracked

Testing enhanced CbDD graph lookup:

✓ 'Spes':
   🎨 Painters: Messmer, Johann Georg
   👤 Commissioners: Stadion, Maria Maximiliana von
   🚪 Room: Die Stiftskirche
   🏛️ Building: Bad Buchau, Stiftskirche
   ⛪ Function: Klosterkirche
   📍 State: Baden-Württemberg
   🏗️ Building architects: D'Ixnard, Pierre Michel

✗ 'Der Goldene Saal': Not found in graph

✗ 'Mannheim, Kurfürstliches

In [23]:
# =============================================================================
# Building Coordinates Lookup from NFDI4Culture KG
# =============================================================================
# The CbDD graph provides building names/addresses but not coordinates.
# We query the NFDI4Culture KG to get lat/lon for buildings.
#
# Building names in CbDD follow patterns like:
#   - "Bad Buchau, Stiftskirche" (City, Building)
#   - "Altenburg, Haus Moritzstraße 6" (City, Street Address)
#   - "München, Schloss Nymphenburg, Hauptschloss" (City, Complex, Building)
#
# Strategy:
#   1. Extract city name from building address (first part before comma)
#   2. Search KG for items containing city name with coordinates
#   3. Match against building name parts
#   4. Cache results for efficiency

from functools import lru_cache
import re

# Cache for building coordinates
_building_coordinates_cache = {}


def extract_address_parts(building_name: str) -> Dict[str, str]:
    """
    Extract city, street, and building parts from a CbDD building name.
    
    Examples:
        "Bad Buchau, Stiftskirche" -> {city: "Bad Buchau", building: "Stiftskirche"}
        "Altenburg, Haus Moritzstraße 6" -> {city: "Altenburg", street: "Moritzstraße 6", building: "Haus"}
        "München, Schloss Nymphenburg, Hauptschloss" -> {city: "München", complex: "Schloss Nymphenburg", building: "Hauptschloss"}
    """
    if not building_name:
        return {}
    
    parts = [p.strip() for p in building_name.split(',')]
    result = {
        'full_name': building_name,
        'city': parts[0] if parts else None,
        'building': parts[-1] if len(parts) > 1 else None,
        'complex': parts[1] if len(parts) > 2 else None,
    }
    
    # Check for street address patterns (contains numbers or street keywords)
    street_patterns = ['straße', 'str.', 'gasse', 'platz', 'weg', 'allee']
    for part in parts[1:]:
        part_lower = part.lower()
        if any(p in part_lower for p in street_patterns) or re.search(r'\d+', part):
            result['street'] = part
            break
    
    return result


@lru_cache(maxsize=500)
def get_building_coordinates_from_kg(building_name: str) -> Optional[Dict]:
    """
    Query NFDI4Culture KG to find coordinates for a building.
    Uses multiple search strategies for better matching.
    
    Args:
        building_name: Building name/address from CbDD (e.g., "Bad Buchau, Stiftskirche")
    
    Returns:
        dict with lat, lon, uri, matched_label, or None if not found
    """
    if not building_name:
        return None
    
    # Extract address components
    addr = extract_address_parts(building_name)
    city = addr.get('city', '')
    
    if not city:
        return None
    
    # Try multiple search strategies
    search_terms = [
        city,  # Just the city
        f"{city}, {addr.get('building', '')}" if addr.get('building') else None,
        addr.get('complex', '') if addr.get('complex') else None,
    ]
    search_terms = [t for t in search_terms if t]
    
    for search_term in search_terms:
        # Clean the search term for SPARQL
        search_clean = search_term.replace('"', '\\"').replace("'", "\\'")
        
        query = f"""
        SELECT ?building ?label ?lat ?lon
        WHERE {{
          {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
          ?feedItem schema:item ?painting .
          
          # Find parent items via CTO_0001019 (is part of) - follow up to 5 levels
          ?painting <https://nfdi4culture.de/ontology/CTO_0001019>* ?building .
          ?building rdfs:label ?label .
          ?building schema:latitude ?lat .
          ?building schema:longitude ?lon .
          
          FILTER(CONTAINS(LCASE(?label), LCASE("{search_clean}")))
        }}
        LIMIT 10
        """
        
        try:
            df = run_sparql(query)
            if not df.empty:
                # Find best match - prefer exact matches
                building_lower = building_name.lower()
                
                for idx, row in df.iterrows():
                    label = row.get('label', '')
                    label_lower = label.lower()
                    
                    # Exact match
                    if building_lower == label_lower:
                        return {
                            'lat': float(row['lat']),
                            'lon': float(row['lon']),
                            'uri': row['building'],
                            'matched_label': label,
                            'match_type': 'exact'
                        }
                
                # Partial match - building name in label or vice versa
                for idx, row in df.iterrows():
                    label = row.get('label', '')
                    label_lower = label.lower()
                    
                    if building_lower in label_lower or label_lower in building_lower:
                        return {
                            'lat': float(row['lat']),
                            'lon': float(row['lon']),
                            'uri': row['building'],
                            'matched_label': label,
                            'match_type': 'partial'
                        }
                
                # City match - if city matches, use it as fallback
                for idx, row in df.iterrows():
                    label = row.get('label', '')
                    if city.lower() in label.lower():
                        return {
                            'lat': float(row['lat']),
                            'lon': float(row['lon']),
                            'uri': row['building'],
                            'matched_label': label,
                            'match_type': 'city'
                        }
                
                # Last resort: return first result
                row = df.iloc[0]
                return {
                    'lat': float(row['lat']),
                    'lon': float(row['lon']),
                    'uri': row['building'],
                    'matched_label': row.get('label', ''),
                    'match_type': 'first'
                }
        except Exception as e:
            pass
    
    return None


def get_coordinates_for_painting(painting_row: pd.Series) -> Dict:
    """
    Get coordinates for a painting, trying multiple sources:
    1. Direct coordinates on painting (from NFDI4Culture)
    2. Building coordinates (via CbDD building -> KG lookup)
    
    Args:
        painting_row: DataFrame row with painting data
    
    Returns:
        dict with lat, lon, source, building_name (if from building)
    """
    # First check if painting has direct coordinates
    lat = painting_row.get('lat')
    lon = painting_row.get('lon')
    
    if lat is not None and lon is not None and str(lat) != 'nan' and str(lon) != 'nan':
        try:
            return {
                'lat': float(lat),
                'lon': float(lon),
                'source': 'painting',
                'building_name': None
            }
        except (ValueError, TypeError):
            pass
    
    # Try to get coordinates from building
    building_name = painting_row.get('building')
    if building_name:
        coords = get_building_coordinates_from_kg(building_name)
        if coords:
            return {
                'lat': coords['lat'],
                'lon': coords['lon'],
                'source': 'building',
                'building_name': building_name,
                'matched_label': coords.get('matched_label'),
                'match_type': coords.get('match_type')
            }
    
    return {'lat': None, 'lon': None, 'source': None, 'building_name': None}


def enrich_dataframe_with_coordinates(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    """
    Add coordinate columns to a DataFrame, trying painting then building coordinates.
    
    Args:
        df: DataFrame with painting data (should have 'building' column from CbDD enrichment)
        verbose: Print progress information
    
    Returns:
        DataFrame with added/updated lat, lon, coord_source columns
    """
    if 'coord_source' not in df.columns:
        df['coord_source'] = None
    
    direct_coords = 0
    building_coords = 0
    no_coords = 0
    
    for idx, row in df.iterrows():
        coords = get_coordinates_for_painting(row)
        
        if coords['lat'] is not None:
            df.at[idx, 'lat'] = coords['lat']
            df.at[idx, 'lon'] = coords['lon']
            df.at[idx, 'coord_source'] = coords['source']
            
            if coords['source'] == 'painting':
                direct_coords += 1
            else:
                building_coords += 1
        else:
            no_coords += 1
    
    if verbose:
        print(f"   ✓ Coordinates enrichment:")
        print(f"      Direct (painting): {direct_coords}")
        print(f"      From building: {building_coords}")
        print(f"      No coordinates: {no_coords}")
    
    return df


# =============================================================================
# Alternative: Query KG for all buildings with coordinates at once
# =============================================================================
def load_all_building_coordinates() -> Dict[str, Dict]:
    """
    Pre-load coordinates for all buildings in the CbDD dataset.
    This is more efficient than individual queries.
    
    Returns:
        dict mapping building_name -> {lat, lon, uri}
    """
    global _building_coordinates_cache
    
    if _building_coordinates_cache:
        return _building_coordinates_cache
    
    print("📍 Loading building coordinates from NFDI4Culture KG...")
    
    # Query for all items with coordinates
    query = f"""
    SELECT DISTINCT ?item ?label ?lat ?lon ?itemType
    WHERE {{
      {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
      ?feedItem schema:item ?painting .
      
      # Get paintings and their parent items
      {{
        ?painting schema:latitude ?lat .
        ?painting schema:longitude ?lon .
        ?painting rdfs:label ?label .
        BIND(?painting AS ?item)
        BIND("painting" AS ?itemType)
      }}
      UNION
      {{
        ?painting <https://nfdi4culture.de/ontology/CTO_0001019> ?parent .
        ?parent schema:latitude ?lat .
        ?parent schema:longitude ?lon .
        ?parent rdfs:label ?label .
        BIND(?parent AS ?item)
        BIND("parent" AS ?itemType)
      }}
    }}
    """
    
    try:
        df = run_sparql(query)
        
        if not df.empty:
            for idx, row in df.iterrows():
                label = row.get('label', '')
                if label:
                    _building_coordinates_cache[label] = {
                        'lat': float(row['lat']),
                        'lon': float(row['lon']),
                        'uri': row['item'],
                        'type': row.get('itemType', 'unknown')
                    }
            
            print(f"   ✓ Loaded coordinates for {len(_building_coordinates_cache)} items")
        else:
            print("   ⚠ No coordinate data found")
            
    except Exception as e:
        print(f"   ⚠ Error loading coordinates: {e}")
    
    return _building_coordinates_cache


def get_cached_coordinates(name: str) -> Optional[Dict]:
    """Get coordinates from cache by exact or partial name match."""
    if not _building_coordinates_cache:
        load_all_building_coordinates()
    
    # Try exact match
    if name in _building_coordinates_cache:
        return _building_coordinates_cache[name]
    
    # Try partial match
    name_lower = name.lower()
    for cached_name, coords in _building_coordinates_cache.items():
        if name_lower in cached_name.lower() or cached_name.lower() in name_lower:
            return coords
    
    return None


print("✅ Building coordinates functions defined:")
print("   - get_building_coordinates_from_kg(building_name) -> query KG for single building")
print("   - get_coordinates_for_painting(row) -> get coords from painting or building")
print("   - enrich_dataframe_with_coordinates(df) -> add coords to DataFrame")
print("   - load_all_building_coordinates() -> pre-load all coords for efficiency")
print("   - get_cached_coordinates(name) -> lookup from cache")

✅ Building coordinates functions defined:
   - get_building_coordinates_from_kg(building_name) -> query KG for single building
   - get_coordinates_for_painting(row) -> get coords from painting or building
   - enrich_dataframe_with_coordinates(df) -> add coords to DataFrame
   - load_all_building_coordinates() -> pre-load all coords for efficiency
   - get_cached_coordinates(name) -> lookup from cache


In [24]:
# =============================================================================
# GND Resolution (Optional - for additional research)
# =============================================================================
# NOTE: For painter/commissioner names, we now use the CbDD Graph (graphData.json)
# which provides direct names without needing GND resolution.
#
# These GND functions are kept for optional research purposes:
#   - Looking up additional person details
#   - Resolving GND URIs found in other contexts
#   - Cross-referencing with the German National Library

import requests
from functools import lru_cache

@lru_cache(maxsize=1000)
def resolve_gnd_uri(gnd_uri: str) -> dict:
    """
    Resolve a GND URI to its preferred name using lobid.org API.
    
    NOTE: For painter/commissioner names, prefer using the CbDD graph
    via enrich_painting_from_graph() which is faster and more reliable.
    
    Args:
        gnd_uri: A GND URI like 'https://d-nb.info/gnd/118636960'
        
    Returns:
        dict with 'name', 'type', 'uri', 'resolved' keys
    """
    result = {'uri': gnd_uri, 'name': None, 'type': None, 'resolved': False}
    
    if not gnd_uri or not isinstance(gnd_uri, str):
        return result
    
    try:
        gnd_id = gnd_uri.split('/')[-1].strip()
        if not gnd_id or len(gnd_id) < 3:
            return result
        
        response = requests.get(
            f'https://lobid.org/gnd/{gnd_id}.json',
            headers={'Accept': 'application/json'},
            timeout=10
        )
        
        if response.ok:
            data = response.json()
            result['name'] = data.get('preferredName')
            type_val = data.get('type', [])
            if isinstance(type_val, list) and type_val:
                result['type'] = type_val[0]
            elif isinstance(type_val, str):
                result['type'] = type_val
            else:
                result['type'] = 'Unknown'
            result['resolved'] = result['name'] is not None
            
    except Exception as e:
        pass
    
    return result


print("✅ GND resolution functions defined (optional, for research):")
print("   - resolve_gnd_uri(gnd_uri) -> resolve single GND URI via lobid.org")
print()
print("📌 NOTE: For painter/commissioner names in this dataset, use:")
print("   - enrich_painting_from_graph(painting_name)")
print("   - enrich_dataframe_from_graph(df)")
print("   These use the CbDD graph data which is faster and more reliable.")

✅ GND resolution functions defined (optional, for research):
   - resolve_gnd_uri(gnd_uri) -> resolve single GND URI via lobid.org

📌 NOTE: For painter/commissioner names in this dataset, use:
   - enrich_painting_from_graph(painting_name)
   - enrich_dataframe_from_graph(df)
   These use the CbDD graph data which is faster and more reliable.


In [25]:
# =============================================================================
# Fetch Paintings from NFDI4Culture Knowledge Graph
# =============================================================================
# This query fetches the core data from NFDI4Culture:
#   - Painting URI, label, year, coordinates, image URL
#   - ICONCLASS/AAT subjects (for thematic analysis)
#   - Parent entity (part-of relationships)
#
# Person data (painters, commissioners) and location details are enriched
# from the CbDD graph (graphData.json) in the next step.

query_paintings = f"""
SELECT DISTINCT ?painting ?label ?year ?lat ?lon ?imageUrl ?license
       (GROUP_CONCAT(DISTINCT ?iconclass; separator="|") AS ?subjects)
       ?parentUri ?parentLabel
WHERE {{
  {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
  ?feedItem schema:item ?painting .
  
  # Required: Title and image
  ?painting rdfs:label ?label .
  ?painting schema:associatedMedia ?image .
  ?image <https://nfdi4culture.de/ontology/CTO_0001021> ?imageUrl .
  
  # Optional properties from NFDI4Culture
  OPTIONAL {{ ?image <https://nfdi4culture.de/ontology/CTO_0001007> ?license . }}
  OPTIONAL {{ ?painting <https://nfdi4culture.de/ontology/CTO_0001073> ?year . }}
  OPTIONAL {{
    ?painting schema:latitude ?lat .
    ?painting schema:longitude ?lon .
  }}
  OPTIONAL {{ ?painting <https://nfdi4culture.de/ontology/CTO_0001026> ?iconclass . }}
  
  # Part-of relationships (CTO_0001019)
  OPTIONAL {{
    ?painting <https://nfdi4culture.de/ontology/CTO_0001019> ?parentUri .
    FILTER(?parentUri != ?painting)
    ?parentUri rdfs:label ?parentLabel .
  }}
}}
GROUP BY ?painting ?label ?year ?lat ?lon ?imageUrl ?license ?parentUri ?parentLabel
LIMIT 50
"""

df_paintings = run_sparql(query_paintings)

# Ensure optional columns exist
for col in ['parentLabel', 'parentUri', 'subjects', 'lat', 'lon']:
    if col not in df_paintings.columns:
        df_paintings[col] = None

print(f"Fetched {len(df_paintings)} paintings from NFDI4Culture Knowledge Graph")
print(f"  - With coordinates: {len(df_paintings[df_paintings['lat'].notna()])}")
print(f"  - With subjects: {len(df_paintings[df_paintings['subjects'].notna() & (df_paintings['subjects'] != '')])}")
print(f"  - With year: {len(df_paintings[df_paintings['year'].notna()])}")

# Show property references
print("\n📋 SPARQL Properties used (from CTO/NFDI ontology):")
if 'resolve_ontology_code' in dir():
    for code in ['CTO_0001021', 'CTO_0001073', 'CTO_0001026', 'CTO_0001019']:
        resolved = resolve_ontology_code(code)
        print(f"   {code}: {resolved['label']}")

df_paintings[['label', 'year', 'lat', 'lon']].head(10)

Fetched 50 paintings from NFDI4Culture Knowledge Graph
  - With coordinates: 9
  - With subjects: 50
  - With year: 46

📋 SPARQL Properties used (from CTO/NFDI ontology):
   CTO_0001021: has content url
   CTO_0001073: has creation period
   CTO_0001026: has external classifier
   CTO_0001019: has related item


,label,year,lat,lon
0,Die Supraporten,1772-1774,NaN,NaN
1,"Weilburg, Obere Orangerie",1703-05,50.48541893393229,8.261075019836428
2,Die Decke des Schlafzimmers,1690-1700,NaN,NaN
3,Ein Cavalier,1745-1755,NaN,NaN
4,Bärenhüter,1543,NaN,NaN
5,"Klösterl am Walchensee, Hauskapelle St. Anna, ...","1686-1689, 1728",47.58757655367821,11.321486131798883
6,1. Porträt an der Südwand,1703–1705,NaN,NaN
7,Janus,1542,NaN,NaN
8,Die Truchsessen beim Servieren der kaiserliche...,um 1725,NaN,NaN
9,Landschaft mit Siedlung,um 1705,NaN,NaN


In [26]:
# =============================================================================
# Enrich Paintings with CbDD Graph Data (Extended)
# =============================================================================
# Match paintings from NFDI4Culture with the CbDD graph by name and add:
#   - Painters (directly from graph, no GND resolution needed)
#   - Commissioners
#   - Room and Building information
#   - Building Function (e.g., Kloster, Schloss)
#   - Location State (Bundesland)
#   - Building Architects
#   - Template Providers (Vorlagenlieferanten)
#   - Technique/Method

print("Enriching paintings with CbDD graph data (extended)...")
print("="*70)

# Enrich the dataframe with ALL available CbDD data
df_enriched = enrich_dataframe_from_graph(df_paintings.copy(), name_column='label')

# Show results summary with NEW fields
print("\n" + "="*70)
print("📊 Enrichment Summary (Extended):")
print("="*70)

# Count non-null values for ALL enriched columns
enrichment_stats = [
    # People from painting
    ('painters', '🎨 Painters'),
    ('commissioners', '👤 Commissioners'),
    ('template_providers', '📐 Template Providers'),
    ('architects', '🏛️ Painting Architects'),
    ('plasterers', '🧱 Plasterers'),
    ('sculptors', '🗿 Sculptors'),
    ('designers', '✏️ Designers'),
    ('other_artists', '🎭 Other Artists'),
    ('reference_persons', '📚 Reference Persons'),
    ('donors', '💰 Donors'),
    # Room data
    ('room', '🚪 Room'),
    ('room_function', '🚪 Room Function'),
    ('room_architects', '🚪 Room Architects'),
    ('room_commissioners', '🚪 Room Commissioners'),
    ('room_plasterers', '🚪 Room Plasterers'),
    ('room_painters', '🚪 Room Painters'),
    # Building data
    ('building', '🏛️ Building'),
    ('building_function', '⚙️ Building Function'),
    ('location_state', '📍 Location State'),
    ('building_architects', '🏗️ Building Architects'),
    ('building_commissioners', '🏗️ Building Commissioners'),
    ('building_builders', '🔨 Building Builders'),
    ('building_sculptors', '🗿 Building Sculptors'),
    ('building_owners', '👑 Building Owners'),
    ('building_date', '📅 Building Date'),
    ('ensemble', '🏘️ Ensemble'),
    # Artwork metadata
    ('method', '🖌️ Technique'),
    ('material', '🎨 Material'),
    ('date_cbdd', '📅 CbDD Date'),
]

print("\\n   === From Painting ===")
for col, label in enrichment_stats[:10]:
    if col in df_enriched.columns:
        count = df_enriched[col].notna().sum()
        pct = 100 * count / len(df_enriched)
        if count > 0:
            print(f"   {label}: {count}/{len(df_enriched)} ({pct:.1f}%)")

print("\\n   === From Room ===")
for col, label in enrichment_stats[10:16]:
    if col in df_enriched.columns:
        count = df_enriched[col].notna().sum()
        pct = 100 * count / len(df_enriched)
        if count > 0:
            print(f"   {label}: {count}/{len(df_enriched)} ({pct:.1f}%)")

print("\\n   === From Building ===")
for col, label in enrichment_stats[16:27]:
    if col in df_enriched.columns:
        count = df_enriched[col].notna().sum()
        pct = 100 * count / len(df_enriched)
        if count > 0:
            print(f"   {label}: {count}/{len(df_enriched)} ({pct:.1f}%)")

print("\\n   === Artwork Metadata ===")
for col, label in enrichment_stats[27:]:
    if col in df_enriched.columns:
        count = df_enriched[col].notna().sum()
        pct = 100 * count / len(df_enriched)
        if count > 0:
            print(f"   {label}: {count}/{len(df_enriched)} ({pct:.1f}%)")

# Show sample results with ALL new fields
print("\n" + "="*70)
print("SAMPLE ENRICHED DATA (showing new fields):")
print("="*70)

for idx, row in df_enriched[df_enriched['painters'].notna()].head(5).iterrows():
    print(f"\n🖼️  {row['label'][:70]}")
    # People
    if row.get('painters'):
        print(f"   🎨 Painter(s): {row['painters']}")
    if row.get('commissioners'):
        print(f"   👤 Commissioner(s): {row['commissioners']}")
    if row.get('template_providers') and pd.notna(row.get('template_providers')):
        print(f"   📐 Template provider(s): {row['template_providers']}")
    if row.get('plasterers') and pd.notna(row.get('plasterers')):
        print(f"   🧱 Plasterer(s): {row['plasterers']}")
    # Room
    if row.get('room'):
        room_extra = f" ({row['room_function']})" if row.get('room_function') and pd.notna(row.get('room_function')) else ""
        print(f"   🚪 Room: {row['room']}{room_extra}")
    if row.get('room_architects') and pd.notna(row.get('room_architects')):
        print(f"      Room architect(s): {row['room_architects']}")
    # Building
    if row.get('building'):
        func = f" ({row['building_function']})" if row.get('building_function') and pd.notna(row.get('building_function')) else ""
        print(f"   🏛️ Building: {row['building']}{func}")
    if row.get('building_architects') and pd.notna(row.get('building_architects')):
        print(f"   🏗️ Building architect(s): {row['building_architects']}")
    if row.get('building_builders') and pd.notna(row.get('building_builders')):
        print(f"   🔨 Builder(s): {row['building_builders']}")
    if row.get('ensemble') and pd.notna(row.get('ensemble')):
        print(f"   🏘️ Ensemble: {row['ensemble']}")
    if row.get('location_state') and pd.notna(row.get('location_state')):
        print(f"   📍 State: {row['location_state']}")
    # Artwork metadata
    if row.get('method'):
        print(f"   🖌️ Technique: {row['method']}")
    if row.get('material') and pd.notna(row.get('material')):
        print(f"   🎨 Material: {row['material']}")
    if row.get('year'):
        print(f"   📅 Year: {row['year']}")

# Show distribution of building functions
print("\n" + "="*70)
print("Building Function Distribution:")
print("="*70)
if 'building_function' in df_enriched.columns:
    func_counts = df_enriched['building_function'].value_counts().head(10)
    for func, count in func_counts.items():
        print(f"   {func}: {count}")

# Show distribution of states
print("\n" + "="*70)
print("Location State Distribution:")
print("="*70)
if 'location_state' in df_enriched.columns:
    state_counts = df_enriched['location_state'].value_counts().head(10)
    for state, count in state_counts.items():
        print(f"   {state}: {count}")

# Display all columns
print("\n" + "="*70)
print("Available columns in enriched DataFrame:")
print(df_enriched.columns.tolist())

Enriching paintings with CbDD graph data (extended)...
   ✓ Matched 41/50 paintings (82.0%) with CbDD graph

📊 Enrichment Summary (Extended):
\n   === From Painting ===
   🎨 Painters: 22/50 (44.0%)
   👤 Commissioners: 32/50 (64.0%)
   📐 Template Providers: 1/50 (2.0%)
\n   === From Room ===
   🚪 Room: 41/50 (82.0%)
   🚪 Room Architects: 15/50 (30.0%)
   🚪 Room Commissioners: 36/50 (72.0%)
   🚪 Room Plasterers: 13/50 (26.0%)
   🚪 Room Painters: 25/50 (50.0%)
\n   === From Building ===
   🏛️ Building: 41/50 (82.0%)
   📍 Location State: 41/50 (82.0%)
   🏗️ Building Architects: 30/50 (60.0%)
   🏗️ Building Commissioners: 38/50 (76.0%)
   🔨 Building Builders: 4/50 (8.0%)
   🗿 Building Sculptors: 5/50 (10.0%)
   📅 Building Date: 39/50 (78.0%)
   🏘️ Ensemble: 3/50 (6.0%)
   🖌️ Technique: 23/50 (46.0%)
\n   === Artwork Metadata ===
   🎨 Material: 18/50 (36.0%)
   📅 CbDD Date: 40/50 (80.0%)

SAMPLE ENRICHED DATA (showing new fields):

🖼️  Die Supraporten
   🎨 Painter(s): Fischer, Casimirus
   👤

In [27]:
# =============================================================================
# Test Building Hierarchy Traversal and Coordinate Enrichment
# =============================================================================
# This verifies:
# 1. The recursive hierarchy traversal reaches buildings (not stopping at rooms)
# 2. We can get coordinates from building names via the KG

print("Testing Building Hierarchy Traversal...")
print("="*70)

# Count how many have rooms vs buildings
with_room = df_enriched[df_enriched['room'].notna()]
with_building = df_enriched[df_enriched['building'].notna()]

print(f"\n📊 Location Hierarchy Coverage:")
print(f"   Paintings with Room: {len(with_room)}/{len(df_enriched)} ({100*len(with_room)/len(df_enriched):.1f}%)")
print(f"   Paintings with Building: {len(with_building)}/{len(df_enriched)} ({100*len(with_building)/len(df_enriched):.1f}%)")

# Show paintings that have room but NO building (these would be the problematic cases)
room_but_no_building = df_enriched[(df_enriched['room'].notna()) & (df_enriched['building'].isna())]
print(f"\n⚠️ Paintings with Room but NO Building (hierarchy traversal issue): {len(room_but_no_building)}")
if len(room_but_no_building) > 0:
    for idx, row in room_but_no_building.head(5).iterrows():
        print(f"   - {row['label'][:50]} (Room: {row['room']})")

# Now test coordinate enrichment from buildings
print("\n" + "="*70)
print("Testing Coordinate Enrichment from Buildings...")
print("="*70)

# Check initial coordinates vs coordinates after building lookup
initial_coords = df_enriched['lat'].notna().sum()
print(f"\n📍 Initial coordinates (from painting directly): {initial_coords}/{len(df_enriched)}")

# Enrich with building coordinates
df_with_coords = enrich_dataframe_with_coordinates(df_enriched.copy(), verbose=True)

# Compare
final_coords = df_with_coords['lat'].notna().sum()
print(f"\n📍 Final coordinates (after building lookup): {final_coords}/{len(df_with_coords)}")
print(f"   🆕 Additional coordinates from buildings: {final_coords - initial_coords}")

# Show some examples of coordinates from buildings
building_coord_rows = df_with_coords[(df_with_coords['coord_source'] == 'building')]
if len(building_coord_rows) > 0:
    print(f"\n📍 Sample paintings with coordinates from building lookup:")
    for idx, row in building_coord_rows.head(5).iterrows():
        print(f"   - {row['label'][:40]}")
        print(f"     Building: {row['building']}")
        print(f"     Coords: ({row['lat']:.4f}, {row['lon']:.4f})")

# Store for later use
df_enriched = df_with_coords

Testing Building Hierarchy Traversal...

📊 Location Hierarchy Coverage:
   Paintings with Room: 41/50 (82.0%)
   Paintings with Building: 41/50 (82.0%)

⚠️ Paintings with Room but NO Building (hierarchy traversal issue): 0

Testing Coordinate Enrichment from Buildings...

📍 Initial coordinates (from painting directly): 9/50
   ✓ Coordinates enrichment:
      Direct (painting): 9
      From building: 41
      No coordinates: 0

📍 Final coordinates (after building lookup): 50/50
   🆕 Additional coordinates from buildings: 41

📍 Sample paintings with coordinates from building lookup:
   - Die Supraporten
     Building: Balve, Haus Wocklum
     Coords: (51.3415, 7.8871)
   - Die Decke des Schlafzimmers
     Building: Berlin, Schloss Charlottenburg
     Coords: (52.5209, 13.2957)
   - Ein Cavalier
     Building: Heilbad Heiligenstadt, Haus des ehem. Reichshofs
     Coords: (51.3775, 10.1371)
   - Bärenhüter
     Building: Landshut, Stadtresidenz
     Coords: (48.5362, 12.1509)
   - 1. Portr

In [28]:
# =============================================================================
# Enhanced Display Function for Enriched Paintings
# =============================================================================
# Displays paintings with all available metadata from:
#   - NFDI4Culture KG: title, year, coordinates, subjects, image
#   - CbDD Graph: painters, commissioners, architects, room, building, 
#                 function, state, technique, template providers
from IPython.display import HTML, display

def display_painting_card(row, max_width=500, resolve_subjects=True, show_all_details=True):
    """
    Display a painting with complete metadata as an HTML card.
    
    Data sources:
    - NFDI4Culture: title (label), year, coordinates, subjects (ICONCLASS/AAT), image
    - CbDD Graph: painters, commissioners, architects, room, building, function, 
                  location_state, method/technique, template_providers
    
    Args:
        row: DataFrame row or dict with painting data
        max_width: Maximum image width in pixels
        resolve_subjects: Whether to resolve ICONCLASS/AAT URIs to labels
        show_all_details: Whether to show all available metadata
    """
    # Basic info (from NFDI4Culture)
    label = row.get('label', 'Unknown')
    year = row.get('year') or row.get('date_cbdd') or 'Unknown date'
    image_url = row.get('imageUrl', '')
    subjects = row.get('subjects', '')
    lat = row.get('lat')
    lon = row.get('lon')
    painting_uri = row.get('painting', '')
    parent_label = row.get('parentLabel', '')
    coord_source = row.get('coord_source', '')
    
    # Enriched info (from CbDD Graph)
    painters = row.get('painters', '')
    commissioners = row.get('commissioners', '')
    architects = row.get('architects', '')
    plasterers = row.get('plasterers', '')
    template_providers = row.get('template_providers', '')
    room = row.get('room', '')
    building = row.get('building', '')
    building_function = row.get('building_function', '')
    location_state = row.get('location_state', '')
    building_architects = row.get('building_architects', '')
    method = row.get('method', '')
    cbdd_id = row.get('cbdd_id', '')
    
    # Geo enrichment info (if present)
    geo_source = row.get('geo_source', 'original')
    matched_place = row.get('matched_place', '')
    wikidata_place = row.get('wikidata_place', '')
    
    # Build HTML sections
    html_parts = []
    
    # Title
    html_parts.append(f'<h3 style="margin-top: 0; color: #333;">{label}</h3>')
    
    # Year/Date
    html_parts.append(f'<p style="color: #000;"><strong>📅 Date:</strong> {year}</p>')
    
    # Technique (from CbDD)
    if method and pd.notna(method):
        html_parts.append(f'<p style="color: #000;"><strong>🖌️ Technique:</strong> {method}</p>')
    
    # Painters (from CbDD)
    if painters and pd.notna(painters):
        html_parts.append(f'<p style="color: #000;"><strong>🎨 Painter(s):</strong> {painters}</p>')
    
    # Commissioners (from CbDD)
    if commissioners and pd.notna(commissioners):
        html_parts.append(f'<p style="color: #000;"><strong>👤 Commissioner(s):</strong> {commissioners}</p>')
    
    # Template providers (from CbDD)
    if show_all_details and template_providers and pd.notna(template_providers):
        html_parts.append(f'<p style="color: #000;"><strong>📐 Template provider(s):</strong> {template_providers}</p>')
    
    # Architects for this painting (from CbDD)
    if show_all_details and architects and pd.notna(architects):
        html_parts.append(f'<p style="color: #000;"><strong>🏗️ Architect(s):</strong> {architects}</p>')
    
    # Plasterers (from CbDD)
    if show_all_details and plasterers and pd.notna(plasterers):
        html_parts.append(f'<p style="color: #000;"><strong>🧱 Plasterer(s):</strong> {plasterers}</p>')
    
    # Room (from CbDD)
    if room and pd.notna(room):
        html_parts.append(f'<p style="color: #000;"><strong>🚪 Room:</strong> {room}</p>')
    
    # Building with function (from CbDD)
    if building and pd.notna(building):
        building_text = building
        if building_function and pd.notna(building_function):
            building_text += f' <span style="color: #666;">({building_function})</span>'
        html_parts.append(f'<p style="color: #000;"><strong>🏛️ Building:</strong> {building_text}</p>')
    
    # Building architects (from CbDD)
    if show_all_details and building_architects and pd.notna(building_architects):
        html_parts.append(f'<p style="color: #000; margin-left: 20px;"><small>🏗️ Building architect(s): {building_architects}</small></p>')
    
    # Location/State (from CbDD)
    if location_state and pd.notna(location_state):
        html_parts.append(f'<p style="color: #000;"><strong>📍 State:</strong> {location_state}</p>')
    
    # Part-of hierarchy (from NFDI4Culture)
    if parent_label and pd.notna(parent_label):
        html_parts.append(f'<p style="color: #000;"><strong>📦 Part of:</strong> {parent_label}</p>')
    
    # Subjects (ICONCLASS/AAT)
    subject_html = ''
    if subjects and resolve_subjects:
        separator = '|' if '|' in str(subjects) else ','
        subject_list = [s.strip() for s in str(subjects).split(separator) if s.strip()]
        subject_items = []
        for uri in subject_list[:5]:
            try:
                resolved = resolve_subject_from_sparql(uri)
            except NameError:
                code = uri.split('/')[-1]
                resolved = {'label': f'[{code}]', 'source': 'ICONCLASS' if 'iconclass' in uri else 'AAT', 'code': code}
            
            badge_color = '#4CAF50' if 'iconclass' in uri.lower() else '#2196F3'
            subject_items.append(
                f'<span style="background: {badge_color}; color: white; padding: 2px 8px; '
                f'border-radius: 12px; font-size: 12px; margin: 2px; display: inline-block;" '
                f'title="{resolved.get("source", "")}: {resolved.get("code", "")}">{resolved["label"]}</span>'
            )
        if subject_items:
            subject_html = f'''
            <div style="margin: 10px 0;">
                <strong style="color: #000;">Subjects:</strong><br>
                <div style="margin-top: 5px;">{"".join(subject_items)}</div>
            </div>'''
    
    html_parts.append(subject_html)
    
    # Coordinates
    if lat is not None and str(lat) != 'nan' and lat != '':
        try:
            lat_f = float(lat)
            lon_f = float(lon) if lon else 0
            
            coord_badge = ''
            if coord_source == 'building':
                coord_badge = '<span style="background: #FF9800; color: white; padding: 2px 6px; border-radius: 4px; font-size: 11px;">Building</span> '
            elif geo_source == 'wikidata' and matched_place:
                coord_badge = '<span style="background: #9C27B0; color: white; padding: 2px 6px; border-radius: 4px; font-size: 11px;">Wikidata</span> '
            
            html_parts.append(f'<p style="color: #000;">📍 {coord_badge}{lat_f:.4f}, {lon_f:.4f}</p>')
        except (ValueError, TypeError):
            pass
    
    # Data source badges
    source_badges = []
    source_badges.append('<span style="background: #1976D2; color: white; padding: 2px 6px; border-radius: 4px; font-size: 10px;">Getty</span>')
    if cbdd_id:
        source_badges.append('<span style="background: #388E3C; color: white; padding: 2px 6px; border-radius: 4px; font-size: 10px;">ICONCLASS</span>')
    
    html_parts.append(f'<p style="margin-top: 10px;">{" ".join(source_badges)}</p>')
    
    # Link to CbDD
    if painting_uri:
        html_parts.append(f'<p><a href="{painting_uri}" target="_blank" style="color: #0066cc;">🔗 View in NFDI4Culture</a></p>')
    
    # Image
    if image_url:
        html_parts.append(f'''
            <img src="{image_url}" style="max-width: {max_width}px; max-height: 500px; border-radius: 4px;" 
                 onerror="this.onerror=null; this.src=''; this.alt='Image could not be loaded';">
        ''')
    
    # Combine all parts
    html = f"""
    <div style="border: 1px solid #ddd; padding: 15px; margin: 10px 0; border-radius: 8px; background: #fafafa;">
        {''.join(html_parts)}
    </div>
    """
    display(HTML(html))


def display_painter_profile(painter_name: str, show_paintings: int = 5):
    """
    Display a profile card for a painter showing their works and collaborators.
    
    Args:
        painter_name: Name of the painter
        show_paintings: Number of paintings to list
    """
    try:
        network = get_painter_network(painter_name)
    except NameError:
        print(f"⚠ Painter network function not available. Run the CbDD Graph Loader cell first.")
        return
    
    if network['painting_count'] == 0:
        print(f"⚠ No paintings found for '{painter_name}'")
        return
    
    html_parts = []
    
    # Header
    html_parts.append(f'<h3 style="margin-top: 0; color: #333;">🎨 {painter_name}</h3>')
    html_parts.append(f'<p style="color: #000;"><strong>Total paintings:</strong> {network["painting_count"]}</p>')
    
    # Buildings worked in
    if network['buildings_worked_in']:
        bldgs = network['buildings_worked_in'][:5]
        html_parts.append(f'<p style="color: #000;"><strong>🏛️ Buildings worked in:</strong> {", ".join(bldgs)}</p>')
    
    # Co-painters
    if network['co_painters']:
        co_list = [f"{name} ({count})" for name, count in list(network['co_painters'].items())[:5]]
        html_parts.append(f'<p style="color: #000;"><strong>👥 Co-painters:</strong> {", ".join(co_list)}</p>')
    
    # Commissioners worked for
    if network['commissioners_worked_for']:
        comms = list(network['commissioners_worked_for'])[:5]
        html_parts.append(f'<p style="color: #000;"><strong>👤 Commissioners:</strong> {", ".join(comms)}</p>')
    
    # Sample paintings
    if network['paintings']:
        paintings_list = '<ul style="margin: 5px 0; padding-left: 20px;">'
        for p in network['paintings'][:show_paintings]:
            paintings_list += f'<li style="color: #333;">{p["name"][:60]}</li>'
        if len(network['paintings']) > show_paintings:
            paintings_list += f'<li style="color: #666;"><em>...and {len(network["paintings"]) - show_paintings} more</em></li>'
        paintings_list += '</ul>'
        html_parts.append(f'<p style="color: #000;"><strong>🖼️ Sample works:</strong></p>{paintings_list}')
    
    html = f"""
    <div style="border: 2px solid #1976D2; padding: 15px; margin: 10px 0; border-radius: 8px; background: #E3F2FD;">
        {''.join(html_parts)}
    </div>
    """
    display(HTML(html))


# Backward-compatible alias
display_painting_full = display_painting_card

print("✅ Display functions defined:")
print("   - display_painting_card(row) -> show painting with all metadata")
print("   - display_painter_profile(name) -> show painter's works & collaborators")
print("\nData sources integrated:")
print("   📊 NFDI4Culture KG: title, year, coordinates, subjects, image")
print("   📊 CbDD Graph: painters, commissioners, architects, plasterers,")
print("                  template_providers, room, building, building_function,")
print("                  location_state, building_architects, technique")

✅ Display functions defined:
   - display_painting_card(row) -> show painting with all metadata
   - display_painter_profile(name) -> show painter's works & collaborators

Data sources integrated:
   📊 NFDI4Culture KG: title, year, coordinates, subjects, image
   📊 CbDD Graph: painters, commissioners, architects, plasterers,
                  template_providers, room, building, building_function,
                  location_state, building_architects, technique


In [29]:
# =============================================================================
# Coordinate Enrichment via Building Lookup
# =============================================================================
# For paintings without direct coordinates, try to get them from the parent building

print("="*70)
print("📍 COORDINATE ENRICHMENT VIA BUILDING LOOKUP")
print("="*70)

# Count paintings without coordinates
without_coords = df_enriched[(df_enriched['lat'].isna()) | (df_enriched['lat'] == '')]
with_building = without_coords[without_coords['building'].notna()]

print(f"\n📊 Before enrichment:")
print(f"   Paintings without coordinates: {len(without_coords)}/{len(df_enriched)}")
print(f"   Of those, with building info: {len(with_building)}")

# Try to enrich coordinates from buildings
if len(with_building) > 0:
    print(f"\n🔍 Attempting to get coordinates for {len(with_building)} buildings...")
    
    coords_found = 0
    buildings_checked = set()
    
    for idx, row in with_building.iterrows():
        building = row['building']
        if building and building not in buildings_checked:
            buildings_checked.add(building)
            
            # Try to get coordinates from KG
            coords = get_building_coordinates_from_kg(building)
            if coords and coords.get('lat'):
                coords_found += 1
                # Update all paintings in this building
                mask = df_enriched['building'] == building
                df_enriched.loc[mask, 'lat'] = coords['lat']
                df_enriched.loc[mask, 'lon'] = coords['lon']
                df_enriched.loc[mask, 'coord_source'] = 'building'
                print(f"   ✓ {building[:50]}: {coords['lat']:.4f}, {coords['lon']:.4f}")
    
    print(f"\n📊 After enrichment:")
    new_without_coords = df_enriched[(df_enriched['lat'].isna()) | (df_enriched['lat'] == '')]
    print(f"   Buildings with coordinates found: {coords_found}/{len(buildings_checked)}")
    print(f"   Paintings still without coordinates: {len(new_without_coords)}/{len(df_enriched)}")
else:
    print("   All paintings already have coordinates or building info!")

# Show summary by coordinate source
print("\n📊 Coordinate source summary:")
if 'coord_source' in df_enriched.columns:
    coord_sources = df_enriched['coord_source'].value_counts()
    for src, count in coord_sources.items():
        print(f"   From {src}: {count}")
    # Count paintings with original coords (no source set)
    original_coords = len(df_enriched[(df_enriched['lat'].notna()) & (df_enriched['lat'] != '') & (df_enriched['coord_source'].isna())])
    print(f"   From painting (direct): {original_coords}")

📍 COORDINATE ENRICHMENT VIA BUILDING LOOKUP

📊 Before enrichment:
   Paintings without coordinates: 0/50
   Of those, with building info: 0
   All paintings already have coordinates or building info!

📊 Coordinate source summary:
   From building: 41
   From painting: 9
   From painting (direct): 0


In [30]:
# =============================================================================
# Painter Network Analysis
# =============================================================================
# Explore relationships between painters and their works using the CbDD graph

print("="*70)
print("🎨 PAINTER NETWORK ANALYSIS")
print("="*70)

# Show top painters
print("\n📊 Most prolific Baroque ceiling painters in Germany:")
print("-"*70)
top = get_top_painters(15)
for i, p in enumerate(top, 1):
    print(f"  {i:2d}. {p['name']}: {p['count']} paintings")

# Profile a famous painter
print("\n" + "="*70)
print("🎨 PAINTER PROFILE: Cosmas Damian Asam")
print("="*70)
display_painter_profile("Asam, Cosmas Damian")

# Show another painter for comparison
print("\n" + "="*70)
print("🎨 PAINTER PROFILE: Johann Oswald Harms")
print("="*70)
display_painter_profile("Harms, Johann Oswald")

# Find painters from our sample who worked together
print("\n" + "="*70)
print("🤝 CO-PAINTER RELATIONSHIPS IN OUR SAMPLE")
print("="*70)
painters_in_sample = df_enriched[df_enriched['painters'].notna()]['painters'].unique()
co_painter_found = []
for p in painters_in_sample:
    # Handle comma-separated painters
    for painter in str(p).split(','):
        painter = painter.strip()
        network = get_painter_network(painter)
        if network['co_painters']:
            for co_painter, count in list(network['co_painters'].items())[:3]:
                co_painter_found.append((painter, co_painter, count))

if co_painter_found:
    print("\nPainters from our sample who worked with others:")
    seen = set()
    for p1, p2, count in sorted(co_painter_found, key=lambda x: -x[2])[:10]:
        pair = tuple(sorted([p1, p2]))
        if pair not in seen:
            seen.add(pair)
            print(f"  • {p1} worked with {p2} ({count} times)")
else:
    print("  No co-painter relationships found in sample.")

🎨 PAINTER NETWORK ANALYSIS

📊 Most prolific Baroque ceiling painters in Germany:
----------------------------------------------------------------------
   1. Harms, Johann Oswald: 146 paintings
   2. Castelli, Carlo Ludovico: 127 paintings
   3. Asam, Cosmas Damian: 123 paintings
   4. Lammers, Seivert: 107 paintings
   5. Kager, Johann Matthias: 100 paintings
   6. Giusti, Tommaso: 86 paintings
   7. Asam, Hans Georg: 80 paintings
   8. Hermann, Franz Georg: 78 paintings
   9. Asam, Maria Theresia: 77 paintings
  10. Aloisi, Andrea: 68 paintings
  11. Colomba, Luca Antonio: 64 paintings
  12. Marchini, Giovanni Francesco: 58 paintings
  13. Peiker, Hermenegild: 56 paintings
  14. Rode, Bernhard: 55 paintings
  15. Gumpp, Johann Anton: 51 paintings

🎨 PAINTER PROFILE: Cosmas Damian Asam


🎨 PAINTER NETWORK ANALYSIS

📊 Most prolific Baroque ceiling painters in Germany:
----------------------------------------------------------------------
   1. Harms, Johann Oswald: 146 paintings
   2. Castelli, Carlo Ludovico: 127 paintings
   3. Asam, Cosmas Damian: 123 paintings
   4. Lammers, Seivert: 107 paintings
   5. Kager, Johann Matthias: 100 paintings
   6. Giusti, Tommaso: 86 paintings
   7. Asam, Hans Georg: 80 paintings
   8. Hermann, Franz Georg: 78 paintings
   9. Asam, Maria Theresia: 77 paintings
  10. Aloisi, Andrea: 68 paintings
  11. Colomba, Luca Antonio: 64 paintings
  12. Marchini, Giovanni Francesco: 58 paintings
  13. Peiker, Hermenegild: 56 paintings
  14. Rode, Bernhard: 55 paintings
  15. Gumpp, Johann Anton: 51 paintings

🎨 PAINTER PROFILE: Cosmas Damian Asam



🎨 PAINTER PROFILE: Johann Oswald Harms



🤝 CO-PAINTER RELATIONSHIPS IN OUR SAMPLE
  No co-painter relationships found in sample.


In [31]:
# Display paintings with full metadata from both sources
import time  # For rate-limiting API calls

print("Displaying paintings with combined NFDI4Culture + CbDD Graph data:")
print("="*70)
print("  📊 NFDI4Culture: title, year, coordinates, subjects, image URL")
print("  📊 CbDD Graph: painters, commissioners, room, building, technique")
print("  🟢 ICONCLASS | 🔵 Getty AAT subjects")
print("="*70 + "\n")

# Display paintings that have painter info from CbDD
paintings_with_painters = df_enriched[df_enriched['painters'].notna()]
print(f"Found {len(paintings_with_painters)} paintings with painter information.\n")

for idx, row in paintings_with_painters.head(5).iterrows():
    display_painting_card(row)
    time.sleep(0.2)  # Small delay for subject resolution API calls

Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



Displaying paintings with combined NFDI4Culture + CbDD Graph data:
  📊 NFDI4Culture: title, year, coordinates, subjects, image URL
  📊 CbDD Graph: painters, commissioners, room, building, technique
  🟢 ICONCLASS | 🔵 Getty AAT subjects

Found 22 paintings with painter information.



In [32]:
# =============================================================================
# Final Summary: Complete Data Extraction Results
# =============================================================================
# Show the comprehensive enrichment from graphData.json and NFDI4Culture KG

print("="*70)
print("📊 FINAL DATA EXTRACTION SUMMARY")
print("="*70)

print("\n🗂️ DATA SOURCES:")
print("-"*70)
print("  NFDI4Culture KG:")
print("    - Title (rdfs:label)")
print("    - Year/Date")  
print("    - Coordinates (lat/lon)")
print("    - Subjects (ICONCLASS/Getty AAT)")
print("    - Image URL")
print("    - Parent structure")
print()
print("  CbDD graphData.json:")
print("    - Painters (PAINTERS links)")
print("    - Commissioners (COMMISSIONERS links)")
print("    - Architects (ARCHITECTS links)")
print("    - Template Providers (TEMPLATE_PROVIDERS links)")
print("    - Plasterers (PLASTERERS links)")
print("    - Room (via PART hierarchy)")
print("    - Building (via PART hierarchy)")
print("    - Building Function (FUNCTION links)")
print("    - Location State (LOCATION links)")
print("    - Building Architects")
print("    - Technique/Method (METHOD links)")

print("\n📈 ENRICHMENT RESULTS:")
print("-"*70)
total = len(df_enriched)
stats = {
    'Painters': len(df_enriched[df_enriched['painters'].notna()]),
    'Commissioners': len(df_enriched[df_enriched['commissioners'].notna()]),
    'Room': len(df_enriched[df_enriched['room'].notna()]),
    'Building': len(df_enriched[df_enriched['building'].notna()]),
    'Building Function': len(df_enriched[df_enriched['building_function'].notna()]),
    'Location State': len(df_enriched[df_enriched['location_state'].notna()]),
    'Building Architects': len(df_enriched[df_enriched['building_architects'].notna()]),
    'Template Providers': len(df_enriched[df_enriched['template_providers'].notna()]),
    'Technique': len(df_enriched[df_enriched['method'].notna()]),
    'Coordinates (total)': len(df_enriched[(df_enriched['lat'].notna()) & (df_enriched['lat'] != '')]),
    'Coords from Building': len(df_enriched[df_enriched['coord_source'] == 'building']),
}

for field, count in stats.items():
    pct = 100 * count / total
    bar = '█' * int(pct / 5) + '░' * (20 - int(pct / 5))
    print(f"  {field:20s}: {count:3d}/{total} ({pct:5.1f}%) {bar}")

print("\n🌍 GEOGRAPHIC DISTRIBUTION:")
print("-"*70)
state_counts = df_enriched['location_state'].value_counts()
for state, count in state_counts.head(10).items():
    print(f"  {state:30s}: {count} paintings")

print("\n🏛️ BUILDING FUNCTION DISTRIBUTION:")
print("-"*70)
func_counts = df_enriched['building_function'].value_counts()
for func, count in func_counts.head(8).items():
    print(f"  {func:50s}: {count}")

print("\n✅ Data extraction complete!")
print(f"   Total paintings analyzed: {total}")
print(f"   Matched with CbDD graph: {len(df_enriched[df_enriched['cbdd_id'].notna()])}")
print(f"   With full location data: {len(df_enriched[(df_enriched['lat'].notna()) & (df_enriched['lat'] != '')])}")

📊 FINAL DATA EXTRACTION SUMMARY

🗂️ DATA SOURCES:
----------------------------------------------------------------------
  NFDI4Culture KG:
    - Title (rdfs:label)
    - Year/Date
    - Coordinates (lat/lon)
    - Subjects (ICONCLASS/Getty AAT)
    - Image URL
    - Parent structure

  CbDD graphData.json:
    - Painters (PAINTERS links)
    - Commissioners (COMMISSIONERS links)
    - Architects (ARCHITECTS links)
    - Template Providers (TEMPLATE_PROVIDERS links)
    - Plasterers (PLASTERERS links)
    - Room (via PART hierarchy)
    - Building (via PART hierarchy)
    - Building Function (FUNCTION links)
    - Location State (LOCATION links)
    - Building Architects
    - Technique/Method (METHOD links)

📈 ENRICHMENT RESULTS:
----------------------------------------------------------------------
  Painters            :  22/50 ( 44.0%) ████████░░░░░░░░░░░░
  Commissioners       :  32/50 ( 64.0%) ████████████░░░░░░░░
  Room                :  41/50 ( 82.0%) ████████████████░░░░
  Bui

### Data Pipeline Summary

The notebook implements a **dual-source data pipeline** combining:
1. **NFDI4Culture Knowledge Graph** (SPARQL) - structured linked data
2. **CbDD Graph Export** (graphData.json) - rich relational data from the source database

| Step | Source | Data Retrieved |
|------|--------|----------------|
| 0. Ontology Resolution | GitHub (cto.ttl, nfdicore.ttl) | Human-readable labels for 267 CTO/NFDI codes |
| 1. Core Data | NFDI4Culture SPARQL | Title, year, image, coordinates, ICONCLASS/AAT subjects |
| 2. Graph Enrichment | CbDD graphData.json | **Painters**, **commissioners**, room, building, technique |
| 3. Subject Resolution | ICONCLASS/Getty SPARQL | Human-readable subject labels |
| 4. Geo Enrichment | Wikidata SPARQL | Missing coordinates from place names |

**🔄 Why Two Data Sources?**

| Aspect | NFDI4Culture KG | CbDD Graph |
|--------|-----------------|------------|
| Access | SPARQL endpoint | Local JSON file |
| Persons | GND URIs (need resolution) | **Direct names with roles** |
| Locations | GND URIs | Room → Building hierarchy |
| Subjects | ICONCLASS/AAT URIs | N/A |
| Coordinates | Yes (some) | N/A |
| Images | Yes (URLs) | N/A |

The CbDD graph provides **explicit role information** (painter vs commissioner) directly, avoiding the need to:
- Fetch GND URIs and call lobid.org API
- Parse profession keywords to classify persons
- Handle API failures and timeouts

**📋 Schema Reference:**

| Source | Property/Link | Description |
|--------|--------------|-------------|
| NFDI4Culture | `CTO_0001073` | Creation date/year |
| NFDI4Culture | `CTO_0001026` | ICONCLASS/AAT subjects |
| NFDI4Culture | `CTO_0001021` | Image URL |
| CbDD Graph | `PAINTERS` link | Painter names (direct) |
| CbDD Graph | `COMMISSIONERS` link | Commissioner names (direct) |
| CbDD Graph | `PART` link | Room/Building hierarchy |
| CbDD Graph | `METHOD` link | Painting technique |

**🔧 Key Functions:**
- `load_cbdd_graph()` → load graphData.json with indices
- `enrich_painting_from_graph(name)` → get all CbDD data for a painting
- `enrich_dataframe_from_graph(df)` → batch enrich a DataFrame
- `display_painting_card(row)` → rich HTML display with all data

## 4. Compare CbDD and Color Slide Archive of Wall and Ceiling Painting

Portal IDs from the registry:
- CbDD: `n4c:E4264`
- Color Slide Archive: `n4c:E4267`

Goal: Count how many records in the KG come from each of these portals.

We assume a pattern similar to:
- `?item schema:isPartOf ?feed`
- `?feed schema:isPartOf ?portal` or `?feed dcterms:isPartOf ?portal`

You may have to adjust the property in the middle depending on what you see in the inspection of the feed nodes.

In [33]:
query_ceiling_portal_counts = """\
SELECT ?portal ?portalLabel (COUNT(DISTINCT ?item) AS ?records)
WHERE {
  VALUES ?portal { n4c:E4264  n4c:E4267 }

  # feed belongs to one of the two portals
  ?feed ?isPartOfPortal ?portal .
  FILTER(?isPartOfPortal IN (schema:isPartOf, dcterms:isPartOf))

  # items belong to that feed
  ?item schema:isPartOf ?feed .

  ?portal schema:name ?portalLabel .
}
GROUP BY ?portal ?portalLabel
ORDER BY DESC(?records)
"""

df_ceiling_portal_counts = run_sparql(query_ceiling_portal_counts)
df_ceiling_portal_counts

""


In [34]:
# Simple bar chart of records per portal (CbDD vs Color Slide Archive)
if not df_ceiling_portal_counts.empty:
    plt.figure(figsize=(6, 4))
    plt.bar(df_ceiling_portal_counts["portalLabel"], df_ceiling_portal_counts["records"].astype(int))
    plt.xticks(rotation=20, ha="right")
    plt.ylabel("Number of records in KG")
    plt.title("Records from baroque wall & ceiling painting portals")
    plt.tight_layout()
    plt.show()
else:
    print("No results yet. Check if the intermediate predicate (?isPartOfPortal) is correct.")

No results yet. Check if the intermediate predicate (?isPartOfPortal) is correct.


## 5. Parquet Database Export - Full Dataset Pipeline

This section transforms the notebook from a sample analysis to a **complete data extraction pipeline** that:

1. **Fetches ALL paintings** from the NFDI4Culture SPARQL endpoint (~6,228 paintings)
2. **Enriches** with CbDD graph data (painters, commissioners, buildings, etc.)
3. **Resolves** ICONCLASS/Getty AAT subjects to human-readable labels
4. **Normalizes** into a relational database schema
5. **Exports** to Parquet files for future analysis

### Database Schema

**Core Entity Tables:**
| Table | Primary Key | Description |
|-------|-------------|-------------|
| `paintings` | `painting_uri` | Main painting records with SPARQL URIs |
| `persons` | `person_id` | Artists, commissioners, architects |
| `buildings` | `building_id` | Churches, palaces with locations |
| `rooms` | `room_id` | Rooms within buildings |
| `subjects` | `subject_uri` | Resolved ICONCLASS/AAT labels |
| `ensembles` | `ensemble_id` | Building complexes |

**Junction Tables (Many-to-Many):**
| Table | Description |
|-------|-------------|
| `painting_persons` | Role-based person relationships |
| `painting_subjects` | Subject classifications |
| `building_persons` | Building-related roles |
| `room_persons` | Room-related roles |

### Key Fields for SPARQL Mapping
- `painting_uri`: Direct SPARQL URI for query reconstruction
- `cbdd_id`: CbDD graph node UUID
- `parent_uri`: SPARQL `CTO_0001019` relationship
- `subject_uri`: ICONCLASS/AAT URI for external lookups

In [51]:
# =============================================================================
# Parquet Database Configuration and Helpers
# =============================================================================
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import time

# Parquet output configuration
PARQUET_OUTPUT_DIR = os.path.dirname(os.path.abspath('__file__'))
PARQUET_PREFIX = "baroque_"

# Batch sizes and rate limiting
SPARQL_BATCH_SIZE = 500  # Paintings per SPARQL query
SUBJECT_BATCH_SIZE = 50   # Subjects to resolve before progress update
API_DELAY_SECONDS = 0.05  # Delay between external API calls

def get_parquet_path(table_name: str) -> str:
    """Get the full path for a Parquet file."""
    return os.path.join(PARQUET_OUTPUT_DIR, f"{PARQUET_PREFIX}{table_name}.parquet")


def save_parquet_with_metadata(df: pd.DataFrame, table_name: str, metadata: dict = None) -> str:
    """
    Save DataFrame to Parquet with export metadata.
    
    Args:
        df: DataFrame to save
        table_name: Name for the table (without prefix/extension)
        metadata: Optional additional metadata
    
    Returns:
        Path to the saved file
    """
    path = get_parquet_path(table_name)
    
    # Prepare metadata
    if metadata is None:
        metadata = {}
    metadata['export_date'] = datetime.now().isoformat()
    metadata['row_count'] = len(df)
    metadata['columns'] = ','.join(df.columns)
    
    # Save to Parquet (pyarrow handles metadata via schema)
    df.to_parquet(path, index=False, engine='pyarrow')
    
    print(f"   ✓ Saved {table_name}: {len(df):,} rows → {os.path.basename(path)}")
    return path


def load_parquet_table(table_name: str) -> pd.DataFrame:
    """Load a Parquet table by name."""
    path = get_parquet_path(table_name)
    if os.path.exists(path):
        return pd.read_parquet(path)
    else:
        print(f"   ⚠ Table not found: {path}")
        return pd.DataFrame()


def enrich_coordinates(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    """
    Enrich DataFrame with coordinates from buildings.
    
    Fills missing lat/lon using:
    1. CbDD graph building coordinates
    2. NFDI4Culture Knowledge Graph queries (with caching)
    
    Args:
        df: DataFrame with 'building_name' or 'building_id' column
        verbose: Print progress
    
    Returns:
        DataFrame with enriched 'lat' and 'lon' columns
    """
    if 'lat' not in df.columns:
        df['lat'] = None
    if 'lon' not in df.columns:
        df['lon'] = None
    
    # Count initial state
    initial_coords = df['lat'].notna().sum()
    
    if verbose:
        print(f"   Starting with {initial_coords:,}/{len(df):,} coordinates")
    
    # Try to get coordinates from CbDD graph buildings first
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    enriched_count = 0
    
    for idx, row in df.iterrows():
        # Skip if already has coordinates
        if pd.notna(row.get('lat')) and pd.notna(row.get('lon')):
            continue
        
        # Try CbDD building first
        building_id = row.get('building_id')
        if building_id:
            building_node = _cbdd_nodes_by_id.get(building_id)
            if building_node:
                lat = building_node.get('lat') or building_node.get('latitude')
                lon = building_node.get('lon') or building_node.get('longitude')
                if lat and lon:
                    df.at[idx, 'lat'] = float(lat)
                    df.at[idx, 'lon'] = float(lon)
                    enriched_count += 1
                    continue
        
        # Fallback to KG query by building name
        building_name = row.get('building_name')
        if building_name and building_name.strip():
            try:
                coords = get_building_coordinates_from_kg(building_name)
                if coords and coords.get('lat') and coords.get('lon'):
                    df.at[idx, 'lat'] = coords['lat']
                    df.at[idx, 'lon'] = coords['lon']
                    if coords.get('address'):
                        df.at[idx, 'address'] = coords['address']
                    if coords.get('city'):
                        df.at[idx, 'city'] = coords['city']
                    if coords.get('country'):
                        df.at[idx, 'country'] = coords['country']
                    enriched_count += 1
            except Exception:
                pass
    
    final_coords = df['lat'].notna().sum()
    
    if verbose:
        print(f"   Enriched {enriched_count:,} additional coordinates")
        print(f"   Final: {final_coords:,}/{len(df):,} ({100*final_coords/len(df):.1f}%)")
    
    return df


print("="*70)
print("PARQUET DATABASE CONFIGURATION")
print("="*70)
print(f"   Output directory: {PARQUET_OUTPUT_DIR}")
print(f"   File prefix: {PARQUET_PREFIX}")
print(f"   SPARQL batch size: {SPARQL_BATCH_SIZE}")
print(f"\n✅ Helper functions defined:")
print("   - get_parquet_path(table_name) -> file path")
print("   - save_parquet_with_metadata(df, table_name) -> save with metadata")
print("   - load_parquet_table(table_name) -> load from disk")
print("   - enrich_coordinates(df) -> add lat/lon from buildings")

PARQUET DATABASE CONFIGURATION
   Output directory: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories
   File prefix: baroque_
   SPARQL batch size: 500

✅ Helper functions defined:
   - get_parquet_path(table_name) -> file path
   - save_parquet_with_metadata(df, table_name) -> save with metadata
   - load_parquet_table(table_name) -> load from disk
   - enrich_coordinates(df) -> add lat/lon from buildings


In [52]:
# =============================================================================
# Fetch ALL Paintings from SPARQL (Paginated)
# =============================================================================
# Uses OFFSET/LIMIT pagination to fetch all ~6,228 paintings from the 
# NFDI4Culture SPARQL endpoint in batches.

def fetch_all_paintings_sparql(batch_size: int = SPARQL_BATCH_SIZE, 
                               max_paintings: int = None,
                               verbose: bool = True) -> pd.DataFrame:
    """
    Fetch ALL paintings from the NFDI4Culture SPARQL endpoint using pagination.
    
    Args:
        batch_size: Number of paintings per SPARQL query (default: 500)
        max_paintings: Optional limit for testing (None = fetch all)
        verbose: Print progress information
    
    Returns:
        DataFrame with all paintings and their SPARQL properties
    """
    all_dfs = []
    offset = 0
    total_fetched = 0
    
    if verbose:
        print(f"📥 Fetching paintings from SPARQL endpoint...")
        print(f"   Batch size: {batch_size}, Max: {max_paintings or 'unlimited'}")
    
    while True:
        # Build paginated query - preserves painting URI as primary key
        # Enhanced to include GNDs for Bildindex cross-referencing
        query = f"""
SELECT DISTINCT ?painting ?label ?year ?lat ?lon ?imageUrl ?license
       (GROUP_CONCAT(DISTINCT ?subject; separator="|") AS ?subjects)
       ?parentUri ?parentLabel
       (GROUP_CONCAT(DISTINCT ?creatorGnd; separator="|") AS ?creatorGnds)
       (GROUP_CONCAT(DISTINCT ?locationGnd; separator="|") AS ?locationGnds)
WHERE {{
  {CBDD_FEED_URI} schema:dataFeedElement ?feedItem .
  ?feedItem schema:item ?painting .
  ?painting rdfs:label ?label .
  
  # Image URL (required for display)
  ?painting schema:associatedMedia ?image .
  ?image <https://nfdi4culture.de/ontology/CTO_0001021> ?imageUrl .
  
  # Optional fields from NFDI4Culture
  OPTIONAL {{ ?image <https://nfdi4culture.de/ontology/CTO_0001007> ?license . }}
  OPTIONAL {{ ?painting <https://nfdi4culture.de/ontology/CTO_0001073> ?year . }}
  OPTIONAL {{ ?painting schema:latitude ?lat . ?painting schema:longitude ?lon . }}
  OPTIONAL {{ ?painting <https://nfdi4culture.de/ontology/CTO_0001026> ?subject . }}
  OPTIONAL {{
    ?painting <https://nfdi4culture.de/ontology/CTO_0001019> ?parentUri .
    FILTER(?parentUri != ?painting)
    ?parentUri rdfs:label ?parentLabel .
  }}
  
  # Creator/Painter GND (for Bildindex cross-referencing)
  # CORRECT: GND is stored via NFDI_0001006 from nfdi.fiz-karlsruhe.de ontology (not nfdi4culture.de/id)
  OPTIONAL {{
    ?painting <https://nfdi4culture.de/ontology/CTO_0001009> ?creatorNode .
    ?creatorNode <https://nfdi.fiz-karlsruhe.de/ontology/NFDI_0001006> ?creatorGnd .
    FILTER(CONTAINS(STR(?creatorGnd), "d-nb.info/gnd"))
  }}
  
  # Location/Building GND (for Bildindex cross-referencing)
  # CORRECT: GND is stored via NFDI_0001006 from nfdi.fiz-karlsruhe.de ontology (not nfdi4culture.de/id)
  OPTIONAL {{
    ?painting <https://nfdi4culture.de/ontology/CTO_0001011> ?locNode .
    ?locNode <https://nfdi.fiz-karlsruhe.de/ontology/NFDI_0001006> ?locationGnd .
    FILTER(CONTAINS(STR(?locationGnd), "d-nb.info/gnd"))
  }}
}}
GROUP BY ?painting ?label ?year ?lat ?lon ?imageUrl ?license ?parentUri ?parentLabel
ORDER BY ?painting
LIMIT {batch_size}
OFFSET {offset}
"""
        
        df_batch = run_sparql(query)
        
        if df_batch.empty:
            if verbose:
                print(f"   ✓ Completed at offset {offset}")
            break
        
        all_dfs.append(df_batch)
        total_fetched += len(df_batch)
        
        if verbose:
            print(f"   Batch {len(all_dfs):3d}: +{len(df_batch):4d} paintings (total: {total_fetched:,})")
        
        # Check limits
        if max_paintings and total_fetched >= max_paintings:
            if verbose:
                print(f"   ✓ Reached max_paintings limit: {max_paintings}")
            break
        
        if len(df_batch) < batch_size:
            if verbose:
                print(f"   ✓ Last batch (got {len(df_batch)} < {batch_size})")
            break
        
        offset += batch_size
        time.sleep(0.1)  # Be nice to the endpoint
    
    if not all_dfs:
        print("   ⚠ No paintings found!")
        return pd.DataFrame()
    
    # Combine all batches
    df_all = pd.concat(all_dfs, ignore_index=True)
    
    # Ensure optional columns exist
    for col in ['parentLabel', 'parentUri', 'subjects', 'lat', 'lon', 'license', 'creatorGnds', 'locationGnds']:
        if col not in df_all.columns:
            df_all[col] = None
    
    # Convert coordinates to numeric
    for col in ['lat', 'lon']:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')
    
    if verbose:
        print(f"\n📊 SPARQL Fetch Summary:")
        print(f"   Total paintings: {len(df_all):,}")
        print(f"   With coordinates: {df_all['lat'].notna().sum():,} ({100*df_all['lat'].notna().sum()/len(df_all):.1f}%)")
        print(f"   With subjects: {(df_all['subjects'].notna() & (df_all['subjects'] != '')).sum():,}")
        print(f"   With year: {df_all['year'].notna().sum():,}")
        print(f"   Unique painting URIs: {df_all['painting'].nunique():,}")
        # GND statistics for Bildindex integration
        print(f"   With creator GNDs: {(df_all['creatorGnds'].notna() & (df_all['creatorGnds'] != '')).sum():,}")
        print(f"   With location GNDs: {(df_all['locationGnds'].notna() & (df_all['locationGnds'] != '')).sum():,}")
    
    return df_all


print("✅ Paginated SPARQL fetch function defined:")
print("   - fetch_all_paintings_sparql(batch_size, max_paintings) -> DataFrame")
print("\n   Example usage:")
print("   df_all = fetch_all_paintings_sparql()  # Fetch ALL paintings")
print("   df_test = fetch_all_paintings_sparql(max_paintings=100)  # Test with 100")

✅ Paginated SPARQL fetch function defined:
   - fetch_all_paintings_sparql(batch_size, max_paintings) -> DataFrame

   Example usage:
   df_all = fetch_all_paintings_sparql()  # Fetch ALL paintings
   df_test = fetch_all_paintings_sparql(max_paintings=100)  # Test with 100


In [53]:
# =============================================================================
# Batch Subject Resolution for Full Dataset
# =============================================================================
# Efficiently resolves all unique ICONCLASS/Getty AAT subjects and creates
# normalized tables for the relational database.

def batch_resolve_subjects(df: pd.DataFrame, 
                          uri_column: str = 'subject_uris',
                          verbose: bool = True) -> pd.DataFrame:
    """
    Resolve all unique subjects in the DataFrame and add resolved data.
    
    Args:
        df: DataFrame with a column containing subject URIs (list or pipe-separated string)
        uri_column: Name of the column containing subject URIs
        verbose: Print progress information
    
    Returns:
        DataFrame with 'subjects_resolved' column added (list of dicts with uri, label, source)
    """
    if verbose:
        print(f"📚 Resolving subjects from {len(df):,} paintings...")
    
    # Extract all unique subject URIs
    all_subjects = set()
    
    for idx, row in df.iterrows():
        subjects_val = row.get(uri_column, [])
        
        if subjects_val is None:
            continue
        
        # Handle both list and string formats
        if isinstance(subjects_val, list):
            uris = subjects_val
        elif isinstance(subjects_val, str):
            uris = [u.strip() for u in str(subjects_val).split('|') if u.strip()]
        else:
            continue
        
        for uri in uris:
            if uri:
                all_subjects.add(uri)
    
    if verbose:
        print(f"   Found {len(all_subjects):,} unique subjects")
    
    if len(all_subjects) == 0:
        df['subjects_resolved'] = [[] for _ in range(len(df))]
        print("   ⚠ No subjects to resolve")
        return df
    
    # Resolve each unique subject - build lookup dict
    resolved_lookup = {}
    subjects_list = list(all_subjects)
    
    for i, subj_uri in enumerate(subjects_list):
        if verbose and (i + 1) % SUBJECT_BATCH_SIZE == 0:
            print(f"   Resolving: {i+1:,}/{len(subjects_list):,} ({100*(i+1)/len(subjects_list):.1f}%)")
        
        # Determine source and extract code
        code = subj_uri.split('/')[-1]
        label = None
        source = 'UNKNOWN'
        
        if 'iconclass.org' in subj_uri:
            source = 'ICONCLASS'
            try:
                label = query_iconclass_sparql(code)
            except Exception:
                pass
        elif 'vocab.getty.edu' in subj_uri or 'getty.edu' in subj_uri:
            source = 'GETTY_AAT'
            try:
                label = query_getty_sparql(code)
            except Exception:
                pass
        
        resolved_lookup[subj_uri] = {
            'uri': subj_uri,
            'code': code,
            'label': label if label else f'[{code}]',
            'source': source,
        }
        
        time.sleep(API_DELAY_SECONDS)  # Rate limiting
    
    # Add resolved subjects to each row
    def resolve_row_subjects(subjects_val):
        if subjects_val is None:
            return []
        
        if isinstance(subjects_val, list):
            uris = subjects_val
        elif isinstance(subjects_val, str):
            uris = [u.strip() for u in str(subjects_val).split('|') if u.strip()]
        else:
            return []
        
        return [resolved_lookup.get(uri, {'uri': uri, 'label': uri, 'source': 'UNKNOWN'}) for uri in uris if uri]
    
    df['subjects_resolved'] = df[uri_column].apply(resolve_row_subjects)
    
    if verbose:
        resolved_count = sum(1 for v in resolved_lookup.values() if v['label'] and not v['label'].startswith('['))
        print(f"\n   ✓ Resolved {resolved_count:,}/{len(resolved_lookup):,} subject labels ({100*resolved_count/len(resolved_lookup):.1f}%)")
        source_counts = {}
        for v in resolved_lookup.values():
            source_counts[v['source']] = source_counts.get(v['source'], 0) + 1
        for src, cnt in source_counts.items():
            print(f"      {src}: {cnt:,}")
    
    return df


print("✅ Batch subject resolution function defined:")
print("   - batch_resolve_subjects(df, uri_column='subject_uris') -> df with 'subjects_resolved' column")

✅ Batch subject resolution function defined:
   - batch_resolve_subjects(df, uri_column='subject_uris') -> df with 'subjects_resolved' column


In [54]:
# =============================================================================
# Extract Normalized Entity Tables from CbDD Graph
# =============================================================================
# These functions extract standalone entity tables from the CbDD graph,
# preserving all node IDs for relational integrity.

def extract_persons_table() -> pd.DataFrame:
    """
    Extract all unique persons from the CbDD graph.
    
    Returns DataFrame with: person_id, name, person_type, val
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    persons = []
    for node in _cbdd_graph['nodes']:
        if node.get('type') in ('ACTOR_PERSON', 'ACTOR_SOCIETY'):
            persons.append({
                'person_id': node['id'],
                'name': node.get('name', ''),
                'person_type': node.get('type'),
                'val': node.get('val')  # Network centrality value
            })
    
    df = pd.DataFrame(persons)
    print(f"   ✓ Extracted {len(df):,} persons ({(df['person_type'] == 'ACTOR_PERSON').sum():,} individuals, {(df['person_type'] == 'ACTOR_SOCIETY').sum():,} organizations)")
    return df


def extract_buildings_table() -> pd.DataFrame:
    """
    Extract all buildings with their metadata from CbDD graph.
    
    Returns DataFrame with: building_id, name, function, location_state, 
                           construction_date, ensemble_id
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    buildings = []
    for node in _cbdd_graph['nodes']:
        if node.get('type') == 'OBJECT_BUILDING':
            info = get_building_info(node['id'])
            buildings.append({
                'building_id': node['id'],
                'name': node.get('name', ''),
                'function': info.get('function'),
                'location_state': info.get('location_state'),
                'construction_date': info.get('construction_date'),
                'ensemble_id': info.get('ensemble_id'),
            })
    
    df = pd.DataFrame(buildings)
    print(f"   ✓ Extracted {len(df):,} buildings")
    print(f"      With function: {df['function'].notna().sum():,}")
    print(f"      With location: {df['location_state'].notna().sum():,}")
    return df


def extract_rooms_table() -> pd.DataFrame:
    """
    Extract all rooms with their metadata from CbDD graph.
    
    Returns DataFrame with: room_id, name, function, building_id
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    rooms = []
    for node in _cbdd_graph['nodes']:
        if node.get('type') == 'OBJECT_ROOM':
            room_info = get_room_info(node['id'])
            
            # Find parent building via PART links
            building_id = None
            part_links = _cbdd_links_by_target.get(node['id'], [])
            for link in part_links:
                if link['type'] == 'PART':
                    parent = _cbdd_nodes_by_id.get(link['source'])
                    if parent and parent.get('type') == 'OBJECT_BUILDING':
                        building_id = parent['id']
                        break
                    elif parent and parent.get('type') == 'OBJECT_ROOM':
                        # Room inside room - traverse up
                        building = traverse_to_building_recursive(parent['id'])
                        if building:
                            building_id = building['id']
                        break
            
            rooms.append({
                'room_id': node['id'],
                'name': node.get('name', ''),
                'function': room_info.get('function'),
                'building_id': building_id,
            })
    
    df = pd.DataFrame(rooms)
    print(f"   ✓ Extracted {len(df):,} rooms")
    print(f"      Linked to buildings: {df['building_id'].notna().sum():,}")
    return df


def traverse_to_building_recursive(node_id: str, depth: int = 0) -> Optional[Dict]:
    """Helper: Recursively traverse up PART hierarchy to find building."""
    if depth > 10:
        return None
    
    part_links = _cbdd_links_by_target.get(node_id, [])
    for link in part_links:
        if link['type'] != 'PART':
            continue
        parent = _cbdd_nodes_by_id.get(link['source'])
        if not parent:
            continue
        if parent.get('type') == 'OBJECT_BUILDING':
            return parent
        elif parent.get('type') in ('OBJECT_ROOM', 'OBJECT_ENSEMBLE'):
            return traverse_to_building_recursive(parent['id'], depth + 1)
    return None


def extract_ensembles_table() -> pd.DataFrame:
    """
    Extract all ensembles (building complexes) from CbDD graph.
    
    Returns DataFrame with: ensemble_id, name
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    ensembles = []
    for node in _cbdd_graph['nodes']:
        if node.get('type') == 'OBJECT_ENSEMBLE':
            ensembles.append({
                'ensemble_id': node['id'],
                'name': node.get('name', ''),
            })
    
    df = pd.DataFrame(ensembles)
    print(f"   ✓ Extracted {len(df):,} ensembles")
    return df


print("✅ Entity extraction functions defined:")
print("   - extract_persons_table() -> all persons/organizations")
print("   - extract_buildings_table() -> all buildings with metadata")
print("   - extract_rooms_table() -> all rooms with building links")
print("   - extract_ensembles_table() -> all building complexes")

✅ Entity extraction functions defined:
   - extract_persons_table() -> all persons/organizations
   - extract_buildings_table() -> all buildings with metadata
   - extract_rooms_table() -> all rooms with building links
   - extract_ensembles_table() -> all building complexes


In [55]:
# =============================================================================
# Extract Junction Tables (Many-to-Many Relationships)
# =============================================================================
# These functions create the junction tables that link entities together,
# preserving the role information from the CbDD graph.

def extract_painting_persons_junction(df_enriched: pd.DataFrame) -> pd.DataFrame:
    """
    Extract painting-person junction table with roles from enriched DataFrame.
    
    Args:
        df_enriched: DataFrame with enriched painting data (from enrich_dataframe_from_graph)
    
    Returns:
        DataFrame with: nfdi_uri, cbdd_painting_id, person_id, person_name, role
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    junction_rows = []
    
    # Map column names to role types
    role_columns = {
        'painters': 'PAINTER',
        'commissioners': 'COMMISSIONER', 
        'architects': 'ARCHITECT',
        'plasterers': 'PLASTERER',
        'sculptors': 'SCULPTOR',
        'designers': 'DESIGNER',
        'template_providers': 'TEMPLATE_PROVIDER',
        'donors': 'DONOR',
        'reference_persons': 'REFERENCE_PERSON',
        'other_artists': 'OTHER_ARTIST',
    }
    
    for idx, row in df_enriched.iterrows():
        nfdi_uri = row.get('painting')  # NFDI4Culture URI - PRIMARY
        cbdd_id = row.get('cbdd_id')     # CbDD graph ID
        
        if not cbdd_id:
            continue
        
        for col, role in role_columns.items():
            persons_str = row.get(col)
            if pd.isna(persons_str) or not persons_str:
                continue
            
            # NOTE: Split by ' | ' to handle German names like 'Harms, Johann Oswald'
            for person_name in str(persons_str).split(' | '):
                person_name = person_name.strip()
                if not person_name:
                    continue
                
                # Find person ID by name
                person_nodes = _cbdd_nodes_by_name.get(person_name.lower(), [])
                person_id = person_nodes[0]['id'] if person_nodes else None
                
                junction_rows.append({
                    'nfdi_uri': nfdi_uri,           # Primary: NFDI4Culture
                    'cbdd_painting_id': cbdd_id,   # Secondary: CbDD
                    'person_id': person_id,
                    'person_name': person_name,
                    'role': role,
                })
    
    df = pd.DataFrame(junction_rows)
    
    # Show role distribution
    if len(df) > 0:
        role_counts = df['role'].value_counts()
        print(f"   ✓ Extracted {len(df):,} painting-person relationships")
        for role, count in role_counts.head(5).items():
            print(f"      {role}: {count:,}")
    
    return df


def extract_building_persons_junction() -> pd.DataFrame:
    """
    Extract building-person relationships directly from CbDD graph.
    
    Returns DataFrame with: building_id, person_id, person_name, role
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    junction_rows = []
    
    role_link_types = {
        'ARCHITECTS': 'ARCHITECT',
        'BUILDERS': 'BUILDER',
        'COMMISSIONERS': 'COMMISSIONER',
        'SCULPTORS': 'SCULPTOR',
        'OWNERS': 'OWNER',
        'PLASTERERS': 'PLASTERER',
        'CONSTRUCTION_MANAGERS': 'CONSTRUCTION_MANAGER',
        'LANDSCAPE_ARCHITECTS': 'LANDSCAPE_ARCHITECT',
    }
    
    for node in _cbdd_graph['nodes']:
        if node.get('type') != 'OBJECT_BUILDING':
            continue
        
        building_id = node['id']
        links = _cbdd_links_by_source.get(building_id, [])
        
        for link in links:
            if link['type'] in role_link_types:
                target = _cbdd_nodes_by_id.get(link['target'])
                if target and target.get('type') in ('ACTOR_PERSON', 'ACTOR_SOCIETY'):
                    junction_rows.append({
                        'building_id': building_id,
                        'person_id': target['id'],
                        'person_name': target.get('name', ''),
                        'role': role_link_types[link['type']],
                    })
    
    df = pd.DataFrame(junction_rows)
    print(f"   ✓ Extracted {len(df):,} building-person relationships")
    return df


def extract_room_persons_junction() -> pd.DataFrame:
    """
    Extract room-person relationships directly from CbDD graph.
    
    Returns DataFrame with: room_id, person_id, person_name, role
    """
    if not _cbdd_graph_loaded:
        load_cbdd_graph()
    
    junction_rows = []
    
    role_link_types = {
        'ARCHITECTS': 'ARCHITECT',
        'COMMISSIONERS': 'COMMISSIONER',
        'PLASTERERS': 'PLASTERER',
        'PAINTERS': 'PAINTER',
        'SCULPTORS': 'SCULPTOR',
        'DESIGNERS': 'DESIGNER',
        'BUILDERS': 'BUILDER',
    }
    
    for node in _cbdd_graph['nodes']:
        if node.get('type') != 'OBJECT_ROOM':
            continue
        
        room_id = node['id']
        links = _cbdd_links_by_source.get(room_id, [])
        
        for link in links:
            if link['type'] in role_link_types:
                target = _cbdd_nodes_by_id.get(link['target'])
                if target and target.get('type') in ('ACTOR_PERSON', 'ACTOR_SOCIETY'):
                    junction_rows.append({
                        'room_id': room_id,
                        'person_id': target['id'],
                        'person_name': target.get('name', ''),
                        'role': role_link_types[link['type']],
                    })
    
    df = pd.DataFrame(junction_rows)
    print(f"   ✓ Extracted {len(df):,} room-person relationships")
    return df


print("✅ Junction table extraction functions defined:")
print("   - extract_painting_persons_junction(df_enriched) -> painting-person links")
print("   - extract_building_persons_junction() -> building-person links")
print("   - extract_room_persons_junction() -> room-person links")

✅ Junction table extraction functions defined:
   - extract_painting_persons_junction(df_enriched) -> painting-person links
   - extract_building_persons_junction() -> building-person links
   - extract_room_persons_junction() -> room-person links


In [56]:
# =============================================================================
# Create Main Paintings Table and Subjects Junction Table
# =============================================================================

def create_paintings_table(df_enriched: pd.DataFrame) -> pd.DataFrame:
    """
    Create the main paintings table with all attributes and foreign keys.
    
    This table contains one row per painting with:
    - Primary key: nfdi_uri (NFDI4Culture Knowledge Graph URI - THE MAIN SOURCE)
    - Secondary key: cbdd_id (CbDD graphData.json node ID - for enrichment tracing)
    - Foreign keys: room_id, building_id
    - All metadata attributes
    
    Data Source Hierarchy:
    1. NFDI4Culture KG (nfdi_uri) - PRIMARY source, provides SPARQL endpoint, images
    2. CbDD graphData.json (cbdd_id) - Enriches with painters, buildings, rooms
    3. ICONCLASS/Getty AAT - Enriches subject classifications
    
    Args:
        df_enriched: Enriched DataFrame from the pipeline
    
    Returns:
        Normalized paintings DataFrame
    """
    columns_to_keep = [
        # PRIMARY KEY - NFDI4Culture Knowledge Graph URI
        'painting',  # Will be renamed to nfdi_uri
        
        # Secondary key - CbDD graph node ID (for enrichment tracing)
        'cbdd_id',
        
        # Foreign keys  
        'room_id', 'building_id',
        
        # Basic metadata (from NFDI4Culture)
        'label', 'year',
        
        # Geographic (enriched from building)
        'lat', 'lon', 'building_address', 'location_state',
        
        # Content URLs (from NFDI4Culture)
        'imageUrl', 'license',
        
        # Subjects (from NFDI4Culture, resolved via ICONCLASS/AAT)
        'subjects',
        
        # Room and building context (denormalized for convenience, from CbDD)
        'room_name', 'building_name', 'building_function',
        
        # Original SPARQL parent structure (from NFDI4Culture)
        'parentUri', 'parentLabel',
        
        # CbDD enrichment fields
        'painters', 'commissioners', 'method',
        
        # GNDs for Bildindex cross-referencing (from SPARQL)
        'creatorGnds', 'locationGnds',
    ]
    
    # Keep only columns that exist
    existing_cols = [c for c in columns_to_keep if c in df_enriched.columns]
    df = df_enriched[existing_cols].copy()
    
    # Rename primary key for clarity - this is THE NFDI4Culture URI
    if 'painting' in df.columns:
        df = df.rename(columns={'painting': 'nfdi_uri'})
    
    # Also keep painting_uri as alias for backward compatibility
    if 'nfdi_uri' in df.columns:
        df['painting_uri'] = df['nfdi_uri']
    
    # Add cbdd_painting_id alias
    if 'cbdd_id' in df.columns:
        df['cbdd_painting_id'] = df['cbdd_id']
    
    # Ensure numeric types for coordinates
    for col in ['lat', 'lon']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print(f"   ✓ Created paintings table with {len(df):,} rows, {len(df.columns)} columns")
    print(f"      Primary key: nfdi_uri (NFDI4Culture Knowledge Graph)")
    print(f"      Secondary key: cbdd_id (CbDD graphData.json)")
    # GND statistics for Bildindex integration
    if 'creatorGnds' in df.columns:
        has_creator = (df['creatorGnds'].notna() & (df['creatorGnds'] != '')).sum()
        print(f"      With creator GNDs: {has_creator:,} ({100*has_creator/len(df):.1f}%)")
    if 'locationGnds' in df.columns:
        has_location = (df['locationGnds'].notna() & (df['locationGnds'] != '')).sum()
        print(f"      With location GNDs: {has_location:,} ({100*has_location/len(df):.1f}%)")
    
    return df


def extract_painting_subjects_junction(df_enriched: pd.DataFrame) -> pd.DataFrame:
    """
    Extract painting-subject junction table from resolved subjects.
    
    Args:
        df_enriched: DataFrame with 'subjects_resolved' column (list of dicts)
    
    Returns:
        DataFrame with: nfdi_uri, cbdd_painting_id, subject_uri, subject_label, subject_source
    """
    junction_rows = []
    
    for idx, row in df_enriched.iterrows():
        nfdi_uri = row.get('painting')  # NFDI4Culture URI
        cbdd_id = row.get('cbdd_id')     # CbDD graph ID
        subjects = row.get('subjects_resolved', [])
        
        if not subjects or not isinstance(subjects, list):
            continue
        
        for subject in subjects:
            if not isinstance(subject, dict):
                continue
            
            junction_rows.append({
                'nfdi_uri': nfdi_uri,           # Primary: NFDI4Culture
                'cbdd_painting_id': cbdd_id,    # Secondary: CbDD
                'subject_uri': subject.get('uri'),
                'subject_label': subject.get('label'),
                'subject_source': subject.get('source'),  # 'ICONCLASS' or 'GETTY_AAT'
            })
    
    df = pd.DataFrame(junction_rows)
    
    if len(df) > 0:
        print(f"   ✓ Extracted {len(df):,} painting-subject relationships")
        source_counts = df['subject_source'].value_counts()
        for src, count in source_counts.items():
            print(f"      {src}: {count:,}")
    else:
        print("   ⚠ No subjects found in data")
    
    return df


def extract_subjects_table(df_subjects_junction: pd.DataFrame) -> pd.DataFrame:
    """
    Extract unique subjects table from junction table.
    
    Args:
        df_subjects_junction: The painting-subjects junction table
    
    Returns:
        DataFrame with: subject_uri (PK), subject_label, subject_source
    """
    if len(df_subjects_junction) == 0:
        return pd.DataFrame(columns=['subject_uri', 'subject_label', 'subject_source'])
    
    df = df_subjects_junction.drop_duplicates(subset=['subject_uri'])[
        ['subject_uri', 'subject_label', 'subject_source']
    ].copy()
    
    print(f"   ✓ Extracted {len(df):,} unique subjects")
    return df


print("✅ Main table creation functions defined:")
print("   - create_paintings_table(df_enriched) -> main paintings table")
print("   - extract_painting_subjects_junction(df_enriched) -> painting-subject links")
print("   - extract_subjects_table(df_junction) -> unique subjects")
print("\n📌 Key Identifiers:")
print("   - nfdi_uri: NFDI4Culture Knowledge Graph URI (PRIMARY SOURCE)")
print("   - cbdd_id: CbDD graphData.json node ID (for enrichment)")

✅ Main table creation functions defined:
   - create_paintings_table(df_enriched) -> main paintings table
   - extract_painting_subjects_junction(df_enriched) -> painting-subject links
   - extract_subjects_table(df_junction) -> unique subjects

📌 Key Identifiers:
   - nfdi_uri: NFDI4Culture Knowledge Graph URI (PRIMARY SOURCE)
   - cbdd_id: CbDD graphData.json node ID (for enrichment)


In [57]:
# =============================================================================
# MAIN PIPELINE: Full Dataset Extraction to Parquet
# =============================================================================
# This is the main execution pipeline that:
# 1. Fetches ALL paintings from SPARQL (paginated)
# 2. Enriches with CbDD graph data
# 3. Resolves subjects (ICONCLASS/AAT)
# 4. Extracts normalized tables
# 5. Exports to Parquet files

def run_parquet_export_pipeline(
    batch_size: int = 500,
    max_paintings: int = None,
    skip_subject_resolution: bool = False,
) -> dict:
    """
    Run the complete Parquet export pipeline.
    
    Args:
        batch_size: Number of paintings per SPARQL query batch
        max_paintings: Maximum paintings to fetch (None = all)
        skip_subject_resolution: Skip ICONCLASS/AAT resolution (faster)
    
    Returns:
        Dictionary with all DataFrames: {'paintings': df, 'persons': df, ...}
    """
    import os
    
    print("=" * 70)
    print("PARQUET DATABASE EXPORT PIPELINE")
    print("=" * 70)
    
    # Step 1: Load CbDD Graph
    print("\n📥 Step 1: Loading CbDD Graph...")
    load_cbdd_graph()
    
    # Step 2: Fetch ALL paintings from SPARQL
    print("\n📥 Step 2: Fetching paintings from SPARQL...")
    df_all = fetch_all_paintings_sparql(batch_size=batch_size, max_paintings=max_paintings)
    
    if len(df_all) == 0:
        print("❌ No paintings fetched!")
        return {}
    
    # Step 3: Enrich with CbDD graph
    print("\n🔗 Step 3: Enriching with CbDD graph data...")
    df_enriched = enrich_dataframe_from_graph(df_all)
    
    # Get room_id and building_id from CbDD graph using cbdd_id
    # The enrich_dataframe_from_graph already populates 'room' and 'building' names
    # We need to find the actual node IDs for foreign keys
    
    def get_room_id_from_cbdd(cbdd_id):
        """Get room node ID by traversing PART links from painting."""
        if not cbdd_id:
            return None
        links = _cbdd_links_by_source.get(cbdd_id, [])
        for link in links:
            if link['type'] == 'PART':
                target = _cbdd_nodes_by_id.get(link['target'])
                if target and target.get('type') == 'OBJECT_ROOM':
                    return target['id']
        return None
    
    def get_building_id_from_cbdd(cbdd_id):
        """Get building node ID by traversing PART links (painting -> room -> building)."""
        if not cbdd_id:
            return None
        building = traverse_to_building_recursive(cbdd_id)
        # traverse_to_building_recursive returns the full node dict, extract just the ID
        return building['id'] if building else None
    
    # Add foreign key IDs
    df_enriched['room_id'] = df_enriched['cbdd_id'].apply(get_room_id_from_cbdd)
    df_enriched['building_id'] = df_enriched['cbdd_id'].apply(get_building_id_from_cbdd)
    
    # Get room and building names (already in 'room' and 'building' columns from enrichment)
    # But add _name versions for clarity
    df_enriched['room_name'] = df_enriched['room']
    df_enriched['building_name'] = df_enriched['building']
    
    # Step 4: Enrich coordinates from buildings
    print("\n📍 Step 4: Enriching coordinates...")
    df_enriched = enrich_coordinates(df_enriched)
    
    # Count coordinate coverage
    has_coords = df_enriched[['lat', 'lon']].notna().all(axis=1).sum()
    print(f"   ✓ {has_coords:,}/{len(df_enriched):,} paintings have coordinates ({100*has_coords/len(df_enriched):.1f}%)")
    
    # Step 5: Resolve subjects (optional - can be slow)
    if not skip_subject_resolution:
        print("\n📖 Step 5: Resolving subjects (ICONCLASS/AAT)...")
        
        # Extract subject URIs from the 'subjects' column (from SPARQL query)
        # Format is pipe-separated: "uri1|uri2|uri3"
        def extract_subject_uris(subjects_str):
            if pd.isna(subjects_str) or not subjects_str:
                return []
            uris = []
            for uri in str(subjects_str).split('|'):
                uri = uri.strip()
                if uri and ('iconclass.org' in uri or 'vocab.getty.edu' in uri):
                    uris.append(uri)
            return uris
        
        df_enriched['subject_uris'] = df_enriched['subjects'].apply(extract_subject_uris)
        
        # Resolve subjects in batches
        df_enriched = batch_resolve_subjects(df_enriched, uri_column='subject_uris')
    else:
        print("\n⏭ Step 5: Skipping subject resolution")
        df_enriched['subjects_resolved'] = [[] for _ in range(len(df_enriched))]
    
    # Step 6: Extract normalized tables
    print("\n📊 Step 6: Extracting normalized tables...")
    
    tables = {}
    
    # Main paintings table
    print("\n   Creating paintings table...")
    tables['paintings'] = create_paintings_table(df_enriched)
    
    # Entity tables (from CbDD graph)
    print("\n   Extracting entity tables from CbDD graph...")
    tables['persons'] = extract_persons_table()
    tables['buildings'] = extract_buildings_table()
    tables['rooms'] = extract_rooms_table()
    tables['ensembles'] = extract_ensembles_table()
    
    # Junction tables
    print("\n   Extracting junction tables...")
    tables['painting_persons'] = extract_painting_persons_junction(df_enriched)
    tables['painting_subjects'] = extract_painting_subjects_junction(df_enriched)
    tables['subjects'] = extract_subjects_table(tables['painting_subjects'])
    tables['building_persons'] = extract_building_persons_junction()
    tables['room_persons'] = extract_room_persons_junction()
    
    # Step 7: Export to Parquet
    print("\n💾 Step 7: Exporting to Parquet files...")
    
    # Create output directory
    os.makedirs(PARQUET_OUTPUT_DIR, exist_ok=True)
    
    for table_name, df in tables.items():
        save_parquet_with_metadata(df, table_name)
    
    # Summary
    print("\n" + "=" * 70)
    print("✅ PIPELINE COMPLETE")
    print("=" * 70)
    print(f"\nOutput directory: {PARQUET_OUTPUT_DIR}")
    print("\nTable Summary:")
    print("-" * 50)
    for name, df in tables.items():
        print(f"   {name:25} {len(df):>8,} rows, {len(df.columns):>3} cols")
    
    total_rows = sum(len(df) for df in tables.values())
    print("-" * 50)
    print(f"   {'TOTAL':25} {total_rows:>8,} rows")
    
    return tables


print("✅ Main pipeline function defined: run_parquet_export_pipeline()")
print("   Options:")
print("   - batch_size: SPARQL query batch size (default: 500)")
print("   - max_paintings: Limit paintings (None = all ~6000+)")
print("   - skip_subject_resolution: Skip ICONCLASS/AAT (faster)")

✅ Main pipeline function defined: run_parquet_export_pipeline()
   Options:
   - batch_size: SPARQL query batch size (default: 500)
   - max_paintings: Limit paintings (None = all ~6000+)
   - skip_subject_resolution: Skip ICONCLASS/AAT (faster)


In [58]:
# =============================================================================
# RUN THE PIPELINE (Full Dataset)
# =============================================================================
# ⚠️ WARNING: This will fetch ~6000+ paintings and may take 10-30 minutes
# depending on network speed and ICONCLASS resolution.
#
# For testing, use: tables = run_parquet_export_pipeline(max_paintings=100)

# Run with subject resolution (slower but complete)
tables = run_parquet_export_pipeline(
    batch_size=100,
    max_paintings=8000,  # Set to e.g. 100 for testing
    skip_subject_resolution=False,
)

PARQUET DATABASE EXPORT PIPELINE

📥 Step 1: Loading CbDD Graph...

📥 Step 2: Fetching paintings from SPARQL...
📥 Fetching paintings from SPARQL endpoint...
   Batch size: 100, Max: 8000
   Batch   1: + 100 paintings (total: 100)
   Batch   2: + 100 paintings (total: 200)
   Batch   3: + 100 paintings (total: 300)
   Batch   4: + 100 paintings (total: 400)
   Batch   5: + 100 paintings (total: 500)
   Batch   6: + 100 paintings (total: 600)
   Batch   7: + 100 paintings (total: 700)
   Batch   8: + 100 paintings (total: 800)
   Batch   9: + 100 paintings (total: 900)
   Batch  10: + 100 paintings (total: 1,000)
   Batch  11: + 100 paintings (total: 1,100)
   Batch  12: + 100 paintings (total: 1,200)
   Batch  13: + 100 paintings (total: 1,300)
   Batch  14: + 100 paintings (total: 1,400)
   Batch  15: + 100 paintings (total: 1,500)
   Batch  16: + 100 paintings (total: 1,600)
   Batch  17: + 100 paintings (total: 1,700)
   Batch  18: + 100 paintings (total: 1,800)
   Batch  19: + 100 pa

In [59]:
# =============================================================================
# Verification and Quality Checks
# =============================================================================

def verify_parquet_database():
    """
    Verify the exported Parquet database integrity and show statistics.
    """
    print("=" * 70)
    print("PARQUET DATABASE VERIFICATION")
    print("=" * 70)
    
    # Load all tables
    table_names = [
        'paintings', 'persons', 'buildings', 'rooms', 'ensembles',
        'painting_persons', 'painting_subjects', 'subjects',
        'building_persons', 'room_persons'
    ]
    
    loaded_tables = {}
    
    print("\n📊 Table Statistics:")
    print("-" * 60)
    
    for name in table_names:
        try:
            df = load_parquet_table(name)
            loaded_tables[name] = df
            nulls = df.isnull().sum().sum()
            print(f"   {name:25} {len(df):>8,} rows, {len(df.columns):>3} cols, {nulls:>6,} nulls")
        except FileNotFoundError:
            print(f"   {name:25} ❌ NOT FOUND")
    
    # Referential Integrity Checks
    print("\n\n🔗 Referential Integrity Checks:")
    print("-" * 60)
    
    if 'paintings' in loaded_tables and 'painting_persons' in loaded_tables:
        pp = loaded_tables['painting_persons']
        p = loaded_tables['paintings']
        
        # Check painting URIs in junction exist in paintings (use nfdi_uri or painting_uri)
        uri_col_pp = 'nfdi_uri' if 'nfdi_uri' in pp.columns else 'painting_uri'
        uri_col_p = 'nfdi_uri' if 'nfdi_uri' in p.columns else 'painting_uri'
        if uri_col_pp in pp.columns and uri_col_p in p.columns:
            pp_uris = set(pp[uri_col_pp].dropna())
            p_uris = set(p[uri_col_p].dropna())
            orphan_pp = pp_uris - p_uris
            print(f"   painting_persons → paintings: {len(pp_uris - orphan_pp):,} valid, {len(orphan_pp):,} orphan")
    
    if 'painting_persons' in loaded_tables and 'persons' in loaded_tables:
        pp = loaded_tables['painting_persons']
        pers = loaded_tables['persons']
        
        if 'person_id' in pp.columns and 'person_id' in pers.columns:
            pp_pids = set(pp['person_id'].dropna())
            pers_pids = set(pers['person_id'].dropna())
            orphan_pers = pp_pids - pers_pids
            print(f"   painting_persons → persons: {len(pp_pids - orphan_pers):,} valid, {len(orphan_pers):,} orphan")
    
    if 'paintings' in loaded_tables and 'buildings' in loaded_tables:
        paint = loaded_tables['paintings']
        build = loaded_tables['buildings']
        
        if 'building_id' in paint.columns and 'building_id' in build.columns:
            p_bids = set(paint['building_id'].dropna())
            b_bids = set(build['building_id'].dropna())
            matched = p_bids & b_bids
            print(f"   paintings → buildings: {len(matched):,} matched, {len(p_bids - b_bids):,} unmatched")
    
    # Coverage Statistics
    print("\n\n📈 Coverage Statistics:")
    print("-" * 60)
    
    if 'paintings' in loaded_tables:
        df = loaded_tables['paintings']
        total = len(df)
        
        coverage = {}
        # Check for NFDI URI (should always be present as primary key)
        if 'nfdi_uri' in df.columns:
            coverage['Has NFDI URI (primary)'] = df['nfdi_uri'].notna().sum()
        if 'cbdd_painting_id' in df.columns:
            coverage['CbDD Graph Match'] = df['cbdd_painting_id'].notna().sum()
        if 'building_id' in df.columns:
            coverage['Has Building'] = df['building_id'].notna().sum()
        if 'room_id' in df.columns:
            coverage['Has Room'] = df['room_id'].notna().sum()
        if 'lat' in df.columns:
            coverage['Has Coordinates'] = df['lat'].notna().sum()
        if 'location_state' in df.columns:
            coverage['Has Location State'] = df['location_state'].notna().sum()
        
        for metric, count in coverage.items():
            pct = 100 * count / total if total > 0 else 0
            print(f"   {metric:25} {count:>6,} / {total:,} ({pct:.1f}%)")
    
    if 'painting_persons' in loaded_tables:
        pp = loaded_tables['painting_persons']
        if 'role' in pp.columns:
            print("\n   Role Distribution (painting_persons):")
            for role, count in pp['role'].value_counts().head(5).items():
                print(f"      {role:20} {count:>6,}")
    
    # Sample SPARQL Reconstruction Query
    print("\n\n🔍 Sample SPARQL Reconstruction (using NFDI4Culture URI):")
    print("-" * 60)
    if 'paintings' in loaded_tables:
        df = loaded_tables['paintings']
        uri_col = 'nfdi_uri' if 'nfdi_uri' in df.columns else 'painting_uri'
        if uri_col in df.columns and len(df) > 0:
            sample_uri = df[uri_col].iloc[0]
            print(f"   Primary source: NFDI4Culture Knowledge Graph")
            print(f"   To fetch original painting from SPARQL:")
            print(f"   SELECT * WHERE {{ <{sample_uri}> ?p ?o }}")
    
    return loaded_tables


# Run verification
loaded_tables = verify_parquet_database()

PARQUET DATABASE VERIFICATION

📊 Table Statistics:
------------------------------------------------------------


   paintings                    4,594 rows,  25 cols, 33,845 nulls
   persons                      2,831 rows,   4 cols,      0 nulls
   buildings                    1,260 rows,   6 cols,  1,661 nulls
   rooms                        2,376 rows,   4 cols,    879 nulls
   ensembles                       32 rows,   2 cols,      0 nulls
   painting_persons             5,848 rows,   5 cols,      0 nulls
   painting_subjects           17,474 rows,   5 cols,  1,093 nulls
   subjects                     4,082 rows,   3 cols,      0 nulls
   building_persons             1,983 rows,   4 cols,      0 nulls
   room_persons                 4,676 rows,   4 cols,      0 nulls


🔗 Referential Integrity Checks:
------------------------------------------------------------
   painting_persons → paintings: 2,771 valid, 0 orphan
   painting_persons → persons: 1,002 valid, 0 orphan
   paintings → buildings: 482 matched, 0 unmatched


📈 Coverage Statistics:
---------------------------------------------------

## Database Schema Documentation

The exported Parquet database follows a normalized relational schema designed to preserve the graph connections from both the NFDI4Culture SPARQL endpoint and the CbDD JSON graph.

### Data Source Hierarchy

1. **NFDI4Culture Knowledge Graph** (PRIMARY SOURCE)
   - Provides the SPARQL endpoint
   - Contains image URLs and media
   - Primary identifier: `nfdi_uri`
   
2. **CbDD graphData.json** (ENRICHMENT)
   - Enriches with painters, commissioners, architects
   - Provides room and building hierarchies
   - Secondary identifier: `cbdd_id`
   
3. **ICONCLASS / Getty AAT** (ENRICHMENT)
   - Resolves subject classification labels
   - Identifier: `subject_uri`

### Core Entity Tables

| Table | Primary Key | Description |
|-------|-------------|-------------|
| `paintings` | `nfdi_uri` | Main paintings table with all metadata and foreign keys |
| `persons` | `person_id` | All persons (painters, architects, commissioners, etc.) from CbDD |
| `buildings` | `building_id` | Buildings containing paintings |
| `rooms` | `room_id` | Rooms within buildings |
| `ensembles` | `ensemble_id` | Painting ensembles (e.g., cycles) |
| `subjects` | `subject_uri` | ICONCLASS and Getty AAT subject classifications |

### Junction Tables (Many-to-Many Relationships)

| Table | Foreign Keys | Description |
|-------|--------------|-------------|
| `painting_persons` | `nfdi_uri`, `person_id` | Links paintings to persons with role (PAINTER, COMMISSIONER, etc.) |
| `painting_subjects` | `nfdi_uri`, `subject_uri` | Links paintings to subject classifications |
| `building_persons` | `building_id`, `person_id` | Links buildings to persons with role |
| `room_persons` | `room_id`, `person_id` | Links rooms to persons with role |

### Key Mapping Fields

To reconstruct SPARQL queries or trace back to original data sources:

- **`nfdi_uri`**: The NFDI4Culture Knowledge Graph URI (PRIMARY SOURCE)
  - Format: `https://nfdi4culture.de/id/...`
  - Use this to query the SPARQL endpoint for original data
  - Contains images, media, and authoritative metadata
  
- **`cbdd_id` / `cbdd_painting_id`**: The CbDD JSON graph node ID
  - Format: `n-xxxxx`
  - Links to graphData.json for enrichment data
  - Provides painter/commissioner/building relationships
  
- **`subject_uri`**: ICONCLASS or Getty AAT URI
  - Format: `http://iconclass.org/...` or `http://vocab.getty.edu/aat/...`
  - Links to external subject classification systems

### Example Queries

**Reconstruct original SPARQL data from NFDI4Culture:**
```python
nfdi_uri = paintings_df['nfdi_uri'].iloc[0]
# SPARQL: SELECT * WHERE { <nfdi_uri> ?p ?o }
```

**Get all painters for a painting:**
```python
painters = painting_persons[
    (painting_persons['nfdi_uri'] == nfdi_uri) & 
    (painting_persons['role'] == 'PAINTER')
]
```

**Find paintings in a building:**
```python
building_paintings = paintings[paintings['building_id'] == building_id]
```

**Trace data source for a painting:**
```python
row = paintings[paintings['nfdi_uri'] == nfdi_uri].iloc[0]
print(f"NFDI4Culture: {row['nfdi_uri']}")  # Main source
print(f"CbDD Graph: {row['cbdd_id']}")      # Enrichment source
```

Analysis of unmatched metadata

In [60]:
# =============================================================================
# ANALYSIS: Paintings NOT Found in CbDD JSON (Unmatched Entities)
# =============================================================================
# This cell analyzes paintings from NFDI4Culture that couldn't be matched
# with entries in the CbDD graphData.json file.

print("=" * 70)
print("🔍 ANALYSIS: Paintings Without CbDD JSON Enrichment")
print("=" * 70)

# Load the paintings table from parquet
df_paintings = loaded_tables.get('paintings', pd.DataFrame())

if len(df_paintings) == 0:
    print("⚠ No paintings table loaded. Run the pipeline first.")
else:
    # Identify unmatched paintings (no cbdd_id)
    unmatched = df_paintings[df_paintings['cbdd_painting_id'].isna()].copy()
    matched = df_paintings[df_paintings['cbdd_painting_id'].notna()].copy()
    
    total = len(df_paintings)
    n_unmatched = len(unmatched)
    n_matched = len(matched)
    
    print(f"\n📊 MATCHING SUMMARY:")
    print(f"   Total paintings from NFDI4Culture: {total:,}")
    print(f"   ✓ Matched with CbDD JSON: {n_matched:,} ({100*n_matched/total:.1f}%)")
    print(f"   ✗ NOT matched: {n_unmatched:,} ({100*n_unmatched/total:.1f}%)")
    
    if n_unmatched > 0:
        print("\n" + "=" * 70)
        print("📋 CHARACTERISTICS OF UNMATCHED PAINTINGS:")
        print("=" * 70)
        
        # What data DO they have from NFDI4Culture?
        print("\n🔹 Data available from NFDI4Culture (primary source):")
        nfdi_cols = ['nfdi_uri', 'label', 'year', 'lat', 'lon', 'imageUrl', 'subjects', 'parentUri']
        for col in nfdi_cols:
            if col in unmatched.columns:
                non_null = unmatched[col].notna().sum()
                pct = 100 * non_null / n_unmatched if n_unmatched > 0 else 0
                print(f"      {col:20}: {non_null:4}/{n_unmatched} ({pct:5.1f}%)")
        
        # What data is MISSING because no CbDD match?
        print("\n🔹 Data MISSING (would come from CbDD enrichment):")
        cbdd_cols = ['painters', 'commissioners', 'room_name', 'building_name', 
                     'building_function', 'location_state', 'method']
        for col in cbdd_cols:
            if col in unmatched.columns:
                non_null = unmatched[col].notna().sum()
                # These should all be 0 or near 0 since no CbDD match
                print(f"      {col:20}: {non_null:4}/{n_unmatched} (missing enrichment)")
        
        # Analyze patterns in unmatched labels
        print("\n" + "=" * 70)
        print("📝 SAMPLE UNMATCHED PAINTING LABELS:")
        print("=" * 70)
        print("   (Analyze these to understand why matching failed)")
        print()
        for i, row in unmatched.head(20).iterrows():
            label = row.get('label', 'N/A')[:80]
            year = row.get('year', 'N/A')
            has_parent = "✓" if pd.notna(row.get('parentUri')) else "✗"
            print(f"   • {label}")
            print(f"     Year: {year}, Has parent: {has_parent}")
        
        if n_unmatched > 20:
            print(f"\n   ... and {n_unmatched - 20} more unmatched paintings")
        
        # Check if there are patterns in parent URIs
        print("\n" + "=" * 70)
        print("🏛️ PARENT STRUCTURES OF UNMATCHED (from NFDI4Culture):")
        print("=" * 70)
        if 'parentLabel' in unmatched.columns:
            parent_counts = unmatched['parentLabel'].value_counts().head(15)
            for parent, count in parent_counts.items():
                if pd.notna(parent):
                    print(f"   {parent[:60]:60} : {count:3} paintings")
        
        # Compare with matched - do they have different characteristics?
        print("\n" + "=" * 70)
        print("📊 COMPARISON: Matched vs Unmatched")
        print("=" * 70)
        
        # Year distribution
        if 'year' in df_paintings.columns:
            matched_years = matched['year'].dropna()
            unmatched_years = unmatched['year'].dropna()
            print(f"\n   Year coverage:")
            print(f"      Matched: {len(matched_years)}/{n_matched} have year data")
            print(f"      Unmatched: {len(unmatched_years)}/{n_unmatched} have year data")
        
        # Coordinate coverage
        if 'lat' in df_paintings.columns:
            matched_coords = matched['lat'].notna().sum()
            unmatched_coords = unmatched['lat'].notna().sum()
            print(f"\n   Coordinate coverage:")
            print(f"      Matched: {matched_coords}/{n_matched} ({100*matched_coords/n_matched:.1f}%)")
            print(f"      Unmatched: {unmatched_coords}/{n_unmatched} ({100*unmatched_coords/n_unmatched:.1f}%)")
        
        # Subject coverage
        if 'subjects' in df_paintings.columns:
            matched_subj = (matched['subjects'].notna() & (matched['subjects'] != '')).sum()
            unmatched_subj = (unmatched['subjects'].notna() & (unmatched['subjects'] != '')).sum()
            print(f"\n   Subject (ICONCLASS/AAT) coverage:")
            print(f"      Matched: {matched_subj}/{n_matched} ({100*matched_subj/n_matched:.1f}%)")
            print(f"      Unmatched: {unmatched_subj}/{n_unmatched} ({100*unmatched_subj/n_unmatched:.1f}%)")
        
        print("\n" + "=" * 70)
        print("💡 POSSIBLE REASONS FOR NO MATCH:")
        print("=" * 70)
        print("   1. Label text differs slightly between NFDI4Culture and CbDD")
        print("   2. Painting exists in NFDI4Culture but not in CbDD dataset")
        print("   3. Different naming conventions (e.g., 'St.' vs 'Sankt')")
        print("   4. Special characters or encoding differences")
        print("   5. CbDD may focus on specific regions/periods")
        
        # Export unmatched to CSV for manual review
        print("\n" + "=" * 70)
        print("💾 EXPORT FOR MANUAL REVIEW:")
        print("=" * 70)
        unmatched_export = unmatched[['nfdi_uri', 'label', 'year', 'parentLabel', 'subjects']].head(500)
        export_path = 'unmatched_paintings_analysis.csv'
        unmatched_export.to_csv(export_path, index=False)
        print(f"   Exported {len(unmatched_export)} unmatched paintings to: {export_path}")
    
    else:
        print("\n🎉 All paintings were successfully matched with CbDD JSON!")

🔍 ANALYSIS: Paintings Without CbDD JSON Enrichment

📊 MATCHING SUMMARY:
   Total paintings from NFDI4Culture: 4,594
   ✓ Matched with CbDD JSON: 3,508 (76.4%)
   ✗ NOT matched: 1,086 (23.6%)

📋 CHARACTERISTICS OF UNMATCHED PAINTINGS:

🔹 Data available from NFDI4Culture (primary source):
      nfdi_uri            : 1086/1086 (100.0%)
      label               : 1086/1086 (100.0%)
      year                :  914/1086 ( 84.2%)
      lat                 : 1083/1086 ( 99.7%)
      lon                 : 1083/1086 ( 99.7%)
      imageUrl            : 1086/1086 (100.0%)
      subjects            : 1086/1086 (100.0%)
      parentUri           :    0/1086 (  0.0%)

🔹 Data MISSING (would come from CbDD enrichment):
      painters            :    0/1086 (missing enrichment)
      commissioners       :    0/1086 (missing enrichment)
      room_name           :    0/1086 (missing enrichment)
      building_name       :    0/1086 (missing enrichment)
      building_function   :    0/1086 (missing en

## 6. Bildindex Data Integration

This section integrates data from the **Bildindex** (n4c:E6161) - a massive collection of ~383,861 historical art images - using GND-based cross-referencing with our CbDD ceiling paintings.

### Integration Strategy

1. **Extract GNDs from parquet files** - Get creator/location GNDs from our CbDD data
2. **Find Bildindex overlaps via SPARQL** - Query which GNDs exist in Bildindex 
3. **Tiered data collection**:
   - **Tier 1**: Items at shared building GNDs
   - **Tier 2**: Items by shared painter GNDs
4. **URL validation** - Filter out soft-404 dead links
5. **Export to parquet** - Store as `bildindex_*` tables

In [61]:
# =============================================================================
# Step 6.1: Extract GNDs from CbDD Parquet Files
# =============================================================================
# Load the paintings table and extract unique GNDs for Bildindex cross-referencing

print("=" * 70)
print("BILDINDEX INTEGRATION - Step 1: GND Extraction from CbDD")
print("=" * 70)

# Load paintings table (should exist from pipeline)
df_cbdd_paintings = load_parquet_table('paintings')

if len(df_cbdd_paintings) == 0:
    print("⚠ Paintings table not found. Run the parquet export pipeline first.")
else:
    print(f"✓ Loaded {len(df_cbdd_paintings):,} CbDD paintings")
    
    # Extract unique creator GNDs
    cbdd_creator_gnds = set()
    for gnds_str in df_cbdd_paintings['creatorGnds'].dropna():
        if gnds_str:
            for gnd in str(gnds_str).split('|'):
                gnd = gnd.strip()
                if gnd and 'd-nb.info/gnd' in gnd:
                    cbdd_creator_gnds.add(gnd)
    
    # Extract unique location GNDs
    cbdd_location_gnds = set()
    for gnds_str in df_cbdd_paintings['locationGnds'].dropna():
        if gnds_str:
            for gnd in str(gnds_str).split('|'):
                gnd = gnd.strip()
                if gnd and 'd-nb.info/gnd' in gnd:
                    cbdd_location_gnds.add(gnd)
    
    print(f"\n📊 GND Extraction Results:")
    print(f"   Unique creator (painter) GNDs: {len(cbdd_creator_gnds):,}")
    print(f"   Unique location (building) GNDs: {len(cbdd_location_gnds):,}")
    
    # Save for later use
    CBDD_CREATOR_GNDS = list(cbdd_creator_gnds)
    CBDD_LOCATION_GNDS = list(cbdd_location_gnds)
    
    # Show sample GNDs
    print(f"\n📋 Sample Creator GNDs (first 5):")
    for gnd in list(cbdd_creator_gnds)[:5]:
        print(f"   {gnd}")
    
    print(f"\n📋 Sample Location GNDs (first 5):")
    for gnd in list(cbdd_location_gnds)[:5]:
        print(f"   {gnd}")

BILDINDEX INTEGRATION - Step 1: GND Extraction from CbDD
✓ Loaded 4,594 CbDD paintings

📊 GND Extraction Results:
   Unique creator (painter) GNDs: 1,074
   Unique location (building) GNDs: 206

📋 Sample Creator GNDs (first 5):
   https://d-nb.info/gnd/1163757071
   https://d-nb.info/gnd/118718940
   https://d-nb.info/gnd/118716913
   https://d-nb.info/gnd/131888412
   https://d-nb.info/gnd/134087992

📋 Sample Location GNDs (first 5):
   https://d-nb.info/gnd/4106691-1
   https://d-nb.info/gnd/4499248-8
   https://d-nb.info/gnd/4297910-9
   https://d-nb.info/gnd/4700033-8
   https://d-nb.info/gnd/4099873-3


In [62]:
# =============================================================================
# Step 6.2: Find Bildindex Overlaps via SPARQL
# =============================================================================
# Query which CbDD GNDs also exist in the Bildindex feed (n4c:E6161)

BILDINDEX_FEED_URI = "n4c:E6161"

def find_bildindex_overlapping_gnds(gnd_list, gnd_type='location', batch_size=50):
    """
    Check which GNDs from CbDD also exist in Bildindex.
    
    Args:
        gnd_list: List of GND URIs to check
        gnd_type: 'location' (buildings) or 'creator' (painters)
        batch_size: Number of GNDs to check per query
    
    Returns:
        Set of GNDs that exist in both CbDD and Bildindex
    """
    overlapping = set()
    
    # Property mapping
    if gnd_type == 'location':
        node_prop = 'cto:CTO_0001011'  # location node
    else:  # creator
        node_prop = 'cto:CTO_0001009'  # creator node
    
    total_batches = (len(gnd_list) + batch_size - 1) // batch_size
    
    for i in range(0, len(gnd_list), batch_size):
        batch = gnd_list[i:i + batch_size]
        batch_num = i // batch_size + 1
        
        # Format GNDs for VALUES clause
        gnd_values = ' '.join(f'<{gnd}>' for gnd in batch)
        
        query = f"""
        {PREFIXES}
        SELECT DISTINCT ?gnd
        WHERE {{
            VALUES ?gnd {{ {gnd_values} }}
            {BILDINDEX_FEED_URI} schema:dataFeedElement ?feedItem .
            ?feedItem schema:item ?item .
            ?item {node_prop} ?node .
            ?node ?pred ?gnd .
        }}
        """
        
        try:
            df_result = run_sparql(query)
            if not df_result.empty:
                found_gnds = set(df_result['gnd'].tolist())
                overlapping.update(found_gnds)
        except Exception as e:
            print(f"   ⚠ Error in batch {batch_num}: {e}")
            continue
        
        if batch_num % 10 == 0:
            print(f"   Progress: {batch_num}/{total_batches} batches, {len(overlapping)} overlaps found")
    
    return overlapping

print("=" * 70)
print("BILDINDEX INTEGRATION - Step 2: Finding Overlapping GNDs")
print("=" * 70)

# Check if we have GNDs from previous step
if 'CBDD_CREATOR_GNDS' not in dir() or 'CBDD_LOCATION_GNDS' not in dir():
    print("⚠ Run Step 6.1 first to extract GNDs from CbDD data")
else:
    # Find overlapping location (building) GNDs
    print(f"\n🏛️ Checking {len(CBDD_LOCATION_GNDS):,} location GNDs...")
    shared_location_gnds = find_bildindex_overlapping_gnds(CBDD_LOCATION_GNDS, 'location')
    print(f"   ✓ Found {len(shared_location_gnds):,} shared location GNDs")
    
    # Find overlapping creator (painter) GNDs
    print(f"\n🎨 Checking {len(CBDD_CREATOR_GNDS):,} creator GNDs...")
    shared_creator_gnds = find_bildindex_overlapping_gnds(CBDD_CREATOR_GNDS, 'creator')
    print(f"   ✓ Found {len(shared_creator_gnds):,} shared creator GNDs")
    
    # Store for later use
    SHARED_LOCATION_GNDS = list(shared_location_gnds)
    SHARED_CREATOR_GNDS = list(shared_creator_gnds)
    
    print(f"\n📊 Overlap Summary:")
    print(f"   Shared building GNDs: {len(SHARED_LOCATION_GNDS):,}")
    print(f"   Shared painter GNDs: {len(SHARED_CREATOR_GNDS):,}")

BILDINDEX INTEGRATION - Step 2: Finding Overlapping GNDs

🏛️ Checking 206 location GNDs...
   ✓ Found 69 shared location GNDs

🎨 Checking 1,074 creator GNDs...
   Progress: 10/22 batches, 167 overlaps found
   Progress: 20/22 batches, 337 overlaps found
   ✓ Found 364 shared creator GNDs

📊 Overlap Summary:
   Shared building GNDs: 69
   Shared painter GNDs: 364


In [67]:
# =============================================================================
# Step 6.3: Bildindex Data Collection Functions
# =============================================================================
# Tiered data collection from Bildindex using shared GNDs

import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuration
BILDINDEX_TEST_MODE = False  # Set to False for full collection
BILDINDEX_MAX_ITEMS = 1000 if BILDINDEX_TEST_MODE else 15000
BILDINDEX_TIER1_RATIO = 0.3  # ~30% from building connections
BILDINDEX_TIER2_RATIO = 0.7  # ~70% from painter connections

def validate_bildindex_url(url, timeout=10):
    """
    Validate a Bildindex URL by checking for soft 404 indicators.
    Returns tuple: (url, is_valid, status_message)
    """
    if not url or pd.isna(url):
        return (url, False, "Empty URL")
    
    try:
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        response = urllib.request.urlopen(req, timeout=timeout)
        content = response.read().decode('utf-8', errors='ignore')
        
        # Soft 404 indicators
        error_indicators = [
            'kein treffer', 'keine treffer', 'no results',
            'nicht gefunden', 'objekt nicht vorhanden', 'seite nicht gefunden'
        ]
        
        content_lower = content.lower()
        for indicator in error_indicators:
            if indicator in content_lower:
                if len(content) > 5000 and 'bildindex' in content_lower:
                    return (url, True, "Valid (has content)")
                return (url, False, f"Soft 404: {indicator}")
        
        return (url, True, "Valid")
        
    except urllib.error.HTTPError as e:
        return (url, False, f"HTTP {e.code}")
    except urllib.error.URLError as e:
        return (url, False, f"URL Error: {str(e.reason)[:50]}")
    except Exception as e:
        return (url, False, f"Error: {str(e)[:50]}")

def validate_urls_batch(urls, max_workers=5, delay=0.2):
    """Validate multiple URLs with rate limiting."""
    results = []
    total = len(urls)
    
    print(f"🔍 Validating {total} URLs...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(validate_bildindex_url, url): url for url in urls}
        
        for i, future in enumerate(as_completed(future_to_url), 1):
            result = future.result()
            results.append({'url': result[0], 'is_valid': result[1], 'status': result[2]})
            
            if i % 50 == 0:
                print(f"   Progress: {i}/{total} ({100*i/total:.1f}%)")
            time.sleep(delay)
    
    df_results = pd.DataFrame(results)
    valid_count = df_results['is_valid'].sum()
    print(f"✅ Validation: {valid_count}/{total} valid URLs ({100*valid_count/total:.1f}%)")
    
    return df_results

def fetch_bildindex_by_building_gnd(building_gnds, limit_per_building=50, total_limit=None):
    """
    Fetch Bildindex items located at buildings with given GNDs.
    Uses pattern: locNode -> any predicate -> GND (discovered in prototype)
    """
    all_items = []
    total_fetched = 0
    
    for i, gnd in enumerate(building_gnds, 1):
        if total_limit and total_fetched >= total_limit:
            break
            
        query = f"""
        {PREFIXES}
        SELECT DISTINCT ?item ?label ?url ?creatorGnd ?creatorLabel ?iconclass
        WHERE {{
            n4c:E6161 schema:dataFeedElement ?feedItem .
            ?feedItem schema:item ?item .
            
            # Location linked via any predicate to this GND
            ?item cto:CTO_0001011 ?locNode .
            ?locNode ?locPred <{gnd}> .
            
            OPTIONAL {{ ?item rdfs:label ?label }}
            OPTIONAL {{ ?item nfdicore:NFDI_0001008 ?url }}
            OPTIONAL {{ 
                ?item cto:CTO_0001009 ?creatorNode .
                ?creatorNode nfdicore:NFDI_0001006 ?creatorGnd .
                OPTIONAL {{ ?creatorNode rdfs:label ?creatorLabel }}
            }}
            OPTIONAL {{ ?item cto:CTO_0001026 ?iconclass }}
        }}
        LIMIT {limit_per_building}
        """
        
        try:
            df_result = run_sparql(query)
            if not df_result.empty:
                df_result['source_building_gnd'] = gnd
                df_result['tier'] = 1
                all_items.append(df_result)
                total_fetched += len(df_result)
                
            if i % 20 == 0:
                print(f"   Buildings: {i}/{len(building_gnds)}, items: {total_fetched:,}")
                
        except Exception as e:
            continue
    
    if all_items:
        df = pd.concat(all_items, ignore_index=True)
        return df.drop_duplicates(subset=['item'])
    return pd.DataFrame()

def fetch_bildindex_by_painter_gnd(painter_gnds, limit_per_painter=30, total_limit=None):
    """
    Fetch Bildindex items created by painters with given GNDs.
    Uses pattern: creatorNode -> any predicate -> GND
    """
    all_items = []
    total_fetched = 0
    
    for i, gnd in enumerate(painter_gnds, 1):
        if total_limit and total_fetched >= total_limit:
            break
            
        query = f"""
        {PREFIXES}
        SELECT DISTINCT ?item ?label ?url ?locationGnd ?locationLabel ?iconclass
        WHERE {{
            n4c:E6161 schema:dataFeedElement ?feedItem .
            ?feedItem schema:item ?item .
            
            # Creator linked via any predicate to this GND
            ?item cto:CTO_0001009 ?creatorNode .
            ?creatorNode ?creatorPred <{gnd}> .
            
            OPTIONAL {{ ?item rdfs:label ?label }}
            OPTIONAL {{ ?item nfdicore:NFDI_0001008 ?url }}
            OPTIONAL {{ 
                ?item cto:CTO_0001011 ?locNode .
                ?locNode nfdicore:NFDI_0001006 ?locationGnd .
                OPTIONAL {{ ?locNode rdfs:label ?locationLabel }}
            }}
            OPTIONAL {{ ?item cto:CTO_0001026 ?iconclass }}
        }}
        LIMIT {limit_per_painter}
        """
        
        try:
            df_result = run_sparql(query)
            if not df_result.empty:
                df_result['source_painter_gnd'] = gnd
                df_result['tier'] = 2
                all_items.append(df_result)
                total_fetched += len(df_result)
                
            if i % 50 == 0:
                print(f"   Painters: {i}/{len(painter_gnds)}, items: {total_fetched:,}")
                
        except Exception:
            continue
    
    if all_items:
        df = pd.concat(all_items, ignore_index=True)
        return df.drop_duplicates(subset=['item'])
    return pd.DataFrame()

print("✅ Bildindex collection functions defined:")
print(f"   Mode: {'TEST' if BILDINDEX_TEST_MODE else 'PRODUCTION'}")
print(f"   Max items: {BILDINDEX_MAX_ITEMS:,}")
print(f"   - fetch_bildindex_by_building_gnd(gnds, limit_per_building, total_limit)")
print(f"   - fetch_bildindex_by_painter_gnd(gnds, limit_per_painter, total_limit)")
print(f"   - validate_urls_batch(urls, max_workers, delay)")

✅ Bildindex collection functions defined:
   Mode: PRODUCTION
   Max items: 15,000
   - fetch_bildindex_by_building_gnd(gnds, limit_per_building, total_limit)
   - fetch_bildindex_by_painter_gnd(gnds, limit_per_painter, total_limit)
   - validate_urls_batch(urls, max_workers, delay)


In [68]:
# =============================================================================
# Step 6.4: Execute Bildindex Data Collection (Tiered)
# =============================================================================

print("=" * 70)
print("BILDINDEX INTEGRATION - Step 4: Data Collection")
print("=" * 70)

# Check prerequisites
if 'SHARED_LOCATION_GNDS' not in dir() or 'SHARED_CREATOR_GNDS' not in dir():
    print("⚠ Run Step 6.2 first to find overlapping GNDs")
else:
    # Calculate limits per tier
    tier1_limit = int(BILDINDEX_MAX_ITEMS * BILDINDEX_TIER1_RATIO)
    tier2_limit = int(BILDINDEX_MAX_ITEMS * BILDINDEX_TIER2_RATIO)
    
    print(f"📊 Collection Plan:")
    print(f"   Tier 1 (buildings): {len(SHARED_LOCATION_GNDS):,} GNDs, limit {tier1_limit:,} items")
    print(f"   Tier 2 (painters): {len(SHARED_CREATOR_GNDS):,} GNDs, limit {tier2_limit:,} items")
    
    # Tier 1: Fetch by building GND
    print(f"\n🏛️ Tier 1: Fetching items from {len(SHARED_LOCATION_GNDS):,} shared buildings...")
    df_bi_tier1 = fetch_bildindex_by_building_gnd(
        SHARED_LOCATION_GNDS, 
        limit_per_building=30, 
        total_limit=tier1_limit
    )
    print(f"   ✓ Tier 1 complete: {len(df_bi_tier1):,} unique items")
    
    # Tier 2: Fetch by painter GND
    print(f"\n🎨 Tier 2: Fetching items from {len(SHARED_CREATOR_GNDS):,} shared painters...")
    # In test mode, limit painters processed
    painters_to_use = SHARED_CREATOR_GNDS[:100] if BILDINDEX_TEST_MODE else SHARED_CREATOR_GNDS
    df_bi_tier2 = fetch_bildindex_by_painter_gnd(
        painters_to_use,
        limit_per_painter=20,
        total_limit=tier2_limit
    )
    print(f"   ✓ Tier 2 complete: {len(df_bi_tier2):,} unique items")
    
    # Combine and deduplicate
    df_bildindex_raw = pd.concat([df_bi_tier1, df_bi_tier2], ignore_index=True)
    df_bildindex_raw = df_bildindex_raw.drop_duplicates(subset=['item'])
    
    # The item URI IS the Bildindex URL (discovered in prototype)
    if not df_bildindex_raw.empty and 'bildindex.de' in str(df_bildindex_raw['item'].iloc[0]):
        df_bildindex_raw['url'] = df_bildindex_raw['item']
    
    # Add collection method
    df_bildindex_raw['collection_method'] = df_bildindex_raw['tier'].apply(
        lambda t: 'building_gnd' if t == 1 else 'painter_gnd'
    )
    
    print(f"\n📊 Collection Summary:")
    print(f"   Total unique items: {len(df_bildindex_raw):,}")
    print(f"   From Tier 1: {len(df_bi_tier1):,}")
    print(f"   From Tier 2: {len(df_bi_tier2):,}")
    print(f"   Overlap (deduplicated): {len(df_bi_tier1) + len(df_bi_tier2) - len(df_bildindex_raw):,}")
    
    if not df_bildindex_raw.empty:
        print(f"\n📋 Sample Bildindex items:")
        display(df_bildindex_raw[['item', 'label', 'tier', 'collection_method']].head())

BILDINDEX INTEGRATION - Step 4: Data Collection
📊 Collection Plan:
   Tier 1 (buildings): 69 GNDs, limit 4,500 items
   Tier 2 (painters): 364 GNDs, limit 10,500 items

🏛️ Tier 1: Fetching items from 69 shared buildings...
   Buildings: 20/69, items: 547
   Buildings: 40/69, items: 1,058
   Buildings: 60/69, items: 1,512
   ✓ Tier 1 complete: 318 unique items

🎨 Tier 2: Fetching items from 364 shared painters...
   Painters: 50/364, items: 734
   Painters: 100/364, items: 1,553
   Painters: 150/364, items: 2,417
   Painters: 200/364, items: 3,326
   Painters: 250/364, items: 4,142
   Painters: 300/364, items: 4,882
   Painters: 350/364, items: 5,682
   ✓ Tier 2 complete: 1,242 unique items

📊 Collection Summary:
   Total unique items: 1,483
   From Tier 1: 318
   From Tier 2: 1,242
   Overlap (deduplicated): 77

📋 Sample Bildindex items:


,item,label,tier,collection_method
0,http://www.bildindex.de/document/obj20117750,Deckendekoration des Festsaals – Triumph der W...,1,building_gnd
1,http://www.bildindex.de/document/obj21014032,Wanddekoration im Vorraum zum Damensalon,1,building_gnd
2,http://www.bildindex.de/document/obj21014033,Wanddekoration im Vorraum zum Damensalon – Aen...,1,building_gnd
3,http://www.bildindex.de/document/obj21014036,Wanddekoration im Vorraum zum Damensalon – Uml...,1,building_gnd
4,http://www.bildindex.de/document/obj21014038,Wanddekoration im Vorraum zum Damensalon – Sup...,1,building_gnd


In [69]:
# =============================================================================
# Step 6.5: URL Validation
# =============================================================================
# Validate Bildindex URLs to filter out dead links (soft 404s)

print("=" * 70)
print("BILDINDEX INTEGRATION - Step 5: URL Validation")
print("=" * 70)

if 'df_bildindex_raw' not in dir() or df_bildindex_raw.empty:
    print("⚠ Run Step 6.4 first to collect Bildindex data")
else:
    # Get URLs to validate
    urls_to_validate = df_bildindex_raw['url'].dropna().unique().tolist()
    
    print(f"🔍 Validating {len(urls_to_validate)} unique URLs...")
    print("   (Rate limited to avoid server overload)")
    
    # Validate (sample in test mode for speed)
    if BILDINDEX_TEST_MODE:
        sample_size = min(len(urls_to_validate), 100)
        urls_sample = urls_to_validate[:sample_size]
        df_url_validation = validate_urls_batch(urls_sample, max_workers=3, delay=0.3)
    else:
        df_url_validation = validate_urls_batch(urls_to_validate, max_workers=5, delay=0.2)
    
    # Get valid URL set
    valid_urls = set(df_url_validation[df_url_validation['is_valid']]['url'].tolist())
    validated_urls = set(df_url_validation['url'].tolist())
    
    # Add validation status to main dataframe
    df_bildindex_raw['url_validated'] = df_bildindex_raw['url'].apply(
        lambda x: x in valid_urls if pd.notna(x) else False
    )
    df_bildindex_raw['validation_status'] = df_bildindex_raw['url'].apply(
        lambda x: 'valid' if x in valid_urls 
                  else ('invalid' if x in validated_urls else 'not_tested')
    )
    
    # Filter to validated items (valid or not yet tested)
    df_bildindex_validated = df_bildindex_raw[
        df_bildindex_raw['url_validated'] | (df_bildindex_raw['validation_status'] == 'not_tested')
    ].copy()
    
    print(f"\n📊 Validation Summary:")
    print(f"   Total collected: {len(df_bildindex_raw):,}")
    print(f"   Validated as working: {df_bildindex_raw['url_validated'].sum():,}")
    print(f"   Not tested (assumed ok): {(df_bildindex_raw['validation_status'] == 'not_tested').sum():,}")
    print(f"   Invalid (filtered out): {(df_bildindex_raw['validation_status'] == 'invalid').sum():,}")
    print(f"   Final dataset size: {len(df_bildindex_validated):,}")
    
    # Show validation status breakdown
    print("\n📋 Validation Status Breakdown:")
    display(df_url_validation['status'].value_counts().head(10))

BILDINDEX INTEGRATION - Step 5: URL Validation
🔍 Validating 1483 unique URLs...
   (Rate limited to avoid server overload)
🔍 Validating 1483 URLs...
   Progress: 50/1483 (3.4%)
   Progress: 100/1483 (6.7%)
   Progress: 150/1483 (10.1%)
   Progress: 200/1483 (13.5%)
   Progress: 250/1483 (16.9%)
   Progress: 300/1483 (20.2%)
   Progress: 350/1483 (23.6%)
   Progress: 400/1483 (27.0%)
   Progress: 450/1483 (30.3%)
   Progress: 500/1483 (33.7%)
   Progress: 550/1483 (37.1%)
   Progress: 600/1483 (40.5%)
   Progress: 650/1483 (43.8%)
   Progress: 700/1483 (47.2%)
   Progress: 750/1483 (50.6%)
   Progress: 800/1483 (53.9%)
   Progress: 850/1483 (57.3%)
   Progress: 900/1483 (60.7%)
   Progress: 950/1483 (64.1%)
   Progress: 1000/1483 (67.4%)
   Progress: 1050/1483 (70.8%)
   Progress: 1100/1483 (74.2%)
   Progress: 1150/1483 (77.5%)
   Progress: 1200/1483 (80.9%)
   Progress: 1250/1483 (84.3%)
   Progress: 1300/1483 (87.7%)
   Progress: 1350/1483 (91.0%)
   Progress: 1400/1483 (94.4%)
   Pr

status
Valid                                  968
URL Error: timed out                   513
Error: The read operation timed out      2
Name: count, dtype: int64

In [70]:
# =============================================================================
# Step 6.6: Export Bildindex Data to Parquet
# =============================================================================
# Create normalized tables and export to parquet files

print("=" * 70)
print("BILDINDEX INTEGRATION - Step 6: Parquet Export")
print("=" * 70)

if 'df_bildindex_validated' not in dir() or df_bildindex_validated.empty:
    print("⚠ Run Steps 6.4 and 6.5 first to collect and validate Bildindex data")
else:
    # Bildindex-specific parquet prefix
    BILDINDEX_PARQUET_PREFIX = "bildindex_"
    
    def get_bildindex_parquet_path(table_name: str) -> str:
        """Get path for Bildindex parquet file."""
        return os.path.join(PARQUET_OUTPUT_DIR, f"{BILDINDEX_PARQUET_PREFIX}{table_name}.parquet")
    
    def save_bildindex_parquet(df: pd.DataFrame, table_name: str) -> str:
        """Save Bildindex DataFrame to parquet."""
        path = get_bildindex_parquet_path(table_name)
        df.to_parquet(path, index=False, engine='pyarrow')
        print(f"   ✓ Saved {table_name}: {len(df):,} rows → {os.path.basename(path)}")
        return path
    
    # === Create main Bildindex items table ===
    df_bi_items = df_bildindex_validated[['item', 'label', 'url', 'tier', 
                                          'collection_method', 'url_validated', 
                                          'validation_status']].copy()
    df_bi_items = df_bi_items.rename(columns={
        'item': 'bildindex_uri',
        'label': 'title',
        'url': 'bildindex_url'
    })
    df_bi_items['source'] = 'bildindex'
    df_bi_items['n4c_feed'] = 'n4c:E6161'
    
    # === Create Bildindex-buildings junction table ===
    df_bi_buildings = df_bildindex_validated[['item', 'source_building_gnd']].dropna(
        subset=['source_building_gnd']
    ).copy()
    df_bi_buildings = df_bi_buildings.rename(columns={
        'item': 'bildindex_uri',
        'source_building_gnd': 'building_gnd'
    })
    
    # === Create Bildindex-painters junction table ===
    df_bi_painters = df_bildindex_validated[['item', 'source_painter_gnd']].dropna(
        subset=['source_painter_gnd']
    ).copy()
    df_bi_painters = df_bi_painters.rename(columns={
        'item': 'bildindex_uri',
        'source_painter_gnd': 'painter_gnd'
    })
    
    # === Create Bildindex-subjects junction table ===
    df_bi_subjects = df_bildindex_validated[['item', 'iconclass']].dropna(
        subset=['iconclass']
    ).copy()
    df_bi_subjects = df_bi_subjects.rename(columns={
        'item': 'bildindex_uri',
        'iconclass': 'iconclass_code'
    })
    
    # === Create GND overlap summary table (for cross-referencing) ===
    gnd_overlap_data = []
    
    # Building GND overlaps
    for gnd in SHARED_LOCATION_GNDS:
        cbdd_count = df_cbdd_paintings[df_cbdd_paintings['locationGnds'].str.contains(gnd, na=False)].shape[0] if 'df_cbdd_paintings' in dir() else 0
        bi_count = df_bi_buildings[df_bi_buildings['building_gnd'] == gnd].shape[0]
        gnd_overlap_data.append({
            'gnd_uri': gnd,
            'gnd_type': 'building',
            'cbdd_count': cbdd_count,
            'bildindex_count': bi_count
        })
    
    # Painter GND overlaps
    for gnd in SHARED_CREATOR_GNDS:
        cbdd_count = df_cbdd_paintings[df_cbdd_paintings['creatorGnds'].str.contains(gnd, na=False)].shape[0] if 'df_cbdd_paintings' in dir() else 0
        bi_count = df_bi_painters[df_bi_painters['painter_gnd'] == gnd].shape[0]
        gnd_overlap_data.append({
            'gnd_uri': gnd,
            'gnd_type': 'painter',
            'cbdd_count': cbdd_count,
            'bildindex_count': bi_count
        })
    
    df_gnd_overlaps = pd.DataFrame(gnd_overlap_data)
    
    # === Save all tables to parquet ===
    print("\n💾 Saving Bildindex tables to parquet...")
    
    bildindex_tables = {
        'items': df_bi_items,
        'buildings': df_bi_buildings,
        'painters': df_bi_painters,
        'subjects': df_bi_subjects,
        'gnd_overlaps': df_gnd_overlaps,
    }
    
    for table_name, df in bildindex_tables.items():
        save_bildindex_parquet(df, table_name)
    
    # Summary
    print("\n" + "=" * 70)
    print("✅ BILDINDEX EXPORT COMPLETE")
    print("=" * 70)
    print(f"\nOutput directory: {PARQUET_OUTPUT_DIR}")
    print("\nBildindex Tables Created:")
    print("-" * 50)
    for name, df in bildindex_tables.items():
        print(f"   bildindex_{name:15} {len(df):>8,} rows")
    
    total_bi_rows = sum(len(df) for df in bildindex_tables.values())
    print("-" * 50)
    print(f"   {'TOTAL':20} {total_bi_rows:>8,} rows")
    
    print("\n🔗 Cross-reference capability:")
    print(f"   - Join bildindex_buildings.building_gnd → baroque_paintings.locationGnds")
    print(f"   - Join bildindex_painters.painter_gnd → baroque_paintings.creatorGnds")
    print(f"   - Join bildindex_subjects.iconclass_code → baroque_painting_subjects.subject_uri")

BILDINDEX INTEGRATION - Step 6: Parquet Export

💾 Saving Bildindex tables to parquet...
   ✓ Saved items: 968 rows → bildindex_items.parquet
   ✓ Saved buildings: 215 rows → bildindex_buildings.parquet
   ✓ Saved painters: 753 rows → bildindex_painters.parquet
   ✓ Saved subjects: 964 rows → bildindex_subjects.parquet
   ✓ Saved gnd_overlaps: 433 rows → bildindex_gnd_overlaps.parquet

✅ BILDINDEX EXPORT COMPLETE

Output directory: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories

Bildindex Tables Created:
--------------------------------------------------
   bildindex_items                968 rows
   bildindex_buildings            215 rows
   bildindex_painters             753 rows
   bildindex_subjects             964 rows
   bildindex_gnd_overlaps         433 rows
--------------------------------------------------
   TOTAL                   3,333 rows

🔗 Cross-reference capability:
   - Join bildindex_buildings.building_gnd → baroque_paintings.locationGnds
   - Join bild

### Bildindex Integration Schema

The Bildindex data is stored in separate parquet tables with the `bildindex_` prefix:

| Table | Primary Key | Description |
|-------|-------------|-------------|
| `bildindex_items` | `bildindex_uri` | Main items table with URLs and metadata |
| `bildindex_buildings` | `bildindex_uri`, `building_gnd` | Junction: items ↔ buildings via GND |
| `bildindex_painters` | `bildindex_uri`, `painter_gnd` | Junction: items ↔ painters via GND |
| `bildindex_subjects` | `bildindex_uri`, `iconclass_code` | Junction: items ↔ ICONCLASS subjects |
| `bildindex_gnd_overlaps` | `gnd_uri` | Summary of shared GNDs between CbDD and Bildindex |

### Cross-Reference Queries

**Find Bildindex images for a CbDD building:**
```python
building_gnd = 'https://d-nb.info/gnd/123456'
bi_items = bildindex_buildings[bildindex_buildings['building_gnd'] == building_gnd]
```

**Find CbDD paintings by a Bildindex painter:**
```python
painter_gnd = 'https://d-nb.info/gnd/789012'  
cbdd_paintings = paintings[paintings['creatorGnds'].str.contains(painter_gnd, na=False)]
```

**Join CbDD and Bildindex via shared subject:**
```python
iconclass_code = 'http://iconclass.org/73B'
cbdd_with_subject = painting_subjects[painting_subjects['subject_uri'].str.contains('73B')]
bi_with_subject = bildindex_subjects[bildindex_subjects['iconclass_code'].str.contains('73B')]
```